```c++
common_params params;
--> common_params_parse;
common_init();
llama_backend_init();
llama_numa_init(params.numa);
// load the model and apply lora adapter, if any
--> common_init_from_params;
llama_model * model = llama_init.model.get();
llama_context * ctx = llama_init.context.get();
struct results_perplexity results;
--> perplexity;
llama_perf_context_print(ctx);
--> llama_backend_free --> ggml_quantize_free;
```

# [`common_params_parse`](https://github.com/ggml-org/llama.cpp/blob/master/common/arg.cpp#L1183)

```c++
common_params_parse(argc, argv, params, LLAMA_EXAMPLE_PERPLEXITY);
----------
bool common_params_parse(int argc, char ** argv, common_params & params, llama_example ex, void(*print_usage)(int, char **)) {}
```

# [`common_init_from_params`](https://github.com/ggml-org/llama.cpp/blob/master/common/common.cpp#L888)

```c++
common_init_result llama_init = common_init_from_params(params);
----------
struct common_init_result common_init_from_params(common_params & params) {}
```

<details>
<summary>struct common_init_result</summary>

```c++
// note: defines object's lifetime
struct common_init_result {
    llama_model_ptr   model;
    llama_context_ptr context;

    std::vector<llama_adapter_lora_ptr> lora;
};
```

</details>

```c++
common_init_result iparams;

auto mparams = common_model_params_to_llama(params);
--> llama_model_load_from_file --> llama_model_load_from_file_impl --> llama_model_load --> llama_model * model;
auto cparams = common_context_params_to_llama(params);
--> llama_init_from_model --> llama_context::llama_context --> llama_context * lctx;
llama_set_warmup(lctx, warmup=true); --> cparams.warmup = warmup;
std::vector<llama_token> tmp;
llama_token bos = llama_vocab_bos(vocab);
llama_token eos = llama_vocab_eos(vocab);
tmp.push_back(bos);
tmp.push_back(eos);
--> llama_decode;
llama_memory_clear(llama_get_memory(lctx), true);
// add the evaluation to the stats
llama_synchronize(lctx); --> lctx->synchronize; --> llama_context::synchronize; --> ggml_backend_sched_synchronize;
llama_perf_context_reset(lctx);
llama_set_warmup(lctx, false);

iparams.model.reset(model);
iparams.context.reset(lctx);
return iparams;
```

## [`llama_model_load`](https://github.com/ggml-org/llama.cpp/blob/master/src/llama.cpp#L87)

```c++
llama_model * model = llama_model_load_from_file(params.model.path.c_str(), mparams);
----------
struct llama_model * llama_model_load_from_file(
    const char * path_model,
    struct llama_model_params params) {
    std::vector<std::string> splits = {};
    --> return llama_model_load_from_file_impl(path_model, splits, params);
}

static struct llama_model * llama_model_load_from_file_impl(
        const std::string & path_model,
        std::vector<std::string> & splits,
        struct llama_model_params params) {
    llama_model * model = new llama_model(params);
    --> const int status = llama_model_load(path_model, splits, *model, params);
    return model;
}

// Returns 0 on success, -1 on error, and -2 on cancellation via llama_progress_callback
static int llama_model_load(const std::string & fname, std::vector<std::string> & splits, llama_model & model, llama_model_params & params) {}
```

<details>
<summary>struct llama_model</summary>

```c++
struct llama_model {
    llm_type type = LLM_TYPE_UNKNOWN;
    llm_arch arch = LLM_ARCH_UNKNOWN;

    std::string name = "n/a";

    llama_hparams hparams = {};
    llama_vocab   vocab;

    struct ggml_tensor * tok_embd   = nullptr;
    struct ggml_tensor * type_embd  = nullptr;
    struct ggml_tensor * pos_embd   = nullptr;
    struct ggml_tensor * tok_norm   = nullptr;
    struct ggml_tensor * tok_norm_b = nullptr;

    struct ggml_tensor * output_norm     = nullptr;
    struct ggml_tensor * output_norm_b   = nullptr;
    struct ggml_tensor * output          = nullptr;
    struct ggml_tensor * output_b        = nullptr;
    struct ggml_tensor * output_norm_enc = nullptr;

    std::vector<llama_layer> layers;

    llama_model_params params;

    // gguf metadata
    std::unordered_map<std::string, std::string> gguf_kv;

    // list of devices used in this model
    std::vector<ggml_backend_dev_t> devices;

    // for quantize-stats only
    std::vector<std::pair<std::string, struct ggml_tensor *>> tensors_by_name;

    // note: can mutate `cparams`
    // TODO: move this to new llm_arch_model_i interface
    llama_memory_i * create_memory(const llama_memory_params & params, llama_cparams & cparams) const;

    // TODO: move this to new llm_arch_model_i interface
    llm_graph_result_ptr build_graph(
            const llm_graph_params & params,
                       ggml_cgraph * gf,
                    llm_graph_type   type) const;

private:
    struct impl;
    std::unique_ptr<impl> pimpl;
};

```
    
</details>

```c++
--> llama_model_loader::llama_model_loader --> llama_model_loader ml;
ml.print_info();
model.load_arch(ml);
--> llama_model::load_hparams;
--> llama_model::load_vocab --> llama_vocab::load --> llama_vocab::impl::load;
model.load_stats(ml);
model.print_info();
--> llama_model::load_tensors;
return 0;
```

### [`llama_model_loader::llama_model_loader`](https://github.com/ggml-org/llama.cpp/blob/master/src/llama-model-loader.cpp#468)

```c++
llama_model_loader ml(fname, splits, params.use_mmap, params.check_tensors, params.kv_overrides, params.tensor_buft_overrides);
----------
llama_model_loader::llama_model_loader(
        const std::string & fname,
        std::vector<std::string> & splits,
        bool use_mmap,
        bool check_tensors,
        const llama_model_kv_override * param_overrides_p,
        const llama_model_tensor_buft_override * param_tensor_buft_overrides_p) {}
```

```c++
// Load the main GGUF
struct ggml_context * ctx = NULL;
struct gguf_init_params params = {
    /*.no_alloc = */ true,
    /*.ctx      = */ &ctx,
};

llama_model_loader:: gguf_context_ptr meta;
--> gguf_init_from_file --> gguf_init_from_file_impl --> meta;

files.emplace_back(new llama_file(fname.c_str(), "rb"));
contexts.emplace_back(ctx);

for (ggml_tensor * cur = ggml_get_first_tensor(ctx); cur; cur = ggml_get_next_tensor(ctx, cur)) {
    std::string tensor_name = std::string(cur->name);
    n_elements += ggml_nelements(cur);
    n_bytes    += ggml_nbytes(cur);
    weights_map.emplace(tensor_name, llama_tensor_weight(files.back().get(), 0, meta.get(), cur));
}

n_kv      = gguf_get_n_kv(meta.get());
n_tensors = weights_map.size();

fver = (enum llama_fver) gguf_get_version(meta.get());

this->use_mmap = use_mmap;
this->check_tensors = check_tensors;
```

#### [`gguf_init_from_file_impl`](https://github.com/ggml-org/llama.cpp/blob/master/ggml/src/gguf.cpp#319)

```c++
meta.reset(gguf_init_from_file(fname.c_str(), params));

struct gguf_context * gguf_init_from_file(const char * fname, struct gguf_init_params params) {
    FILE * file = ggml_fopen(fname, "rb");
    --> struct gguf_context * result = gguf_init_from_file_impl(file, params);
    fclose(file);
    return result;
}

struct gguf_context * gguf_init_from_file_impl(FILE * file, struct gguf_init_params params) {}
```

<details>
<summary>struct gguf_context</summary>

```c++
struct gguf_context {
    uint32_t version = GGUF_VERSION;

    std::vector<struct gguf_kv> kv;
    std::vector<struct gguf_tensor_info> info;

    size_t alignment = GGUF_DEFAULT_ALIGNMENT;
    size_t offset    = 0; // offset of `data` from beginning of file
    size_t size      = 0; // size of `data` in bytes

    void * data = nullptr;
};
```
    
</details>

```c++
const struct gguf_reader gr(file);
struct gguf_context * ctx = new gguf_context;
// file magic
gr.read(magic, 4);
// header
gr.read(ctx->version);
gr.read(n_tensors);
gr.read(n_kv);
// KV pairs
for (int64_t i = 0; ok && i < n_kv; ++i)
    gr.read(key);
    gr.read(type);
    is_array = true; gr.read(type); gr.read(n);
    gguf_read_emplace_helper<xxx>    (gr, ctx->kv, key, is_array, n);
const int alignment_idx = gguf_find_key(ctx, GGUF_KEY_GENERAL_ALIGNMENT);
ctx->alignment = alignment_idx == -1 ? GGUF_DEFAULT_ALIGNMENT : gguf_get_val_u32(ctx, alignment_idx);

// read the tensor info
for (int64_t i = 0; ok && i < n_tensors; ++i)
    struct gguf_tensor_info info;
    std::string name; gr.read(name); ggml_set_name(&info.t, name.c_str()); // tensor name
    uint32_t n_dims = -1; gr.read(n_dims); gr.read(info.t.ne); // tensor shape
    gr.read(info.t.type); // tensor type
    // calculate byte offsets given the tensor shape and type
    const size_t  type_size = ggml_type_size(info.t.type);
    const int64_t blck_size = ggml_blck_size(info.t.type);
    info.t.nb;
    gr.read(info.offset); // tensor data offset within buffer
    ctx->info.push_back(info);

// store the current file offset - this is where the data section starts
ctx->offset = ftell(file);

// compute the total size of the data section, taking into account the alignment
ctx->size = 0;
for (size_t i = 0; i < ctx->info.size(); ++i)
    const gguf_tensor_info & ti = ctx->info[i];
    ctx->size += GGML_PAD(ggml_nbytes(&ti.t), ctx->alignment);


// load the tensor data only if requested
// compute the exact size needed for the new ggml_context
const size_t mem_size = n_tensors * ggml_tensor_overhead();

struct ggml_init_params pdata = {
    /*mem_size   =*/ mem_size,
    /*mem_buffer =*/ nullptr,
    /*no_alloc   =*/ params.no_alloc,
};
--> ggml_init --> *params.ctx;
struct ggml_context * ctx_data = *params.ctx;

// create the tensors
for (size_t i = 0; i < ctx->info.size(); ++i) {
    const struct gguf_tensor_info & info = ctx->info[i];
    --> ggml_new_tensor --> ggml_new_tensor_impl --> struct ggml_tensor * cur;
    ggml_set_name(cur, info.t.name);
}
return ctx;
```

##### [`ggml_init`](https://github.com/ggml-org/llama.cpp/blob/master/ggml/src/ggml.c#L1420)

```c++
*params.ctx = ggml_init(pdata);

struct ggml_context * ggml_init(struct ggml_init_params params) {}
```

<details>
<summary>struct ggml_context</summary>

```c++
struct ggml_context {
    size_t mem_size;
    void * mem_buffer;
    bool   mem_buffer_owned;
    bool   no_alloc;

    int    n_objects;

    struct ggml_object * objects_begin;
    struct ggml_object * objects_end;
};

size_t ggml_tensor_overhead(void) {
    return GGML_OBJECT_SIZE + GGML_TENSOR_SIZE;
}
```

</details>


```c++
struct ggml_context * ctx = GGML_MALLOC(sizeof(struct ggml_context));

// allow to call ggml_init with 0 size
if (params.mem_size == 0) {
    params.mem_size = GGML_MEM_ALIGN;
}

const size_t mem_size = params.mem_buffer ? params.mem_size : GGML_PAD(params.mem_size, GGML_MEM_ALIGN);

*ctx = (struct ggml_context) {
    /*.mem_size           =*/ mem_size,
    /*.mem_buffer         =*/ params.mem_buffer ? params.mem_buffer : ggml_aligned_malloc(mem_size),
    /*.mem_buffer_owned   =*/ params.mem_buffer ? false : true,
    /*.no_alloc           =*/ params.no_alloc,
    /*.n_objects          =*/ 0,
    /*.objects_begin      =*/ NULL,
    /*.objects_end        =*/ NULL,
};

return ctx;
```

##### [`ggml_new_tensor_impl`](https://github.com/ggml-org/llama.cpp/blob/master/ggml/src/ggml.c#L1647)

```c++
struct ggml_tensor * cur = ggml_new_tensor(ctx_data, info.t.type, GGML_MAX_DIMS, info.t.ne);

struct ggml_tensor * ggml_new_tensor(
        struct ggml_context * ctx,
        enum   ggml_type      type,
        int                   n_dims,
        const int64_t       * ne) {
    --> return ggml_new_tensor_impl(ctx, type, n_dims, ne, NULL, 0);
}

static struct ggml_tensor * ggml_new_tensor_impl(
        struct ggml_context * ctx,
        enum   ggml_type      type,
        int                   n_dims,
        const int64_t       * ne,
        struct ggml_tensor  * view_src,
        size_t                view_offs) {}
```

<details>
<summary>struct ggml_tensor</summary>

```c++
// n-dimensional tensor
struct ggml_tensor {
    enum ggml_type type;

    struct ggml_backend_buffer * buffer;

    int64_t ne[GGML_MAX_DIMS]; // number of elements
    size_t  nb[GGML_MAX_DIMS]; // stride in bytes:
                               // nb[0] = ggml_type_size(type)
                               // nb[1] = nb[0]   * (ne[0] / ggml_blck_size(type)) + padding
                               // nb[i] = nb[i-1] * ne[i-1]

    // compute data
    enum ggml_op op;

    // op params - allocated as int32_t for alignment
    int32_t op_params[GGML_MAX_OP_PARAMS / sizeof(int32_t)];

    int32_t flags;

    struct ggml_tensor * src[GGML_MAX_SRC];

    // source tensor and offset for views
    struct ggml_tensor * view_src;
    size_t               view_offs;

    void * data;

    char name[GGML_MAX_NAME];

    void * extra; // extra things e.g. for ggml-cuda.cu

    char padding[8];
};

static const size_t GGML_TENSOR_SIZE = sizeof(struct ggml_tensor);
```
</details>


```c++
--> ggml_new_object --> obj_new;
struct ggml_tensor * const result = (struct ggml_tensor *)((char *)ctx->mem_buffer + obj_new->offs);
*result = (struct ggml_tensor) {
    /*.type         =*/ type,
    /*.buffer       =*/ NULL,
    /*.ne           =*/ { 1, 1, 1, 1 },
    /*.nb           =*/ { 0, 0, 0, 0 },
    /*.op           =*/ GGML_OP_NONE,
    /*.op_params    =*/ { 0 },
    /*.flags        =*/ 0,
    /*.src          =*/ { NULL },
    /*.view_src     =*/ view_src,
    /*.view_offs    =*/ view_offs,
    /*.data         =*/ view_src != NULL ? view_src->data + view_offs : NULL,
    /*.name         =*/ { 0 },
    /*.extra        =*/ NULL,
    /*.padding      =*/ { 0 },
};

result->ne;
result->nb;
ctx->n_objects++;
return result;
```

[`ggml_new_object`](https://github.com/ggml-org/llama.cpp/blob/master/ggml/src/ggml.c#L1525)

```c++
struct ggml_object * const obj_new = ggml_new_object(ctx, GGML_OBJECT_TYPE_TENSOR, GGML_TENSOR_SIZE);

static struct ggml_object * ggml_new_object(struct ggml_context * ctx, enum ggml_object_type type, size_t size) {}
```

<details>
<summary>struct ggml_object</summary>

```c++
struct ggml_object {
    size_t offs;
    size_t size;

    struct ggml_object * next;

    enum ggml_object_type type;

    char padding[4];
};

static const size_t GGML_OBJECT_SIZE = sizeof(struct ggml_object);
```
</details>


```c++
// always insert objects at the end of the context's memory pool
struct ggml_object * obj_cur = ctx->objects_end;

const size_t cur_offs = obj_cur == NULL ? 0 : obj_cur->offs;
const size_t cur_size = obj_cur == NULL ? 0 : obj_cur->size;
const size_t cur_end  = cur_offs + cur_size;

// align to GGML_MEM_ALIGN
size_t size_needed = GGML_PAD(size, GGML_MEM_ALIGN);

char * const mem_buffer = ctx->mem_buffer;
struct ggml_object * const obj_new = (struct ggml_object *)(mem_buffer + cur_end);

*obj_new = (struct ggml_object) {
    .offs = cur_end + GGML_OBJECT_SIZE,
    .size = size_needed,
    .next = NULL,
    .type = type,
};

GGML_ASSERT_ALIGNED(mem_buffer + obj_new->offs);

if (obj_cur != NULL) {
    obj_cur->next = obj_new;
} else {
    // this is the first object in this context
    ctx->objects_begin = obj_new;
}

ctx->objects_end = obj_new;
return obj_new;
```

### [`llama_model::load_hparams`](https://github.com/ggml-org/llama.cpp/blob/master/src/llama-model.cpp#423)

```c++
model.load_hparams(ml);
----------
void llama_model::load_hparams(llama_model_loader & ml) {}
```

```c++
const gguf_context * ctx = ml.meta.get();
// get metadata as string
// gguf metadata
llama_model:: std::unordered_map<std::string, std::string> gguf_kv;
gguf_kv.emplace(name, value);
hparams.xxx = xxx; // via ml.get_key
pimpl->n_bytes = ml.n_bytes;
pimpl->desc_str = arch_name() + " " + type_name() + " " + ml.ftype_name();
```

### [`llama_vocab::impl::load`](https://github.com/ggml-org/llama.cpp/blob/master/src/llama-vocab.cpp#1372)

```c++
model.load_vocab(ml);
----------
void llama_model::load_vocab(llama_model_loader & ml) {
    const auto kv = LLM_KV(arch);
    --> vocab.load(ml, kv);
}

void llama_vocab::load(llama_model_loader & ml, const LLM_KV & kv) {
    --> pimpl->load(ml, kv);
}

void llama_vocab::impl::load(llama_model_loader & ml, const LLM_KV & kv) {}
```

```c++
struct gguf_context * ctx = ml.meta.get();
// determine vocab type
ml.get_key(LLM_KV_TOKENIZER_MODEL, tokenizer_model);
ml.get_key(LLM_KV_TOKENIZER_PRE,   tokenizer_pre, false);
ml.get_key(LLM_KV_TOKENIZER_TOKEN_TYPE_COUNT, n_token_types, false);
// for now, only BPE models have pre-tokenizers
llama_vocab::impl::
    std::unordered_map<std::string, llama_token> token_to_id;
    std::vector<token_data> id_to_token;
--> init_tokenizer(type); --> tokenizer = std::make_unique<llm_tokenizer_bpe>(vocab);
// determine the newline token: LLaMA "<0x0A>" == 10 == '\n', Falcon 193 == '\n
// special tokens
    std::set<llama_token> special_eog_ids; // set of all tokens that cause "end of generation"
// build special tokens cache
    std::vector<llama_token> cache_special_tokens;
// build token to piece cache
    std::vector<std::string> cache_token_to_piece; // llama_token_to_piece(special = true);
```

### [`llama_model::load_tensors`](https://github.com/ggml-org/llama.cpp/blob/master/src/llama-model.cpp#1467)

```c++
model.load_tensors(ml);
----------
bool llama_model::load_tensors(llama_model_loader & ml) {}
```

```c++
// build a list of buffer types for the CPU and GPU devices
pimpl->cpu_buft_list = make_cpu_buft_list(devices);
// calculate the split points
std::vector<float> splits(n_devices());
std::copy(tensor_split, tensor_split + n_devices(), splits.begin());
// sum and normalize the splits to get the split points
ggml_backend_dev_t cpu_dev = ggml_backend_dev_by_type(GGML_BACKEND_DEVICE_TYPE_CPU);
auto get_layer_buft_list = [&](int il) -> llama_model::impl::layer_dev {}
// assign the input layer, there is very little benefit to offloading the input layer, so always keep it on the CPU
pimpl->dev_input = { cpu_dev, &pimpl->cpu_buft_list };
// assign the repeating layers to the devices according to the splits
pimpl->dev_layer.resize(n_layer);
for (int il = 0; il < n_layer; ++il) pimpl->dev_layer[il] = get_layer_buft_list(il);
// assign the output layer
pimpl->dev_output = get_layer_buft_list(n_layer);
// one ggml context per buffer type
int max_n_tensors = ml.n_tensors;
max_n_tensors += 1;         // duplicated output tensor
max_n_tensors += n_layer*2; // duplicated rope freq tensors
const size_t ctx_size = ggml_tensor_overhead()*max_n_tensors;
std::map<ggml_backend_buffer_type_t, ggml_context *> ctx_map;
auto ctx_for_buft = [&](ggml_backend_buffer_type_t buft) -> ggml_context * {
    auto it = ctx_map.find(buft);
    if (it == ctx_map.end()) {
        ggml_init_params params = {
            /*.mem_size   =*/ ctx_size,
            /*.mem_buffer =*/ NULL,
            /*.no_alloc   =*/ true,
        };
        ggml_context * ctx = ggml_init(params);
        ctx_map[buft] = ctx;
        pimpl->ctxs.emplace_back(ctx);
        return ctx;
    }
    return it->second;
};
// create tensors for the weights
auto create_tensor = [&](const LLM_TN_IMPL & tn, const std::initializer_list<int64_t> & ne, int flags) -> ggml_tensor * {
    ggml_tensor * t_meta = ml.get_tensor_meta(tn.str().c_str());
    llm_tensor_info info = llm_tensor_info_for(tn_tensor);
    // select the buffer type for this tensor
    switch (info.layer) {}
    --> select_weight_buft --> weight_buft_supported --> buft;
    ggml_context * ctx = ctx_for_buft(buft);
    --> return llama_model_loader::create_tensor;
}
```

```c++
layers.resize(n_layer);
// TODO: move to a separate function
const auto tn = LLM_TN(arch);

tok_embd = create_tensor(tn(LLM_TENSOR_TOKEN_EMBD, "weight"), {n_embd, n_vocab}, 0);

// output
output_norm = create_tensor(tn(LLM_TENSOR_OUTPUT_NORM, "weight"), {n_embd}, 0);
output      = create_tensor(tn(LLM_TENSOR_OUTPUT,      "weight"), {n_embd, n_vocab}, TENSOR_NOT_REQUIRED);

// if output is NULL, init from the input tok embed
if (output == NULL)
    output = create_tensor(tn(LLM_TENSOR_TOKEN_EMBD, "weight"), {n_embd, n_vocab}, TENSOR_DUPLICATED);

for (int i = 0; i < n_layer; ++i)
    auto & layer = layers[i];

    layer.attn_norm = create_tensor(tn(LLM_TENSOR_ATTN_NORM, "weight", i), {n_embd}, 0);

    layer.wq = create_tensor(tn(LLM_TENSOR_ATTN_Q,   "weight", i), {n_embd, n_embd_head_k * n_head}, 0);
    layer.wk = create_tensor(tn(LLM_TENSOR_ATTN_K,   "weight", i), {n_embd, n_embd_k_gqa}, 0);
    layer.wv = create_tensor(tn(LLM_TENSOR_ATTN_V,   "weight", i), {n_embd, n_embd_v_gqa}, 0);
    layer.wo = create_tensor(tn(LLM_TENSOR_ATTN_OUT, "weight", i), {n_embd_head_k * n_head, n_embd}, 0);

    // optional bias tensors
    layer.bq = create_tensor(tn(LLM_TENSOR_ATTN_Q,   "bias", i), {n_embd},     TENSOR_NOT_REQUIRED);
    layer.bk = create_tensor(tn(LLM_TENSOR_ATTN_K,   "bias", i), {n_embd_gqa}, TENSOR_NOT_REQUIRED);
    layer.bv = create_tensor(tn(LLM_TENSOR_ATTN_V,   "bias", i), {n_embd_gqa}, TENSOR_NOT_REQUIRED);
    layer.bo = create_tensor(tn(LLM_TENSOR_ATTN_OUT, "bias", i), {n_embd},     TENSOR_NOT_REQUIRED);

    layer.ffn_norm = create_tensor(tn(LLM_TENSOR_FFN_NORM, "weight", i), {n_embd}, 0);

    layer.ffn_gate = create_tensor(tn(LLM_TENSOR_FFN_GATE, "weight", i), {n_embd,   n_ff}, 0);
    layer.ffn_down = create_tensor(tn(LLM_TENSOR_FFN_DOWN, "weight", i), {  n_ff, n_embd}, 0);
    layer.ffn_up   = create_tensor(tn(LLM_TENSOR_FFN_UP,   "weight", i), {n_embd,   n_ff}, 0);

    // optional MLP bias
    layer.ffn_gate_b = create_tensor(tn(LLM_TENSOR_FFN_GATE, "bias", i), {n_ff}, TENSOR_NOT_REQUIRED);
    layer.ffn_down_b = create_tensor(tn(LLM_TENSOR_FFN_DOWN, "bias", i), {n_embd}, TENSOR_NOT_REQUIRED);
    layer.ffn_up_b   = create_tensor(tn(LLM_TENSOR_FFN_UP,   "bias", i), {n_ff}, TENSOR_NOT_REQUIRED);
```

```c++
ml.done_getting_tensors();
--> llama_model_loader::init_mappings --> ml.mappings;
pimpl->mappings.reserve(ml.mappings.size());
for (auto & mapping : ml.mappings)
    pimpl->mappings.emplace_back(std::move(mapping));

// create the backend buffers
using llama_buf_map = std::unordered_map<uint32_t, ggml_backend_buffer_t>;
std::vector<std::pair<ggml_context *, llama_buf_map>> ctx_bufs;
ctx_bufs.reserve(ctx_map.size());

// Ensure we have enough capacity for the maximum backend buffer we will potentially create
pimpl->bufs.reserve(ctx_map.size());

for (auto & it : ctx_map)
    ggml_context * ctx = it.second;
    llama_buf_map buf_map;
    buf_map.reserve(1); // ml.files.size()
    //if (ml.use_mmap && use_mmap_buffer && buffer_from_host_ptr_supported && is_default_buft)
    //for (uint32_t idx = 0; idx < ml.files.size(); idx++)
    // only the mmap region containing the tensors in the model is mapped to the backend buffer
    // this is important for metal with apple silicon: if the entire model could be mapped to a metal buffer, then we could just use metal for all layers
    // this allows using partial offloading when the model size exceeds the metal buffer size, but not the RAM size
    void * addr = nullptr;
    size_t first, last; // NOLINT
    --> llama_model_loader::get_mapping_range;
    --> ggml_backend_dev_buffer_from_host_ptr --> ggml_backend_cpu_buffer_from_ptr --> ggml_backend_buffer_t buf;
    // indicate that this buffer contains weights, this is used by ggml_backend_sched to improve op scheduling: ops that use a weight are preferably scheduled to the backend that contains the weight
    ggml_backend_buffer_set_usage(buf, GGML_BACKEND_BUFFER_USAGE_WEIGHTS);
    pimpl->bufs.emplace_back(buf);
    buf_map.emplace(0, buf);

    ctx_bufs.emplace_back(ctx, buf_map);

// populate tensors_by_name
for (auto & ctx : pimpl->ctxs) for (auto * cur = ggml_get_first_tensor(ctx.get()); cur != NULL; cur = ggml_get_next_tensor(ctx.get(), cur)) tensors_by_name.emplace_back(ggml_get_name(cur), cur);

// load tensor data
for (auto & it : ctx_bufs)
    ggml_context * ctx = it.first;
    auto & bufs = it.second;
    --> llama_model_loader::load_all_data;

return true;
```

#### [`weight_buft_supported`](https://github.com/ggml-org/llama.cpp/blob/master/src/llama-model.cpp#L138)

```c++
ggml_backend_buffer_type_t buft = select_weight_buft(hparams, t_meta, op, *buft_list);
----------
// find the first buffer type in the list that can use the tensor
static ggml_backend_buffer_type_t select_weight_buft(const llama_hparams & hparams, ggml_tensor * tensor, ggml_op op, const buft_list_t & buft_list) {
    for (const auto & cur : buft_list)
        --> if (weight_buft_supported(hparams, tensor, op, cur_buft = cur.second, cur_dev = cur.first)) {
            return cur_buft;
}

// checks if the weight tensor can be used with the specified buffer type and device
static bool weight_buft_supported(const llama_hparams & hparams, ggml_tensor * w, ggml_op op, ggml_backend_buffer_type_t buft, ggml_backend_dev_t dev) {}
```

```c++
ggml_init_params params = {
    /*.mem_size   =*/ ggml_tensor_overhead()*8,
    /*.mem_buffer =*/ NULL,
    /*.no_alloc   =*/ true,
};
ggml_context_ptr ctx_ptr { ggml_init(params) };

ggml_context * ctx = ctx_ptr.get();
ggml_tensor * op_tensor = nullptr;
```

```c++
switch (op) {
    case GGML_OP_GET_ROWS:
        {
            ggml_tensor * b = ggml_new_tensor_1d(ctx, GGML_TYPE_I32, 512);
            op_tensor = ggml_get_rows(ctx, w, b);
        } break;
    case GGML_OP_MUL_MAT:
        {
            ggml_tensor * b = ggml_new_tensor_4d(ctx, GGML_TYPE_F32, w->ne[0], 512, w->ne[2], w->ne[3]);
            op_tensor = ggml_mul_mat(ctx, w, b);
        } break;
    case GGML_OP_ADD:
        {
            ggml_tensor * a = ggml_new_tensor_4d(ctx, GGML_TYPE_F32, w->ne[0], w->ne[1], w->ne[2], w->ne[3]);
            op_tensor = ggml_add(ctx, a, w);
        } break;
    case GGML_OP_MUL:
        {
            ggml_tensor * a = ggml_new_tensor_4d(ctx, GGML_TYPE_F32, w->ne[0], w->ne[1], w->ne[2], w->ne[3]);
            op_tensor = ggml_mul(ctx, a, w);
        } break;
    case GGML_OP_DIV:
        {
            ggml_tensor * a = ggml_new_tensor_1d(ctx, GGML_TYPE_F32, w->ne[0]);
            op_tensor = ggml_div(ctx, a, w);
        } break;
    case GGML_OP_ROPE:
        {
            int n_embd_head = hparams.n_embd_head_v;
            int n_head = hparams.n_head();
            ggml_tensor * a = ggml_new_tensor_3d(ctx, GGML_TYPE_F32, n_embd_head, n_head, 512);
            ggml_tensor * b = ggml_new_tensor_1d(ctx, GGML_TYPE_I32, 512);
            op_tensor = ggml_rope_ext(
                ctx, a, b, w,
                0, 0, 0, 0, 0,
                0, 0, 0, 0
            );
        } break;
}
```


<details>
<summary>ggml_new_tensor</summary>

```c++
struct ggml_tensor * ggml_new_tensor_1d()
    return ggml_new_tensor(ctx, type, 1, &ne0);

struct ggml_tensor * ggml_new_tensor_2d()
    const int64_t ne[2] = { ne0, ne1};
    return ggml_new_tensor(ctx, type, 2, ne);

struct ggml_tensor * ggml_new_tensor_3d()
    const int64_t ne[3] = { ne0, ne1, ne2 };
    return ggml_new_tensor(ctx, type, 3, ne);

struct ggml_tensor * ggml_new_tensor_4d()
    const int64_t ne[4] = { ne0, ne1, ne2, ne3 };
    return ggml_new_tensor(ctx, type, 4, ne);

struct ggml_tensor * ggml_get_rows()
    struct ggml_tensor * result = ggml_new_tensor_4d(ctx, type, a->ne[0], b->ne[0], b->ne[1], b->ne[2]);
    result->op     = GGML_OP_GET_ROWS;
    result->src[0] = a;
    result->src[1] = b;
    return result;

static struct ggml_tensor * ggml_rope_impl()
    int sections[4] = {0, 0, 0, 0};

    struct ggml_tensor * result = inplace ? ggml_view_tensor(ctx, a) : ggml_dup_tensor(ctx, a);

    int32_t params[15] = { /*n_past*/ 0, n_dims, mode, /*n_ctx*/ 0, n_ctx_orig };
    memcpy(params +  5, &freq_base,    sizeof(float));
    memcpy(params +  6, &freq_scale,   sizeof(float));
    memcpy(params +  7, &ext_factor,   sizeof(float));
    memcpy(params +  8, &attn_factor,  sizeof(float));
    memcpy(params +  9, &beta_fast,    sizeof(float));
    memcpy(params + 10, &beta_slow,    sizeof(float));
    memcpy(params + 11, &sections,     sizeof(int)*4);
    ggml_set_op_params(result, params, sizeof(params));

    result->op     = GGML_OP_ROPE;
    result->src[0] = a;
    result->src[1] = b;
    result->src[2] = c;

    return result;
```
    
</details>

```c++
w->buffer = ggml_backend_buft_alloc_buffer(buft, 0);
bool op_supported = ggml_backend_dev_supports_op(dev, op_tensor);
ggml_backend_buffer_free(w->buffer);
return op_supported;
```

#### [`llama_model_loader::create_tensor`](https://github.com/ggml-org/llama.cpp/blob/master/src/llama-model-loader.cpp#L789)

```c++
return ml.create_tensor(ctx, tn, ne, flags);
----------
struct ggml_tensor * llama_model_loader::create_tensor(struct ggml_context * ctx, const std::string & name, const std::initializer_list<int64_t> & ne, int flags) {}
```

```c++
--> check_tensor_dims --> cur;
bool duplicated = flags & TENSOR_DUPLICATED;
--> ggml_dup_tensor --> tensor;
ggml_set_name(tensor, ggml_get_name(cur));
if (duplicated) {
    size_data += ggml_nbytes(cur);
} else {
    n_created++;
}
return tensor;
```

[`llama_model_loader::check_tensor_dims`](https://github.com/ggml-org/llama.cpp/blob/master/src/llama-model-loader.cpp#L759)

```c++
const struct ggml_tensor * cur = check_tensor_dims(name, ne, !(flags & TENSOR_NOT_REQUIRED));
----------
const struct ggml_tensor * llama_model_loader::check_tensor_dims(const std::string & name, const std::vector<int64_t> & ne, bool required) const {
    const struct ggml_tensor * cur = get_tensor_meta(name.c_str());
    return cur;
}
```

[`ggml_dup_tensor`](https://github.com/ggml-org/llama.cpp/blob/master/ggml/src/ggml.c#L1698)

```c++
struct ggml_tensor * tensor = ggml_dup_tensor(ctx, cur);
----------
struct ggml_tensor * ggml_dup_tensor(struct ggml_context * ctx, const struct ggml_tensor * src) {
    return ggml_new_tensor(ctx, src->type, GGML_MAX_DIMS, src->ne);
}
```

#### [`llama_model_loader::init_mappings`](https://github.com/ggml-org/llama.cpp/blob/master/src/llama-model-loader.cpp#L845)

```c++
ml.init_mappings(true, nullptr);
----------
void llama_model_loader::init_mappings(bool prefetch, llama_mlocks * mlock_mmaps) {}
```

```c++
--> llama_mmap::llama_mmap --> llama_mmap::impl::impl --> mapping;
llama_model_loader:: std::vector<std::pair<size_t, size_t>> mmaps_used;
mmaps_used.emplace_back(mapping->size(), 0);

using llama_mmaps  = std::vector<std::unique_ptr<llama_mmap>>;
llama_model_loader:: llama_mmaps mappings;
mappings.emplace_back(std::move(mapping));
```

[`llama_mmap::llama_mmap`](https://github.com/ggml-org/llama.cpp/blob/master/src/llama-mmap.cpp#L441)

```c++
std::unique_ptr<llama_mmap> mapping = std::make_unique<llama_mmap>(file.get(), prefetch ? -1 : 0, is_numa);
----------
llama_mmap::llama_mmap(struct llama_file * file, size_t prefetch, bool numa) : pimpl(std::make_unique<impl>(file, prefetch, numa)) {}
struct llama_mmap::impl {
    std::vector<std::pair<size_t, size_t>> mapped_fragments;
    impl(struct llama_file * file, size_t prefetch, bool numa) {
        size = file->size();
        int fd = file->file_id();
        int flags = MAP_SHARED;
        if (prefetch) { flags |= MAP_POPULATE; }
        --> addr = mmap(NULL, file->size(), PROT_READ, flags, fd, 0);
        if (prefetch > 0) {
            if (posix_madvise(addr, std::min(file->size(), prefetch), POSIX_MADV_WILLNEED)) {
                LLAMA_LOG_WARN("warning: posix_madvise(.., POSIX_MADV_WILLNEED) failed: %s\n",
                        strerror(errno));
            }
        }
        mapped_fragments.emplace_back(0, file->size());
    }
}
```

#### [`llama_model_loader::get_mapping_range`](https://github.com/ggml-org/llama.cpp/blob/master/src/llama-model-loader.cpp#L878)

```c++
ml.get_mapping_range(&first, &last, &addr, idx=0, ctx);
----------
void llama_model_loader::get_mapping_range(size_t * first, size_t * last, void ** addr, int idx, ggml_context * ctx) const {}
```

```c++
const auto & mapping = mappings.at(idx);

*first = mapping->size();
*last  = 0;
*addr = mapping->addr();
for (ggml_tensor * tensor = ggml_get_first_tensor(ctx); tensor; tensor = ggml_get_next_tensor(ctx, tensor)) {
    const auto * weight = get_weight(ggml_get_name(tensor));
    if (!weight || weight->idx != idx) {
        continue;
    }
    *first = std::min(*first, weight->offs);
    *last  = std::max(*last,  weight->offs + ggml_nbytes(tensor));
}
```

<details>
<summary>struct llama_model_loader::llama_tensor_weight</summary>

```c++
// Holds information on a model weight
struct llama_tensor_weight {
    uint16_t  idx; // source file index
    size_t   offs; // tensor data offset in the original file
    ggml_tensor * tensor;
}

const llama_model_loader::llama_tensor_weight * llama_model_loader::get_weight(const char * name) const {
    auto pos = weights_map.find(name);
    if (pos != weights_map.end()) {
        return &pos->second;
    }

    return nullptr;
}
```

</details>


#### [`ggml_backend_cpu_buffer_from_ptr`](https://github.com/ggml-org/llama.cpp/blob/master/ggml/src/ggml-backend.cpp#L2013)

```c++
ggml_backend_buffer_t buf = ggml_backend_dev_buffer_from_host_ptr(dev, (char *) addr + first, last - first, ggml_get_max_tensor_size(ctx));
----------
ggml_backend_buffer_t ggml_backend_dev_buffer_from_host_ptr(ggml_backend_dev_t device, void * ptr, size_t size, size_t max_tensor_size) {
    --> return device->iface.buffer_from_host_ptr(device, ptr, size, max_tensor_size);
}

static ggml_backend_buffer_t ggml_backend_cpu_device_buffer_from_host_ptr(ggml_backend_dev_t dev, void * ptr, size_t size, size_t max_tensor_size) {
    --> return ggml_backend_cpu_buffer_from_ptr(ptr, size);
}

ggml_backend_buffer_t ggml_backend_cpu_buffer_from_ptr(void * ptr, size_t size) {
    --> return ggml_backend_buffer_init(ggml_backend_cpu_buffer_from_ptr_type(), ggml_backend_cpu_buffer_from_ptr_i, ptr, size);
}
```

[`ggml_backend_buffer_init`](https://github.com/ggml-org/llama.cpp/blob/master/ggml/src/ggml-backend.cpp#L82)

<details>
<summary>struct ggml_backend_buffer</summary>

```c++
struct ggml_backend_buffer {
    struct ggml_backend_buffer_i  iface;
    ggml_backend_buffer_type_t    buft;
    void * context;
    size_t size;
    enum ggml_backend_buffer_usage usage;
};

typedef struct ggml_backend_buffer * ggml_backend_buffer_t;
```

</details>

```c++
// backend buffer

ggml_backend_buffer_t ggml_backend_buffer_init(
               ggml_backend_buffer_type_t buft,
        struct ggml_backend_buffer_i      iface,
               void *                     context,
               size_t                     size) {
    ggml_backend_buffer_t buffer = new ggml_backend_buffer {
        /* .interface = */ iface,
        /* .buft      = */ buft,
        /* .context   = */ context,
        /* .size      = */ size,
        /* .usage     = */ GGML_BACKEND_BUFFER_USAGE_ANY
    };

    return buffer;
}
```

[`ggml_backend_cpu_buffer_from_ptr_type`](https://github.com/ggml-org/llama.cpp/blob/master/ggml/src/ggml-backend.cpp#L1996)

<details>
<summary>struct ggml_backend_buffer_type</summary>
    
```c++
struct ggml_backend_buffer_type {
    struct ggml_backend_buffer_type_i  iface;
    ggml_backend_dev_t device;
    void * context;
};

typedef struct ggml_backend_buffer_type * ggml_backend_buffer_type_t;
```

</details>

```c++

static ggml_backend_buffer_type_t ggml_backend_cpu_buffer_from_ptr_type(void) {
    static struct ggml_backend_buffer_type ggml_backend_cpu_buffer_type = {
        /* .iface   = */ {
            /* .get_name         = */ ggml_backend_cpu_buffer_from_ptr_type_get_name,
            /* .alloc_buffer     = */ ggml_backend_cpu_buffer_type_alloc_buffer,
            /* .get_alignment    = */ ggml_backend_cpu_buffer_type_get_alignment,
            /* .get_max_size     = */ NULL, // defaults to SIZE_MAX
            /* .get_alloc_size   = */ NULL, // defaults to ggml_nbytes
            /* .is_host          = */ ggml_backend_cpu_buffer_type_is_host,
        },
        /* .device  = */ NULL, // FIXME ggml_backend_reg_dev_get(ggml_backend_cpu_reg(), 0),
        /* .context = */ NULL,
    };

    return &ggml_backend_cpu_buffer_type;
}
```

#### [`llama_model_loader::load_all_data`](https://github.com/ggml-org/llama.cpp/blob/master/src/llama-model-loader.cpp#L918)

```c++
ml.load_all_data(ctx, bufs, use_mlock ? &pimpl->mlock_mmaps : NULL, params.progress_callback, params.progress_callback_user_data);
----------
bool llama_model_loader::load_all_data(
        struct ggml_context * ctx,
        llama_buf_map & bufs,
        llama_mlocks * lmlocks,
        llama_progress_callback progress_callback,
        void * progress_callback_user_data) {}
```

```c++
for (struct ggml_tensor * cur = ggml_get_first_tensor(ctx); cur != NULL; cur = ggml_get_next_tensor(ctx, cur))
    const auto * weight = get_weight(ggml_get_name(cur));
    size_t n_size = ggml_nbytes(cur);
    const auto & mapping = mappings.at(weight->idx);
    ggml_backend_buffer_t buf_mmap = bufs.at(weight->idx);
    uint8_t * data = (uint8_t *) mapping->addr() + weight->offs;
    --> ggml_backend_tensor_alloc(buffer=buf_mmap, tensor=cur, addr=data);
        tensor->buffer = buffer;
        tensor->data = addr;
        return ggml_backend_buffer_init_tensor(buffer, tensor);
    auto & mmap_used = mmaps_used[weight->idx];
    mmap_used.first  = std::min(mmap_used.first,  weight->offs);
    mmap_used.second = std::max(mmap_used.second, weight->offs + n_size);
    size_done += n_size;
// check if this is the last call and do final cleanup
for (uint32_t idx = 0; idx < mappings.size(); idx++)
    const auto & mmap_used = mmaps_used.at(idx);
    auto & mapping = mappings.at(idx);
    --> mapping->unmap_fragment(0, mmap_used.first);
    --> if (mmap_used.second != 0) mapping->unmap_fragment(mmap_used.second, mapping->size());
return true;
```

[`llama_mmap::unmap_fragment`](https://github.com/ggml-org/llama.cpp/blob/master/src/llama-mmap.cpp#L447)

```c++
void llama_mmap::unmap_fragment(size_t first, size_t last) { pimpl->unmap_fragment(first, last); }

struct llama_mmap::impl {
    void unmap_fragment(size_t first, size_t last) {
        int page_size = sysconf(_SC_PAGESIZE);
        align_range(&first, &last, page_size);
        size_t len = last - first;
        void * next_page_start = (uint8_t *) addr + first;
        munmap(next_page_start, len);
        for (const auto & frag : mapped_fragments) new_mapped_fragments.emplace_back(last, frag.second);
        mapped_fragments = std::move(new_mapped_fragments);
    }
}
```

## [`llama_context::llama_context`](https://github.com/ggml-org/llama.cpp/blob/master/src/llama-context.cpp#L18)

```c++
llama_context * lctx = llama_init_from_model(model, cparams);
----------
llama_context * llama_init_from_model(
                 llama_model * model,
        llama_context_params   params) {
    --> auto * ctx = new llama_context(*model, params);
    return ctx;
}

llama_context::llama_context(
        const llama_model & model,
              llama_context_params params) :
    model(model),
    batch_allocr(std::make_unique<llama_batch_allocr>()) {}
```

<details>
<summary>struct llama_context</summary>

```c++
struct llama_context {
private:
    const llama_model & model;

    llama_cparams       cparams;

    std::unique_ptr<llama_memory_i> memory;

    // TODO: temporary, until the llama_kv_self_defrag() API is removed
    bool memory_force_optimize = false;

    // decode output (2-dimensional array: [n_outputs][n_vocab])
    size_t  logits_size = 0; // capacity (of floats) for logits
    float * logits      = nullptr;

    // reuse the batch_allocr to avoid unnecessary memory allocations
    std::unique_ptr<llama_batch_allocr> batch_allocr;

    int32_t n_outputs     = 0; // number of actually-used outputs in the current ubatch or last logical batch
    int32_t n_outputs_max = 0; // capacity (of tokens positions) for the output buffers

    std::vector<int32_t> output_ids; // map batch token positions to ids of the logits and embd buffers

    ggml_backend_sched_ptr sched;

    ggml_backend_t backend_cpu = nullptr;
    std::vector<ggml_backend_ptr> backends;

    ggml_context_ptr ctx_compute;

    // training
    ggml_opt_context_t opt_ctx = nullptr;

    ggml_threadpool_t threadpool       = nullptr;
    ggml_threadpool_t threadpool_batch = nullptr;

    ggml_abort_callback abort_callback      = nullptr;
    void *              abort_callback_data = nullptr;

    std::vector<std::pair<ggml_backend_t, ggml_backend_set_n_threads_t>> set_n_threads_fns;

    // buffer types used for the compute buffer of each backend
    std::vector<ggml_backend_t>             backend_ptrs;
    std::vector<ggml_backend_buffer_type_t> backend_buft;

    // memory buffers used to evaluate the model
    std::vector<uint8_t> buf_compute_meta;

    // host buffer for the model output (logits and embeddings)
    ggml_backend_buffer_ptr buf_output;

    bool has_evaluated_once = false;

    // perf
    mutable int64_t t_start_us  = 0;
    mutable int64_t t_load_us   = 0;
    mutable int64_t t_p_eval_us = 0;
    mutable int64_t t_eval_us   = 0;

    mutable int64_t t_compute_start_us = 0;
    mutable int64_t n_queued_tokens    = 0;

    mutable int32_t n_p_eval = 0; // number of tokens in eval calls for the prompt (with batch size > 1)
    mutable int32_t n_eval   = 0; // number of eval calls
};
```

</details>


```c++
// GPU backends
// add ACCEL backends (such as BLAS)
// add CPU backend
llama_context:: ggml_backend_t backend_cpu = nullptr;
--> ggml_backend_init_by_type --> ggml_backend_dev_init --> ggml_backend_cpu_device_init_backend --> ggml_backend_cpu_init --> backend_cpu;
backends.emplace_back(backend_cpu);
// create a list of the set_n_threads functions in the backends
llama_context:: std::vector<std::pair<ggml_backend_t, ggml_backend_set_n_threads_t>>
set_n_threads_fns.emplace_back(backend.get(), ggml_backend_set_n_threads_fn);
// graph outputs buffer
// resized during inference when a batch uses more outputs
--> llama_context::output_reserve;

// init the memory module
llama_memory_params params_mem = {
    /*.type_k   =*/ params.type_k,
    /*.type_v   =*/ params.type_v,
    /*.swa_full =*/ params.swa_full,
};
llama_context:: std::unique_ptr<llama_memory_i> memory;
--> llama_model::create_memory --> llama_context:: memory;

// init backends
backend_buft.clear();
backend_ptrs.clear();
//for (auto & backend : backends)
auto * buft = ggml_backend_get_default_buffer_type(backend.get());
auto backend_type = ggml_backend_dev_type(ggml_backend_get_device(backend.get()));
backend_buft.push_back(buft);
backend_ptrs.push_back(backend.get());

const size_t max_nodes = this->graph_max_nodes();
// memory buffers used to evaluate the model
llama_context:: std::vector<uint8_t> buf_compute_meta;
// buffer used to store the computation graph and the tensor meta data
--> ggml_graph_nbytes --> buf_compute_meta;
llama_context:: ggml_backend_sched_ptr sched;
--> ggml_backend_sched_new --> sched;

// reserve worst-case graph
const uint32_t n_seqs = cparams.n_seq_max;
const uint32_t n_tokens = std::min(cparams.n_ctx, cparams.n_ubatch);
// simulate full KV cache
--> llama_kv_cache_unified_state::llama_kv_cache_unified_state --> mstate;

// reserve pp graph first so that buffers are only allocated once
--> llama_context::graph_reserve --> gf;
// reserve with tg graph to get the number of splits and nodes
--> llama_context::graph_reserve
// reserve again with pp graph to avoid ggml-alloc reallocations during inference
--> llama_context::graph_reserve
```

### [`ggml_backend_cpu_init`](https://github.com/ggml-org/llama.cpp/blob/master/ggml/src/ggml-cpu/ggml-cpu.cpp#L196)

```c++
backend_cpu = ggml_backend_init_by_type(GGML_BACKEND_DEVICE_TYPE_CPU, nullptr);
----------
ggml_backend_t ggml_backend_init_by_type(enum ggml_backend_dev_type type, const char * params) {
    ggml_backend_dev_t dev = ggml_backend_dev_by_type(type);
    if (!dev) {
        return nullptr;
    }
    --> return ggml_backend_dev_init(dev, params);
}


ggml_backend_dev_t ggml_backend_dev_by_type(enum ggml_backend_dev_type type)
    for (size_t i = 0; i < ggml_backend_dev_count(); i++) {
        ggml_backend_dev_t dev = ggml_backend_dev_get(i);
        if (ggml_backend_dev_type(dev) == type) {
            return dev;
        }
    }

ggml_backend_t ggml_backend_dev_init(ggml_backend_dev_t device, const char * params)
    --> return device->iface.init_backend(device, params);

static ggml_backend_t ggml_backend_cpu_device_init_backend(ggml_backend_dev_t dev, const char * params)
    --> return ggml_backend_cpu_init();

ggml_backend_t ggml_backend_cpu_init(void) {}
```

<details>
<summary>struct ggml_backend</summary>

```c++

struct ggml_backend {
    ggml_guid_t guid;
    struct ggml_backend_i iface;
    ggml_backend_dev_t device;
    void * context;
};

typedef struct ggml_backend * ggml_backend_t;
```
    
</details>

```c++
// initialize CPU backend now to avoid slowing the first graph computation
ggml_cpu_init();

struct ggml_backend_cpu_context * ctx = new ggml_backend_cpu_context;

ctx->n_threads           = GGML_DEFAULT_N_THREADS;
ctx->threadpool          = NULL;
ctx->work_data           = NULL;
ctx->work_size           = 0;
ctx->abort_callback      = NULL;
ctx->abort_callback_data = NULL;

ggml_backend_t cpu_backend = new ggml_backend {
    /* .guid      = */ ggml_backend_cpu_guid(),
    /* .interface = */ ggml_backend_cpu_i,
    /* .device    = */ ggml_backend_reg_dev_get(ggml_backend_cpu_reg(), 0),
    /* .context   = */ ctx,
};

return cpu_backend;
```

### [`llama_context::output_reserve`](https://github.com/ggml-org/llama.cpp/blob/master/src/llama-context.cpp#L1239)

```c++
output_reserve(params.n_seq_max);
----------
int32_t llama_context::output_reserve(int32_t n_outputs) {}
```

```c++
struct llama_context
    // decode output (2-dimensional array: [n_outputs][n_vocab])
    size_t  logits_size = 0; // capacity (of floats) for logits
    float * logits      = nullptr;

// map batch token positions to ids of the logits and embd buffers
llama_context:: std::vector<int32_t> output_ids;
output_ids.resize(n_batch);

const int64_t n_outputs_max = std::max<int64_t>(n_outputs, n_seq_max());
logits_size = n_vocab*n_outputs_max;
const size_t new_size  = (logits_size + embd_size) * sizeof(float);
auto * buft = ggml_backend_cpu_buffer_type();
// host buffer for the model output (logits and embeddings)
llama_context:: ggml_backend_buffer_ptr buf_output;
--> ggml_backend_buft_alloc_buffer --> ggml_backend_cpu_buffer_type_alloc_buffer --> buf_output;

--> ggml_backend_buffer_get_base --> ggml_backend_cpu_buffer_get_base --> output_base;
logits = output_base;
// set all ids as invalid (negative)
std::fill(output_ids.begin(), output_ids.end(), -1);
this->n_outputs = 0;
this->n_outputs_max = n_outputs_max;
return n_outputs_max;
```

[`ggml_backend_cpu_buffer_type_alloc_buffer`](https://github.com/ggml-org/llama.cpp/blob/master/ggml/src/ggml-backend.cpp#L1950)

```c++
ggml_backend_buffer_ptr llama_context:: buf_output.reset(ggml_backend_buft_alloc_buffer(buft, new_size));
----------
ggml_backend_buffer_t ggml_backend_buft_alloc_buffer(ggml_backend_buffer_type_t buft, size_t size) {
    if (size == 0) {
        // return a dummy buffer for zero-sized allocations
        return ggml_backend_buffer_init(buft, {}, NULL, 0);
    }
    --> return buft->iface.alloc_buffer(buft, size);
}

static ggml_backend_buffer_t ggml_backend_cpu_buffer_type_alloc_buffer(ggml_backend_buffer_type_t buft, size_t size) {
    void * data = ggml_aligned_malloc(size);
    return ggml_backend_buffer_init(buft, ggml_backend_cpu_buffer_i, data, size);
}

```

[`ggml_backend_cpu_buffer_get_base`](https://github.com/ggml-org/llama.cpp/blob/master/ggml/src/ggml-backend.cpp#L1869)

```c++
float * output_base = (float *) ggml_backend_buffer_get_base(buf_output.get());
----------
void * ggml_backend_buffer_get_base(ggml_backend_buffer_t buffer) {
    // get_base is optional if the buffer is zero-sized
    if (buffer->size == 0) {
        return NULL;
    }
    return buffer->iface.get_base(buffer);
}

static void * ggml_backend_cpu_buffer_get_base(ggml_backend_buffer_t buffer) {
    uintptr_t data = (uintptr_t)buffer->context;

    // align the buffer
    if (data % TENSOR_ALIGNMENT != 0) {
        data = GGML_PAD(data, TENSOR_ALIGNMENT);
    }

    return (void *)data;
}
```

### [`llama_model::create_memory`](https://github.com/ggml-org/llama.cpp/blob/master/src/llama-model.cpp#L13206)


```c++
memory.reset(model.create_memory(params_mem, cparams));
----------
llama_memory_i * llama_model::create_memory(const llama_memory_params & params, llama_cparams & cparams) const {
    llama_memory_i * res;
    const auto padding = llama_kv_cache_unified::get_padding(cparams);
        --> llama_kv_cache_unified::get_padding(const llama_cparams & cparams)
        // the FA kernels require padding to avoid extra runtime boundary checks
        return cparams.flash_attn ? 256u : 32u;
    cparams.n_ctx = GGML_PAD(cparams.n_ctx, padding);
    --> res = new llama_kv_cache_unified(
            *this,
            nullptr,
            params.type_k,
            params.type_v,
            !cparams.flash_attn,
            cparams.offload_kqv,
            cparams.n_ctx,
            cparams.n_seq_max,
            padding,
            hparams.n_swa,
            hparams.swa_type);
    return res;
}

llama_kv_cache_unified::llama_kv_cache_unified(
        const llama_model &  model,
          layer_filter_cb && filter,
                ggml_type    type_k,
                ggml_type    type_v,
                     bool    v_trans,
                     bool    offload,
                 uint32_t    kv_size,
                 uint32_t    n_seq_max,
                 uint32_t    n_pad,
                 uint32_t    n_swa,
           llama_swa_type    swa_type) :
    model(model), hparams(model.hparams), v_trans(v_trans),
    n_seq_max(n_seq_max), n_pad(n_pad), n_swa(n_swa), swa_type(swa_type) {}
```

<details>
<summary>class llama_kv_cache_unified</summary>

```c++
class llama_kv_cache_unified : public llama_memory_i {
    using ubatch_heads = std::vector<uint32_t>;
private:
    const llama_model & model;
    const llama_hparams & hparams;

    struct kv_layer {
        // layer index in the model
        // note: can be different from the layer index in the KV cache
        uint32_t il;

        ggml_tensor * k;
        ggml_tensor * v;
    };

    bool v_trans = true;  // the value tensor is transposed

    // the current index from where we start searching for a free slot in the ring buffer of KV cells (see find_slot())
    // note: this is not part of the KV state and it's only used to speed-up the find_slot() method
    uint32_t head = 0;

    const uint32_t n_seq_max = 1;

    // required padding

    const llama_swa_type swa_type = LLAMA_SWA_TYPE_NONE;

    std::vector<ggml_context_ptr>        ctxs;
    std::vector<ggml_backend_buffer_ptr> bufs;

    llama_kv_cells_unified cells;

    std::vector<kv_layer> layers;

    // model layer id -> KV cache layer id
    std::unordered_map<int32_t, int32_t> map_layer_ids;
};
```

</details>


<details>
<summary>class llama_kv_cells_unified</summary>

```c++
class llama_kv_cells_unified {
private:
    bool has_shift = false;

    // set of indices of used cells (i.e. pos[i] != -1, allowed to not have any seq_id)
    std::set<uint32_t> used;

    std::vector<llama_pos> pos;

    // this array accumulates any applied shifts to the pos array since the last reset_shift() call
    // this is used to queue multiple updates to the pos array, which in the end can be applied in one go:
    //
    //   cells.pos_add(x, shift_x);
    //   cells.pos_div(y, shift_y);
    //   ...
    //
    //   if (cells.has_shift()) {
    //      for (int i = 0; i < n; ++i) {
    //          auto shift_i = cells.get_shift(i);
    //          ...
    //      }
    //      cells.reset_shift();
    //   }
    //
    std::vector<llama_pos> shift;

    using bits_t = std::bitset<LLAMA_MAX_SEQ>;

    // the bitset seq[i] tells us which sequences are currently occupying the i-th cell
    std::vector<bits_t> seq;

    // the set seq_pos[s] tells us which positions are currently present for sequence s
    // this way seq_pos[s].begin() and seq_pos[s].rbegin() give us the min/max positions currently in the cache
    std::set<llama_pos> seq_pos[LLAMA_MAX_SEQ];
};

// copy the state of cells [i, i + n) (used for save/restore the state of the cells)
llama_kv_cells_unified cp(uint32_t i, uint32_t n) const {
    llama_kv_cells_unified res;
    res.resize(n);
    for (uint32_t j = 0; j < n; ++j) {
        res.pos[j] = pos[i + j];
        res.seq[j] = seq[i + j];
    }
    return res;
}

// set the state of cells [i, i + other.pos.size()) (used for save/restore the state of the cells)
void set(uint32_t i, const llama_kv_cells_unified & other) {
    for (uint32_t j = 0; j < other.pos.size(); ++j) {
        if (pos[i + j] == -1 && other.pos[j] != -1) {
            used.insert(i + j);
        }

        if (pos[i + j] != -1 && other.pos[j] == -1) {
            used.erase(i + j);
        }

        if (pos[i + j] != -1) {
            seq_pos_rm(i + j);
        }

        pos[i + j] = other.pos[j];
        seq[i + j] = other.seq[j];

        if (pos[i + j] != -1) {
            seq_pos_add(i + j);
        }
    }
}

    // remove cell i
    void seq_pos_rm(uint32_t i) {
        for (int s = 0; s < LLAMA_MAX_SEQ; ++s) {
            if (seq[i].test(s)) {
                seq_pos[s].erase(pos[i]);
            }
        }
    }

    // add cell i
    void seq_pos_add(uint32_t i) {
        for (int s = 0; s < LLAMA_MAX_SEQ; ++s) {
            if (seq[i].test(s)) {
                seq_pos[s].insert(pos[i]);
            }
        }
    }

    // set the position of an empty cell
    // does not modify "has_shift"
    // note: call only if the cell is empty
    void pos_set(uint32_t i, llama_pos p) {
        pos[i] = p;
        used.insert(i);
    }

    // note: call only if the cell is not empty and the seq_id is not in the cell
    void seq_add(uint32_t i, llama_seq_id seq_id) {
        seq[i].set(seq_id);
        seq_pos[seq_id].insert(pos[i]);
    }

```

</details>

```c++
// create a context for each buffer type
std::map<ggml_backend_buffer_type_t, ggml_context *> ctx_map;
auto ctx_for_buft = [&](ggml_backend_buffer_type_t buft) -> ggml_context * {
    auto it = ctx_map.find(buft);
    if (it == ctx_map.end()) {
        ggml_init_params params = {
            /*.mem_size   =*/ size_t(2u*hparams.n_layer*ggml_tensor_overhead()),
            /*.mem_buffer =*/ NULL,
            /*.no_alloc   =*/ true,
        };
        ggml_context * ctx = ggml_init(params);
        ctx_map[buft] = ctx;
        ctxs.emplace_back(ctx);
        return ctx;
    }
    return it->second;
};

head = 0;
// llama_kv_cache_unified:: llama_kv_cells_unified cells;
cells.resize(kv_size);
for (uint32_t il = 0; il < hparams.n_layer; il++)
    const uint32_t n_embd_k_gqa = hparams.n_embd_k_gqa(il) + hparams.n_embd_k_s();
    const uint32_t n_embd_v_gqa = hparams.n_embd_v_gqa(il) + hparams.n_embd_v_s();
    auto * dev = model.dev_layer(il);
    buft = ggml_backend_dev_buffer_type(dev);
    dev_name = ggml_backend_dev_name(dev);
    ggml_context * ctx = ctx_for_buft(buft);
    ggml_tensor * k = ggml_new_tensor_2d(ctx, type_k, n_embd_k_gqa, kv_size);
    ggml_tensor * v = ggml_new_tensor_2d(ctx, type_v, n_embd_v_gqa, kv_size);
    ggml_format_name(k, "cache_k_l%d", il);
    ggml_format_name(v, "cache_v_l%d", il);
    // model layer id -> KV cache layer id
    // llama_kv_cache_unified:: std::unordered_map<int32_t, int32_t> map_layer_ids;
    map_layer_ids[il] = layers.size();
    // llama_kv_cache_unified:: std::vector<kv_layer> layers;
    layers.push_back({ il, k, v });

// allocate tensors and initialize the buffers to avoid NaNs in the padding
auto * buft = ctx_map[0].first;
auto * ctx  = ctx_map[0].second;
--> ggml_backend_alloc_ctx_tensors_from_buft -> buf;
ggml_backend_buffer_clear(buf, 0);
// llama_kv_cache_unified:: std::vector<ggml_backend_buffer_ptr> bufs;
bufs.emplace_back(buf);
```

#### [`ggml_backend_alloc_ctx_tensors_from_buft`](https://github.com/ggml-org/llama.cpp/blob/master/ggml/src/ggml-alloc.c#L987)


```c++
ggml_backend_buffer_t buf = ggml_backend_alloc_ctx_tensors_from_buft(ctx, buft);
----------
ggml_backend_buffer_t ggml_backend_alloc_ctx_tensors_from_buft(struct ggml_context * ctx, ggml_backend_buffer_type_t buft) {}
```


```c++
size_t alignment = ggml_backend_buft_get_alignment(buft);
size_t max_size = ggml_backend_buft_get_max_size(buft);
size_t cur_buf_size = 0;
struct ggml_tensor * first = ggml_get_first_tensor(ctx);
for (struct ggml_tensor * t = first; t != NULL; t = ggml_get_next_tensor(ctx, t))
    size_t this_size = GGML_PAD(ggml_backend_buft_get_alloc_size(buft, t), alignment); --> ggml_nbytes(tensor)
    cur_buf_size += this_size;
// allocate remaining tensors
--> alloc_tensor_range

ggml_backend_buffer_t buffer = buffers[0];
free(buffers);
return buffer;
```

[`alloc_tensor_range`](https://github.com/ggml-org/llama.cpp/blob/master/ggml/src/ggml-alloc.c#L946)

```c++
alloc_tensor_range(ctx, first, NULL, buft, cur_buf_size, &buffers, &n_buffers);
----------
static bool alloc_tensor_range(struct ggml_context * ctx,
        struct ggml_tensor * first, struct ggml_tensor * last,
        ggml_backend_buffer_type_t buft, size_t size,
        ggml_backend_buffer_t ** buffers, size_t * n_buffers) {
    // ggml_backend_buffer_t buffer = ggml_backend_buft_alloc_buffer(buft, size);
    --> ggml_backend_buft_alloc_buffer --> ggml_backend_buffer_t buffer;
    *buffers = realloc(*buffers, sizeof(ggml_backend_buffer_t) * (*n_buffers + 1));
    (*buffers)[(*n_buffers)++] = buffer;
    --> ggml_tallocr_new --> struct ggml_tallocr tallocr;
    for (struct ggml_tensor * t = first; t != last; t = ggml_get_next_tensor(ctx, t))
        --> ggml_tallocr_alloc;
}
```

[`ggml_tallocr_new`](https://github.com/ggml-org/llama.cpp/blob/master/ggml/src/ggml-alloc.c#L77)

<details>
<summary>struct ggml_tallocr</summary>

```c++
// Tensor allocator
struct ggml_tallocr {
    ggml_backend_buffer_t buffer;
    void * base;
    size_t alignment;
    size_t offset;
};
```

</details>

```c++
struct ggml_tallocr tallocr = ggml_tallocr_new(buffer);
----------
struct ggml_tallocr ggml_tallocr_new(ggml_backend_buffer_t buffer) {
    void * base = ggml_backend_buffer_get_base(buffer);
    size_t align = ggml_backend_buffer_get_alignment(buffer);

    assert(align && !(align & (align - 1))); // power of 2

    struct ggml_tallocr talloc = (struct ggml_tallocr) {
        /*.buffer    = */ buffer,
        /*.base      = */ base,
        /*.alignment = */ align,
        /*.offset    = */ aligned_offset(base, 0, align),
    };
    return talloc;
}
```

[`ggml_tallocr_alloc`](https://github.com/ggml-org/llama.cpp/blob/master/ggml/src/ggml-alloc.c#L92)

```c++
status = ggml_tallocr_alloc(&tallocr, t);
----------
enum ggml_status ggml_tallocr_alloc(struct ggml_tallocr * talloc, struct ggml_tensor * tensor) {
    size_t size = ggml_backend_buffer_get_alloc_size(talloc->buffer, tensor);
    size = GGML_PAD(size, talloc->alignment);

    void * addr = (char *)ggml_backend_buffer_get_base(talloc->buffer) + talloc->offset;
    talloc->offset += size;

    assert(((uintptr_t)addr % talloc->alignment) == 0);

    return ggml_backend_tensor_alloc(talloc->buffer, tensor, addr);
}
```

### [`ggml_graph_nbytes`](https://github.com/ggml-org/llama.cpp/blob/master/ggml/src/ggml.c#L5961)

```c++
buf_compute_meta.resize(ggml_tensor_overhead()*max_nodes + ggml_graph_overhead_custom(max_nodes, false));
----------
size_t ggml_graph_overhead_custom(size_t size, bool grads) {
    return GGML_OBJECT_SIZE + GGML_PAD(ggml_graph_nbytes(size, grads), GGML_MEM_ALIGN);
}

static size_t ggml_graph_nbytes(size_t size, bool grads) {}
```

```c++
size_t hash_size = ggml_hash_size(size * 2);
void * p = 0;
incr_ptr_aligned(&p, sizeof(struct ggml_cgraph), 1);
incr_ptr_aligned(&p, size * sizeof(struct ggml_tensor *), sizeof(struct ggml_tensor *)); // nodes
incr_ptr_aligned(&p, size * sizeof(struct ggml_tensor *), sizeof(struct ggml_tensor *)); // leafs
incr_ptr_aligned(&p, hash_size * sizeof(struct ggml_tensor *), sizeof(struct ggml_tensor *)); // hash keys
incr_ptr_aligned(&p, ggml_bitset_size(hash_size) * sizeof(ggml_bitset_t), sizeof(ggml_bitset_t));

size_t nbytes = (size_t) p;
return nbytes;
```

### [`ggml_backend_sched_new`](https://github.com/ggml-org/llama.cpp/blob/master/ggml/src/ggml-backend.cpp#L1455)

```c++
sched.reset(ggml_backend_sched_new(backend_ptrs.data(), backend_buft.data(), backend_ptrs.size(), max_nodes, pipeline_parallel, cparams.op_offload));
----------
ggml_backend_sched_t ggml_backend_sched_new(
        ggml_backend_t * backends,
        ggml_backend_buffer_type_t * bufts,
        int n_backends,
        size_t graph_size,
        bool parallel,
        bool op_offload) {}
```

<details>
<summary>struct ggml_backend_sched</summary>

```c++
struct ggml_backend_sched {
    bool is_reset; // true if the scheduler has been reset since the last graph split
    bool is_alloc;

    int n_backends;

    ggml_backend_t backends[GGML_SCHED_MAX_BACKENDS];
    ggml_backend_buffer_type_t bufts[GGML_SCHED_MAX_BACKENDS];
    ggml_gallocr_t galloc;

    // hash map of the nodes in the graph
    struct ggml_hash_set  hash_set;
    int                 * hv_tensor_backend_ids; // [hash_set.size]
    struct ggml_tensor ** hv_tensor_copies;      // [hash_set.size][n_backends][n_copies]

    int * node_backend_ids; // [graph_size]
    int * leaf_backend_ids; // [graph_size]

    int * prev_node_backend_ids; // [graph_size]
    int * prev_leaf_backend_ids; // [graph_size]

    // copy of the graph with modified inputs
    struct ggml_cgraph graph;

    // graph splits
    struct ggml_backend_sched_split * splits;
    int n_splits;
    int splits_capacity;

    // pipeline parallelism support
    int n_copies;
    int cur_copy;
    ggml_backend_event_t events[GGML_SCHED_MAX_BACKENDS][GGML_SCHED_MAX_COPIES];
    struct ggml_tensor * graph_inputs[GGML_SCHED_MAX_SPLIT_INPUTS];
    int n_graph_inputs;

    struct ggml_context * ctx;

    ggml_backend_sched_eval_callback callback_eval;
    void * callback_eval_user_data;

    char * context_buffer;
    size_t context_buffer_size;

    bool op_offload;

    int debug;
};

typedef struct ggml_backend_sched * ggml_backend_sched_t;
```

</details>

```c++
struct ggml_backend_sched * sched = (ggml_backend_sched *) calloc(1, sizeof(struct ggml_backend_sched));
sched->n_backends = n_backends;
sched->n_copies = parallel ? GGML_SCHED_MAX_COPIES : 1;
// initialize hash table
// FIXME: needs to be size*2 to account for leafs (do it in graph_split instead)
sched->hash_set    = ggml_hash_set_new(graph_size);
sched->hv_tensor_backend_ids = (int *) malloc(sched->hash_set.size * sizeof(sched->hv_tensor_backend_ids[0]));
sched->hv_tensor_copies      = (ggml_tensor **) malloc(sched->hash_set.size * sched->n_backends * sched->n_copies * sizeof(struct ggml_tensor *));
const size_t ggml_sched_max_splits = graph_size; // at most there is one split for each node in the graph
const size_t nodes_size = graph_size + ggml_sched_max_splits*GGML_SCHED_MAX_SPLIT_INPUTS*2;
sched->node_backend_ids = (int *) calloc(nodes_size, sizeof(sched->node_backend_ids[0]));
sched->leaf_backend_ids = (int *) calloc(nodes_size, sizeof(sched->leaf_backend_ids[0]));
sched->prev_node_backend_ids = (int *) calloc(nodes_size, sizeof(sched->prev_node_backend_ids[0]));
sched->prev_leaf_backend_ids = (int *) calloc(nodes_size, sizeof(sched->prev_leaf_backend_ids[0]));

sched->context_buffer_size = ggml_sched_max_splits*GGML_SCHED_MAX_SPLIT_INPUTS*2*sizeof(struct ggml_tensor) + ggml_graph_overhead_custom(graph_size, false);
sched->context_buffer = (char *) malloc(sched->context_buffer_size);

const int initial_splits_capacity = 16;
sched->splits = (ggml_backend_sched_split *) calloc(initial_splits_capacity, sizeof(sched->splits[0]));
sched->splits_capacity = initial_splits_capacity;

sched->backends[0] = backends[0];
sched->bufts[0] = bufts[0];

--> ggml_gallocr_new_n --> sched->galloc;
sched->op_offload = op_offload;
ggml_backend_sched_reset(sched);
    // reset state for the next run
    if (!sched->is_reset) {
        ggml_hash_set_reset(&sched->hash_set);
        memset(sched->hv_tensor_backend_ids, -1, sched->hash_set.size * sizeof(sched->hv_tensor_backend_ids[0]));
        memset(sched->hv_tensor_copies,       0, sched->hash_set.size * sched->n_backends * sched->n_copies * sizeof(struct ggml_tensor *));
        sched->is_reset = true;
    }
    sched->is_alloc = false;
return sched;
```


#### [`ggml_gallocr_new_n`](https://github.com/ggml-org/llama.cpp/blob/master/ggml/src/ggml-alloc.c#L380)

<details>
<summary>struct ggml_gallocr</summary>

```c++
struct ggml_gallocr {
    ggml_backend_buffer_type_t * bufts; // [n_buffers]
    ggml_backend_buffer_t * buffers; // [n_buffers]
    struct ggml_dyn_tallocr ** buf_tallocs; // [n_buffers]
    int n_buffers;

    struct ggml_hash_set hash_set;
    struct hash_node * hash_values; // [hash_set.size]

    struct node_alloc * node_allocs; // [n_nodes]
    int n_nodes;

    struct leaf_alloc * leaf_allocs; // [n_leafs]
    int n_leafs;
};


typedef struct ggml_gallocr * ggml_gallocr_t;
```

</details>


```c++
sched->galloc = ggml_gallocr_new_n(sched->bufts, n_backends);
----------
ggml_gallocr_t ggml_gallocr_new_n(ggml_backend_buffer_type_t * bufts, int n_bufs) {
    galloc->bufts = calloc(n_bufs, sizeof(ggml_backend_buffer_type_t));
    galloc->buffers = calloc(n_bufs, sizeof(ggml_backend_buffer_t));
    galloc->buf_tallocs = calloc(n_bufs, sizeof(struct ggml_dyn_tallocr *));
    // for (int i = 0; i < n_bufs; i++)
    galloc->bufts[0] = bufts[0];
    galloc->buffers[0] = NULL;
    size_t alignment = ggml_backend_buft_get_alignment(bufts[0]);
    --> ggml_dyn_tallocr_new --> galloc->buf_tallocs[0];

    galloc->n_buffers = n_bufs;
    return galloc;
}
```

[`ggml_dyn_tallocr_new`](https://github.com/ggml-org/llama.cpp/blob/master/ggml/src/ggml-alloc.c#L310)

<details>
<summary>struct ggml_dyn_tallocr</summary>

```c++
struct ggml_dyn_tallocr {
    size_t alignment;
    int n_free_blocks;
    struct free_block free_blocks[MAX_FREE_BLOCKS];
    size_t max_size;
};
```

</details>

```c++
galloc->buf_tallocs[i] = ggml_dyn_tallocr_new(alignment);
----------
static struct ggml_dyn_tallocr * ggml_dyn_tallocr_new(size_t alignment) {
    struct ggml_dyn_tallocr * alloc = (struct ggml_dyn_tallocr *)malloc(sizeof(struct ggml_dyn_tallocr));

    *alloc = (struct ggml_dyn_tallocr) {
        /*.alignment     = */ alignment,
        /*.n_free_blocks = */ 0,
        /*.free_blocks   = */ {{0}},
        /*.max_size      = */ 0,
    };

    ggml_dyn_tallocr_reset(alloc);
        alloc->n_free_blocks = 1;
        alloc->free_blocks[0].offset = 0;
        alloc->free_blocks[0].size = SIZE_MAX/2; // restrict maximum size of a measure allocator to half size_t max to avoid overflows
        alloc->max_size = 0;
    return alloc;
}
```

### [`llama_kv_cache_unified_state::llama_kv_cache_unified_state`](https://github.com/ggml-org/llama.cpp/blob/master/src/llama-kv-cache-unified.cpp#L1668)

<details>
<summary>class llama_kv_cache_unified_state</summary>

```c++
class llama_kv_cache_unified_state : public llama_memory_state_i {
    using ubatch_heads = llama_kv_cache_unified::ubatch_heads;
private:
    llama_memory_status status;

    llama_kv_cache_unified * kv;
    llama_context * lctx;

    //
    // update state
    //

    bool do_shift = false;

    defrag_info dinfo;

    //
    // batch processing state
    //

    llama_sbatch sbatch;

    // the index of the next ubatch to process
    size_t i_next = 0;

    ubatch_heads heads;

    std::vector<llama_ubatch> ubatches;

    //
    // data needed for building the compute graph for the current ubatch:
    //

    // a heuristic, to avoid attending the full cache if it is not yet utilized
    // as the cache gets filled, the benefit from this heuristic disappears
    int32_t n_kv;

    // the beginning of the current slot in which the ubatch will be inserted
    int32_t head;
};
```

</details>

```c++
const auto mstate = memory->init_full();
----------
llama_memory_state_ptr llama_kv_cache_unified::init_full() {
    --> return std::make_unique<llama_kv_cache_unified_state>(this);
}

class llama_kv_cache_unified_state : public llama_memory_state_i {
public:
    // used to create a full-cache state
    --> llama_kv_cache_unified_state(llama_kv_cache_unified * kv);
}

llama_kv_cache_unified_state::llama_kv_cache_unified_state(llama_kv_cache_unified * kv) : status(LLAMA_MEMORY_STATUS_SUCCESS), kv(kv) {
    n_kv = kv->get_size();
    head = 0;
}           
```

### [`llama_context::graph_reserve`](https://github.com/ggml-org/llama.cpp/blob/master/src/llama-context.cpp#L1305)

```c++
auto * gf = graph_reserve(n_tokens, n_seqs, n_tokens, mstate.get());
auto * gf = graph_reserve(1, 1, 1, mstate.get());

ggml_cgraph * llama_context::graph_reserve(uint32_t n_tokens, uint32_t n_seqs, uint32_t n_outputs, const llama_memory_state_i * mstate) {}
```

```c++
this->n_outputs = n_outputs;
llama_token token = model.vocab.token_bos(); // not actually used by llama_build_graph, but required to choose between token and embedding inputs graph
llama_ubatch ubatch = { true, n_tokens, n_tokens / n_seqs, n_seqs, &token, nullptr, nullptr, nullptr, nullptr, nullptr};

--> llama_context::graph_init --> ggml_cgraph * gf;
--> llama_context::graph_build --> llama_model::build_graph --> llm_build_llama::llm_build_llama --> std::unique_ptr<llm_graph_result> res;

ggml_backend_sched_reset(sched.get());

// initialize scheduler with the specified graph
--> ggml_backend_sched_reserve;
return gf;
```

#### [`llama_context::graph_init`](https://github.com/ggml-org/llama.cpp/blob/master/src/llama-context.cpp#L1319)

```c++
auto * gf = graph_init();
----------
ggml_cgraph * llama_context::graph_init() {}
```

<details>
<summary>struct ggml_cgraph</summary>

```c++
struct ggml_cgraph {
    int size;    // maximum number of nodes/leafs/grads/grad_accs
    int n_nodes; // number of nodes currently in use
    int n_leafs; // number of leafs currently in use

    struct ggml_tensor ** nodes;     // tensors with data that can change if the graph is evaluated
    struct ggml_tensor ** grads;     // the outputs of these tensors are the gradients of the nodes
    struct ggml_tensor ** grad_accs; // accumulators for node gradients
    struct ggml_tensor ** leafs;     // tensors with constant data

    struct ggml_hash_set visited_hash_set;

    enum ggml_cgraph_eval_order order;
};
```

</details>

```c++
ggml_init_params params = {
    /*.mem_size   =*/ buf_compute_meta.size(),
    /*.mem_buffer =*/ buf_compute_meta.data(),
    /*.no_alloc   =*/ true,
};
// llama_context:: ggml_context_ptr ctx_compute;
ctx_compute.reset(ggml_init(params));
--> return ggml_new_graph_custom;
```

##### [`ggml_new_graph_custom`](https://github.com/ggml-org/llama.cpp/blob/master/ggml/src/ggml.c#L5986)


```c++
return ggml_new_graph_custom(ctx_compute.get(), graph_max_nodes(), false);
----------
struct ggml_cgraph * ggml_new_graph_custom(struct ggml_context * ctx, size_t size, bool grads) {
    const size_t obj_size = ggml_graph_nbytes(size, grads);
    struct ggml_object * obj = ggml_new_object(ctx, GGML_OBJECT_TYPE_GRAPH, obj_size);
    struct ggml_cgraph * cgraph = (struct ggml_cgraph *) ((char *) ctx->mem_buffer + obj->offs);

    // the size of the hash table is doubled since it needs to hold both nodes and leafs
    size_t hash_size = ggml_hash_size(size * 2);

    void * p = cgraph + 1;

    struct ggml_tensor ** nodes_ptr     =         incr_ptr_aligned(&p, size      * sizeof(struct ggml_tensor *), sizeof(struct ggml_tensor *));
    struct ggml_tensor ** leafs_ptr     =         incr_ptr_aligned(&p, size      * sizeof(struct ggml_tensor *), sizeof(struct ggml_tensor *));
    struct ggml_tensor ** hash_keys_ptr =         incr_ptr_aligned(&p, hash_size * sizeof(struct ggml_tensor *), sizeof(struct ggml_tensor *));
    struct ggml_tensor ** grads_ptr     = grads ? incr_ptr_aligned(&p, hash_size * sizeof(struct ggml_tensor *), sizeof(struct ggml_tensor *)) : NULL;
    struct ggml_tensor ** grad_accs_ptr = grads ? incr_ptr_aligned(&p, hash_size * sizeof(struct ggml_tensor *), sizeof(struct ggml_tensor *)) : NULL;

    ggml_bitset_t * hash_used = incr_ptr_aligned(&p, ggml_bitset_size(hash_size) * sizeof(ggml_bitset_t), sizeof(ggml_bitset_t));

    *cgraph = (struct ggml_cgraph) {
        /*.size         =*/ size,
        /*.n_nodes      =*/ 0,
        /*.n_leafs      =*/ 0,
        /*.nodes        =*/ nodes_ptr,
        /*.grads        =*/ grads_ptr,
        /*.grad_accs    =*/ grad_accs_ptr,
        /*.leafs        =*/ leafs_ptr,
        /*.hash_table   =*/ { hash_size, hash_used, hash_keys_ptr },
        /*.order        =*/ GGML_CGRAPH_EVAL_ORDER_LEFT_TO_RIGHT,
    };

    ggml_hash_set_reset(&cgraph->visited_hash_set);
    return cgraph;
}

```

#### [`llm_build_llama::llm_build_llama`](https://github.com/ggml-org/llama.cpp/blob/master/src/llama-model.cpp#L4686)


<details>
<summary>class llm_graph_result</summary>

```c++
//
// llm_graph_result
//

// these objects deliver the result from the graph build process back to the llama_context
// note that the input tensors created for the graph are referenced here - the goal is to be able to populate their
//   specific data, by calling the set_inputs() method
// along with the input tensors, the object also provides commonly used outputs tensors, such as logits, embeddings, etc.
//   these are used by the llama_context to extact the relevant data, based on the compute parameters

class llm_graph_result_i {
public:
    virtual ~llm_graph_result_i() = default;

    virtual ggml_tensor * get_tokens()      = 0;
    virtual ggml_tensor * get_logits()      = 0;
    virtual ggml_tensor * get_embd()        = 0;
    virtual ggml_tensor * get_embd_pooled() = 0;

    virtual void set_inputs(const llama_ubatch * ubatch) = 0;
};

using llm_graph_result_ptr = std::unique_ptr<llm_graph_result_i>;

class llm_graph_result : public llm_graph_result_i {
public:
    // important graph nodes
    ggml_tensor * t_tokens      = nullptr;
    ggml_tensor * t_logits      = nullptr;
    ggml_tensor * t_embd        = nullptr;
    ggml_tensor * t_embd_pooled = nullptr;

    std::vector<llm_graph_input_ptr> inputs;
}

    void set_inputs(const llama_ubatch * ubatch) override {
        for (auto & input : inputs) {
            input->set_input(ubatch);
        }
    }

    llm_graph_input_i * add_input(llm_graph_input_ptr input) {
        inputs.emplace_back(std::move(input));
        return inputs.back().get();
    }
```

</details>

<details>
<summary>struct llm_graph_context</summary>

```c++
struct llm_graph_context {
    const llm_arch arch;

    const llama_hparams & hparams;
    const llama_cparams & cparams;
    const llama_ubatch  & ubatch;

    const int64_t n_embd;
    const int64_t n_layer;
    const int64_t n_rot;
    const int64_t n_ctx;       // user-specified context size (can be different from n_ctx_train)
    const int64_t n_head;
    const int64_t n_head_kv;
    const int64_t n_embd_head_k;
    const int64_t n_embd_k_gqa;
    const int64_t n_embd_head_v;
    const int64_t n_embd_v_gqa;
    const int64_t n_expert;
    const int64_t n_expert_used;

    const float freq_base;
    const float freq_scale;
    const float ext_factor;
    const float attn_factor;
    const float beta_fast;
    const float beta_slow;
    const float norm_eps;
    const float norm_rms_eps;

    const int32_t n_tokens;
    const int32_t n_outputs;
    const int32_t n_ctx_orig; // yarn

    const enum llama_pooling_type pooling_type;
    const enum llama_rope_type    rope_type;

    ggml_context * ctx0 = nullptr;

    ggml_backend_sched_t sched;

    ggml_backend_t backend_cpu; // TODO: needed by build_attn_mha, figure out a way to remove?

    const llama_adapter_cvec   * cvec;
    const llama_adapter_loras  * loras;
    const llama_memory_state_i * mstate;
    const llama_cross          * cross;

    const llm_graph_cb & cb_func;

    std::unique_ptr<llm_graph_result> res;
};
```

</details>


```c++
auto res = graph_build(ctx_compute.get(), gf, ubatch, LLM_GRAPH_TYPE_DEFAULT, mstate);
----------
llm_graph_result_ptr llama_context::graph_build(
                    ggml_context * ctx,
                     ggml_cgraph * gf,
              const llama_ubatch & ubatch,
                  llm_graph_type   gtype,
      const llama_memory_state_i * mstate) {
      --> return model.build_graph(
            {
                /*.ctx         =*/ ctx,
                /*.arch        =*/ model.arch,
                /*.hparams     =*/ model.hparams,
                /*.cparams     =*/ cparams,
                /*.ubatch      =*/ ubatch,
                /*.sched       =*/ sched.get(),
                /*.backend_cpu =*/ backend_cpu,
                /*.cvec        =*/ &cvec,
                /*.loras       =*/ &loras,
                /*.mstate      =*/ mstate,
                /*.cross       =*/ &cross,
                /*.n_outputs   =*/ n_outputs,
                /*.cb          =*/ graph_get_cb(),
            }, gf, gtype);
}

llm_graph_result_ptr llama_model::build_graph(
        const llm_graph_params & params,
                   ggml_cgraph * gf,
                llm_graph_type   type) const {
    --> std::unique_ptr<llm_graph_context> llm = std::make_unique<llm_build_llama>(*this, params, gf);
    return std::move(llm->res);
}

struct llm_build_llama : public llm_graph_context {
    llm_build_llama(const llama_model & model, const llm_graph_params & params, ggml_cgraph * gf) : llm_graph_context(params) {}
};
```

<details>
<summary>class llm_graph_input_i</summary>

```c++
class llm_graph_input_i {
public:
    virtual ~llm_graph_input_i() = default;

    virtual void set_input(const llama_ubatch * ubatch) = 0;
};

using llm_graph_input_ptr = std::unique_ptr<llm_graph_input_i>;
```

</details>

```c++
const int64_t n_embd_head = hparams.n_embd_head_v;

ggml_tensor * cur;
ggml_tensor * inpL;

--> build_inp_embd --> inpL;
// inp_pos - contains the positions
--> build_inp_pos --> ggml_tensor * inp_pos;
--> build_attn_inp_kv_unified --> auto * inp_attn;

const float kq_scale = hparams.f_attention_scale == 0.0f ? 1.0f/sqrtf(float(n_embd_head)) : hparams.f_attention_scale;

for (int il = 0; il < n_layer; ++il) {
    ggml_tensor * inpSA = inpL;

    // norm
    cur = build_norm(inpL,
            model.layers[il].attn_norm, NULL,
            LLM_NORM_RMS, il);
    cb(cur, "attn_norm", il);

    // self-attention
    {
        // rope freq factors for llama3; may return nullptr for llama2 and other models
        ggml_tensor * rope_factors = model.get_rope_factors(cparams, il);

        // compute Q and K and RoPE them
        ggml_tensor * Qcur = build_lora_mm(model.layers[il].wq, cur);
        cb(Qcur, "Qcur", il);
        ggml_tensor * Kcur = build_lora_mm(model.layers[il].wk, cur);
        cb(Kcur, "Kcur", il);
        ggml_tensor * Vcur = build_lora_mm(model.layers[il].wv, cur);
        cb(Vcur, "Vcur", il);
        Qcur = ggml_reshape_3d(ctx0, Qcur, n_embd_head, n_head,    n_tokens);
        Kcur = ggml_reshape_3d(ctx0, Kcur, n_embd_head, n_head_kv, n_tokens);
        Vcur = ggml_reshape_3d(ctx0, Vcur, n_embd_head, n_head_kv, n_tokens);
        Qcur = ggml_rope_ext(
                ctx0, Qcur, inp_pos, rope_factors,
                n_rot, rope_type, n_ctx_orig, freq_base, freq_scale,
                ext_factor, attn_factor, beta_fast, beta_slow
                );
        Kcur = ggml_rope_ext(
                ctx0, Kcur, inp_pos, rope_factors,
                n_rot, rope_type, n_ctx_orig, freq_base, freq_scale,
                ext_factor, attn_factor, beta_fast, beta_slow
                );
        cb(Qcur, "Qcur", il);
        cb(Kcur, "Kcur", il);
        cb(Vcur, "Vcur", il);
        --> cur = build_attn(inp_attn, gf,
                model.layers[il].wo, model.layers[il].bo,
                Qcur, Kcur, Vcur, nullptr, nullptr, kq_scale, il);
        cb(cur, "attn_out", il);
    }

    if (il == n_layer - 1) {
        // skip computing output for unused tokens
        ggml_tensor * inp_out_ids = build_inp_out_ids();
        cur   = ggml_get_rows(ctx0,   cur, inp_out_ids);
        inpSA = ggml_get_rows(ctx0, inpSA, inp_out_ids);
    }

    ggml_tensor * ffn_inp = ggml_add(ctx0, cur, inpSA);
    cb(ffn_inp, "ffn_inp", il);
    // feed-forward network (non-MoE)
    cur = build_norm(ffn_inp,
            model.layers[il].ffn_norm, NULL,
            LLM_NORM_RMS, il);
    cb(cur, "ffn_norm", il);

    cur = build_ffn(cur,
            model.layers[il].ffn_up,   model.layers[il].ffn_up_b,   NULL,
            model.layers[il].ffn_gate, model.layers[il].ffn_gate_b, NULL,
            model.layers[il].ffn_down, model.layers[il].ffn_down_b, NULL,
            NULL,
            LLM_FFN_SILU, LLM_FFN_PAR, il);
    cb(cur, "ffn_out", il);
    cur = ggml_add(ctx0, cur, ffn_inp);
    cb(cur, "ffn_out", il);
    cur = build_cvec(cur, il);
    cb(cur, "l_out", il);
    // input for next layer
    inpL = cur;
}

cur = inpL;

cur = build_norm(cur,
        model.output_norm, NULL,
        LLM_NORM_RMS, -1);

cb(cur, "result_norm", -1);
res->t_embd = cur;

// lm_head
cur = build_lora_mm(model.output, cur);

cb(cur, "result_output", -1);
res->t_logits = cur;

--> ggml_build_forward_expand --> ggml_build_forward_impl --> ggml_visit_parents;
```

##### [`llm_graph_context::build_inp_embd`](https://github.com/ggml-org/llama.cpp/blob/master/src/llama-graph.cpp#L847)

<details>
<summary>class llm_graph_input_embd</summary>

```c++
class llm_graph_input_embd : public llm_graph_input_i {
public:
    ggml_tensor * tokens = nullptr; // I32 [n_batch]
    ggml_tensor * embd   = nullptr; // F32 [n_embd, n_batch]
};


void llm_graph_input_embd::set_input(const llama_ubatch * ubatch) {
    const int64_t n_tokens = ubatch->n_tokens;
    ggml_backend_tensor_set(tokens, ubatch->token, 0, n_tokens*ggml_element_size(tokens));
}
```

</details>

```c++
inpL = build_inp_embd(model.tok_embd);
----------
// input embeddings with optional lora
ggml_tensor * llm_graph_context::build_inp_embd(ggml_tensor * tok_embd) const {}
```

```c++
const int64_t n_embd = hparams.n_embd;
auto inp = std::make_unique<llm_graph_input_embd>();
inp->tokens = ggml_new_tensor_1d(ctx0, GGML_TYPE_I32, ubatch.n_tokens);
ggml_set_input(inp->tokens);
    tensor->flags |= GGML_TENSOR_FLAG_INPUT;
res->t_tokens = inp->tokens;
ggml_tensor * cur = ggml_get_rows(ctx0, tok_embd, inp->tokens);
cb(cur, "inp_embd", -1);
res->add_input(std::move(inp));
return cur;
```

##### [`llm_graph_context::build_inp_pos`](https://github.com/ggml-org/llama.cpp/blob/master/src/llama-graph.cpp#L898)

<details>
<summary>class llm_graph_input_pos</summary>

```c++
class llm_graph_input_pos : public llm_graph_input_i {
public:
    ggml_tensor * pos = nullptr; // I32 [n_batch]
    const int64_t n_pos_per_embd = 1;
};

void llm_graph_input_pos::set_input(const llama_ubatch * ubatch) {
    // if (ubatch->pos && pos)
    const int64_t n_tokens = ubatch->n_tokens;
    ggml_backend_tensor_set(pos, ubatch->pos, 0, n_tokens*n_pos_per_embd*ggml_element_size(pos));
}
```

</details>

```c++
// inp_pos - contains the positions
ggml_tensor * inp_pos = build_inp_pos();
----------
ggml_tensor * llm_graph_context::build_inp_pos() const {}
```

```c++
auto inp = std::make_unique<llm_graph_input_pos>(n_pos_per_embd());
auto & cur = inp->pos;
cur = ggml_new_tensor_1d(ctx0, GGML_TYPE_I32, n_tokens*n_pos_per_embd());
ggml_set_input(cur);
res->add_input(std::move(inp));
return cur;
```

##### [`llm_graph_context::build_attn_inp_kv_unified`](https://github.com/ggml-org/llama.cpp/blob/master/src/llama-graph.cpp#L1224)

<details>
<summary>llm_graph_input_attn_kv_unified</summary>

```c++
class llm_graph_input_attn_kv_unified : public llm_graph_input_i {
public:
    ggml_tensor * self_kq_mask     = nullptr; // F32 [n_kv, n_batch]
    ggml_tensor * self_kq_mask_cnv = nullptr; //     [n_kv, n_batch]

    const llama_hparams & hparams;
    const llama_cparams & cparams;

    const llama_kv_cache_unified_state * kv_state;
};
```

</details>


```c++
auto * inp_attn = build_attn_inp_kv_unified();
----------
llm_graph_input_attn_kv_unified * llm_graph_context::build_attn_inp_kv_unified() const {}
```

```c++
const auto * kv_state = static_cast<const llama_kv_cache_unified_state *>(mstate);
auto inp = std::make_unique<llm_graph_input_attn_kv_unified>(hparams, cparams, kv_state);
const auto n_kv = kv_state->get_n_kv();
inp->self_kq_mask = ggml_new_tensor_2d(ctx0, GGML_TYPE_F32, n_kv, GGML_PAD(n_tokens, GGML_KQ_MASK_PAD));
ggml_set_input(inp->self_kq_mask);
inp->self_kq_mask_cnv = cparams.flash_attn ? ggml_cast(ctx0, inp->self_kq_mask, GGML_TYPE_F16) : inp->self_kq_mask;
return (llm_graph_input_attn_kv_unified *) res->add_input(std::move(inp));
```

##### [`llm_graph_context::build_inp_out_ids`](src/llama-graph.cpp#L925)

```c++
ggml_tensor * inp_out_ids = build_inp_out_ids();
----------
ggml_tensor * llm_graph_context::build_inp_out_ids() const {}
```

<details>
<summary>class llm_graph_input_out_ids</summary>

```c++
class llm_graph_input_out_ids : public llm_graph_input_i {
public:
    ggml_tensor * out_ids; // I32 [n_outputs]

    const llama_hparams & hparams;
    const llama_cparams & cparams;

    const int32_t n_outputs;
};


void llm_graph_input_out_ids::set_input(const llama_ubatch * ubatch) {
    const int64_t n_tokens = ubatch->n_tokens;

    int32_t * data = (int32_t *) out_ids->data;

    if (n_outputs == n_tokens) {
        for (int i = 0; i < n_tokens; ++i) {
            data[i] = i;
        }
    } else if (ubatch->output) {
        int32_t n_outputs = 0;
        for (int i = 0; i < n_tokens; ++i) {
            if (ubatch->output[i]) {
                data[n_outputs++] = i;
            }
        }
    } else if (n_outputs == 1) {
        // only keep last output
        data[0] = n_tokens - 1;
    }
}
```

</details>

```c++
auto inp = std::make_unique<llm_graph_input_out_ids>(hparams, cparams, n_outputs);
auto & cur = inp->out_ids;
cur = ggml_new_tensor_1d(ctx0, GGML_TYPE_I32, n_outputs);
ggml_set_input(cur);
res->add_input(std::move(inp));
return cur;
```

##### [`llm_graph_context::build_attn`](https://github.com/ggml-org/llama.cpp/blob/master/src/llama-graph.cpp#L1255)


```c++
cur = build_attn(inp_attn, gf,
                model.layers[il].wo, model.layers[il].bo,
                Qcur, Kcur, Vcur, nullptr, nullptr, kq_scale, il);
----------
ggml_tensor * llm_graph_context::build_attn(
        llm_graph_input_attn_kv_unified * inp,
        ggml_cgraph * gf,
        ggml_tensor * wo,
        ggml_tensor * wo_b,
        ggml_tensor * q_cur,
        ggml_tensor * k_cur,
        ggml_tensor * v_cur,
        ggml_tensor * kq_b,
        ggml_tensor * v_mla,
            float     kq_scale,
            int       il) const {}
```

```c++
// these nodes are added to the graph together so that they are not reordered
// by doing so, the number of splits in the graph is reduced
ggml_build_forward_expand(gf, q_cur);
ggml_build_forward_expand(gf, k_cur);
ggml_build_forward_expand(gf, v_cur);

const auto * kv_state = static_cast<const llama_kv_cache_unified_state *>(mstate);

// store to KV cache
ggml_build_forward_expand(gf, kv_state->cpy_k(ctx0, k_cur, il));
ggml_build_forward_expand(gf, kv_state->cpy_v(ctx0, v_cur, il));

const auto & kq_mask = inp->get_kq_mask();

ggml_tensor * q = q_cur;
ggml_tensor * k = kv_state->get_k(ctx0, il);
ggml_tensor * v = kv_state->get_v(ctx0, il);

--> lm_graph_context::build_attn_mha --> ggml_tensor * cur;
cb(cur, "kqv_out", il);
cur = build_lora_mm(wo, cur);
return cur;
```

- [`llama_kv_cache_unified_state::cpy_k`]()
- [`llama_kv_cache_unified_state::cpy_v`]()
- [`ggml_cpy_impl`]()

```c++
kv_state->cpy_k(ctx0, k_cur, il);
kv_state->cpy_v(ctx0, v_cur, il);
----------
ggml_tensor * llama_kv_cache_unified_state::cpy_k(ggml_context * ctx, ggml_tensor * k_cur, int32_t il) const
    return kv->cpy_k(ctx, k_cur, il, head);

ggml_tensor * llama_kv_cache_unified_state::cpy_v(ggml_context * ctx, ggml_tensor * v_cur, int32_t il) const
    return kv->cpy_v(ctx, v_cur, il, head);

ggml_tensor * llama_kv_cache_unified::cpy_k(ggml_context * ctx, ggml_tensor * k_cur, int32_t il, uint32_t head_cur) const {
    const int32_t ikv = map_layer_ids.at(il);

    auto * k = layers[ikv].k;

    const int64_t n_tokens = k_cur->ne[2];

    ggml_tensor * k_view = ggml_view_1d(ctx, k,
            n_tokens*hparams.n_embd_k_gqa(il),
            ggml_row_size(k->type, hparams.n_embd_k_gqa(il))*head_cur);

    --> return ggml_cpy(ctx, k_cur, k_view);
}

ggml_tensor * llama_kv_cache_unified::cpy_v(ggml_context * ctx, ggml_tensor * v_cur, int32_t il, uint32_t head_cur) const {
    const int32_t ikv = map_layer_ids.at(il);

    auto * v = layers[ikv].v;

    const int64_t n_tokens = v_cur->ne[2];

    v_cur = ggml_reshape_2d(ctx, v_cur, hparams.n_embd_v_gqa(il), n_tokens);

    ggml_tensor * v_view = nullptr;

    if (!v_trans) {
        v_view = ggml_view_1d(ctx, v,
                n_tokens*hparams.n_embd_v_gqa(il),
                ggml_row_size(v->type, hparams.n_embd_v_gqa(il))*head_cur);
    } else {
        // note: the V cache is transposed when not using flash attention
        v_view = ggml_view_2d(ctx, v, n_tokens, hparams.n_embd_v_gqa(il),
                (v->ne[1])*ggml_element_size(v),
                (head_cur)*ggml_element_size(v));

        v_cur = ggml_transpose(ctx, v_cur);
    }

    --> return ggml_cpy(ctx, v_cur, v_view);
}

struct ggml_tensor * ggml_cpy(
        struct ggml_context * ctx,
        struct ggml_tensor * a,
        struct ggml_tensor * b) {
    --> return ggml_cpy_impl(ctx, a, b);
}

static struct ggml_tensor * ggml_cpy_impl(
        struct ggml_context * ctx,
        struct ggml_tensor  * a,
        struct ggml_tensor  * b) {
    // make a view of the destination
    struct ggml_tensor * result = ggml_view_tensor(ctx, b);
    if (strlen(b->name) > 0) {
        ggml_format_name(result, "%s (copy of %s)", b->name, a->name);
    } else {
        ggml_format_name(result, "%s (copy)", a->name);
    }

    result->op     = GGML_OP_CPY;
    result->src[0] = a;
    result->src[1] = b;

    return result;
}
```

[`llm_graph_context::build_attn_mha`](https://github.com/ggml-org/llama.cpp/blob/master/src/llama-graph.cpp#L1061)

```c++
ggml_tensor * cur = build_attn_mha(gf, q, k, v, kq_b, kq_mask, v_mla, kq_scale);
----------
ggml_tensor * llm_graph_context::build_attn_mha(
         ggml_cgraph * gf,
         ggml_tensor * q,
         ggml_tensor * k,
         ggml_tensor * v,
         ggml_tensor * kq_b,
         ggml_tensor * kq_mask,
         ggml_tensor * v_mla,
             float     kq_scale) const {
    const bool v_trans = v->nb[1] > v->nb[2];

    q = ggml_permute(ctx0, q, 0, 2, 1, 3);
    k = ggml_permute(ctx0, k, 0, 2, 1, 3);
    v = ggml_permute(ctx0, v, 0, 2, 1, 3);

    const auto n_tokens = q->ne[1];
    const auto n_head   = q->ne[2];
    ggml_tensor * kq = ggml_mul_mat(ctx0, k, q);
    // note: this op tends to require high floating point range
    // while for some models F16 is enough, for others it is not, so we default to F32 here
    ggml_mul_mat_set_prec(kq, GGML_PREC_F32);
    kq = ggml_soft_max_ext(ctx0, kq, kq_mask, kq_scale, hparams.f_max_alibi_bias);
    ggml_tensor * kqv = ggml_mul_mat(ctx0, v, kq);
    ggml_tensor * cur = ggml_permute(ctx0, kqv, 0, 2, 1, 3);
    cur = ggml_cont_2d(ctx0, cur, cur->ne[0]*n_head, n_tokens);
    ggml_build_forward_expand(gf, cur);
    return cur;
}
```

##### [`ggml_visit_parents`](https://github.com/ggml-org/llama.cpp/blob/master/ggml/src/ggml.c#L5794)

```c++
ggml_build_forward_expand(gf, cur);
----------
void ggml_build_forward_expand(struct ggml_cgraph * cgraph, struct ggml_tensor * tensor) {
    --> ggml_build_forward_impl(cgraph, tensor, true);
}

static void ggml_build_forward_impl(struct ggml_cgraph * cgraph, struct ggml_tensor * tensor, bool expand) {
    const int n0 = cgraph->n_nodes;
    --> ggml_visit_parents(cgraph, tensor);
    const int n_new = cgraph->n_nodes - n0;
}

static void ggml_visit_parents(struct ggml_cgraph * cgraph, struct ggml_tensor * node) {}
```

<details>
<summary>struct ggml_hash_set</summary>

```c++
struct ggml_hash_set {
    size_t size;
    ggml_bitset_t * used;       // whether or not the keys are in use i.e. set
    struct ggml_tensor ** keys; // actual tensors in the set, keys[i] is only defined if ggml_bitset_get(used, i)
};

static size_t ggml_hash_insert(struct ggml_hash_set * hash_set, struct ggml_tensor * key) {
    size_t h = ggml_hash(key) % hash_set->size;

    // linear probing
    size_t i = h;
    do {
        if (!ggml_bitset_get(hash_set->used, i)) {
            ggml_bitset_set(hash_set->used, i);
            hash_set->keys[i] = key;
            return i;
        }
        if (hash_set->keys[i] == key) {
            return GGML_HASHSET_ALREADY_EXISTS;
        }
        i = (i + 1) % hash_set->size;
    } while (i != h);

    // visited all hash table entries -> not found
    GGML_ABORT("fatal error");
}

static size_t ggml_hash_find_or_insert(struct ggml_hash_set * hash_set, struct ggml_tensor * key) {
    ...
        if (hash_set->keys[i] == key) {
            return i;
        }
    ...
}
```

</details>

```c++
// check if already visited
if (ggml_hash_insert(&cgraph->visited_hash_set, node) == GGML_HASHSET_ALREADY_EXISTS) {
    return;
}

for (int i = 0; i < GGML_MAX_SRC; ++i) {
    const int k =
        (cgraph->order == GGML_CGRAPH_EVAL_ORDER_LEFT_TO_RIGHT) ? i :
        (cgraph->order == GGML_CGRAPH_EVAL_ORDER_RIGHT_TO_LEFT) ? (GGML_MAX_SRC-1-i) :
        /* unknown order, just fall back to using i*/ i;
    if (node->src[k]) {
        ggml_visit_parents(cgraph, node->src[k]);
    }
}

if (node->op == GGML_OP_NONE && !(node->flags & GGML_TENSOR_FLAG_PARAM)) {
    // reached a leaf node, not part of the gradient graph (e.g. a constant)
    if (strlen(node->name) == 0) {
        ggml_format_name(node, "leaf_%d", cgraph->n_leafs);
    }
    cgraph->leafs[cgraph->n_leafs] = node;
    cgraph->n_leafs++;
} else {
    if (strlen(node->name) == 0) {
        ggml_format_name(node, "node_%d", cgraph->n_nodes);
    }
    cgraph->nodes[cgraph->n_nodes] = node;
    cgraph->n_nodes++;
}
```

#### [`ggml_backend_sched_reserve`](https://github.com/ggml-org/llama.cpp/blob/master/ggml/src/ggml-backend.cpp#L1549)

```c++
ggml_backend_sched_reserve(sched.get(), gf);
----------
bool ggml_backend_sched_reserve(ggml_backend_sched_t sched, struct ggml_cgraph * measure_graph) {}
```

```c++
--> ggml_backend_sched_split_graph;
--> ggml_backend_sched_synchronize;
--> ggml_gallocr_reserve_n;
ggml_backend_sched_reset(sched);
return true;
```

##### [`ggml_backend_sched_split_graph`](https://github.com/ggml-org/llama.cpp/blob/master/ggml/src/ggml-backend.cpp#L865)

```c++
ggml_backend_sched_split_graph(sched, measure_graph);
----------
// assigns backends to ops and splits the graph into subgraphs that can be computed on the same backend
static void ggml_backend_sched_split_graph(ggml_backend_sched_t sched, struct ggml_cgraph * graph) {}
```

<details>
<summary>struct ggml_backend_sched_split</summary>

```c++
struct ggml_backend_sched_split {
    int backend_id;
    int i_start;
    int i_end;
    struct ggml_tensor * inputs[GGML_SCHED_MAX_SPLIT_INPUTS];
    int n_inputs;
    // graph view of this split
    struct ggml_cgraph graph;
};
```

</details>

```c++
// reset splits
sched->n_splits = 0;
sched->n_graph_inputs = 0;
sched->is_reset = false;

struct ggml_init_params params = {
    /* .mem_size =   */ sched->context_buffer_size,
    /* .mem_buffer = */ sched->context_buffer,
    /* .no_alloc =   */ true
};

ggml_free(sched->ctx);
sched->ctx = ggml_init(params);
// pass 1: assign backends to ops with pre-allocated inputs
for (int i = 0; i < graph->n_leafs; i++)
    if (*leaf_backend_id == -1) *leaf_backend_id = ggml_backend_sched_backend_id_from_cur(sched, leaf);
for (int i = 0; i < graph->n_nodes; i++)
    if (*node_backend_id == -1) *node_backend_id = ggml_backend_sched_backend_id_from_cur(sched, node);
// pass 2: expand current backend assignments
// assign the same backend to adjacent nodes
// expand gpu backends (i.e. non last prio) up and down, ignoring cpu (the lowest priority backend)
// thus, cpu will never be used unless weights are on cpu, or there are no gpu ops between cpu ops
// ops unsupported by the backend being expanded will be left unassigned so that they can be assigned later when the locations of its inputs are known
// expand gpu down
for (int i = 0; i < graph->n_nodes; i++) if (*node_backend_id == sched->n_backends - 1) cur_backend_id = -1; // skip cpu (lowest prio backend)
// expand gpu up
for (int i = graph->n_nodes - 1; i >= 0; i--) if (*node_backend_id == sched->n_backends - 1) cur_backend_id = -1; // skip cpu (lowest prio backend)
// expand rest down
for (int i = 0; i < graph->n_nodes; i++)
// expand rest up
for (int i = graph->n_nodes - 1; i >= 0; i--)
    if (cur_backend_id != -1 && *node_backend_id == -1) ggml_backend_sched_set_if_supported(sched, node, cur_backend_id, node_backend_id);
// pass 3: upgrade nodes to higher prio backends with compatible buffer types
// if the tensor is already in the same buffer type (*) as another higher priority backend, we should move it there
// however, we also need to verify that the sources are in compatible buffer types
// (*) the actual requirement is more relaxed, the buffer type of the backend should be supported by all the users of this tensor further down the graph
// however, this is slow to verify, so we have a more strict requirement that the buffer type is the same
// this is not uncommon since multiple backends can use host memory, with the same buffer type (eg. BLAS and CPU)
// additionally, set remaining unassigned nodes to the backend with the most supported inputs
// only nodes that could not be assigned during expansion due to the backend not supporting the op should be unassigned at this point
for (int i = 0; i < graph->n_nodes; i++)
    if (*node_backend_id == -1); // unassigned node: find the backend with the most supported inputs
    else; // assigned node: upgrade to higher prio backend if possible
// pass 4: assign backends to remaining src from dst and view_src
for (int i = 0; i < graph->n_nodes; i++)
    if (node->view_src != NULL && *cur_backend_id == -1);
    // views are always on the same backend as the source
// pass 5: split graph, find tensors that need to be copied
int i_split = 0;
struct ggml_backend_sched_split * split = &sched->splits[0];
// find the backend of the first split, skipping view ops
int i = 0;
for (; i < graph->n_nodes; i++) {
    struct ggml_tensor * node = graph->nodes[i];
    if (!ggml_is_view_op(node->op)) {
        split->backend_id = tensor_backend_id(node);
        break;
    }
}
split->i_start = 0;
split->n_inputs = 0;
int cur_backend_id = split->backend_id;
for (; i < graph->n_nodes; i++)
    // check if we should start a new split based on the sources of the current node
    // find inputs that are not on the same backend
split->i_end = graph->n_nodes;
sched->n_splits = i_split + 1;
// swap node_backend_ids and leaf _backend_ids with prevs
// --> for `ggml_backend_sched_alloc_splits` func
int graph_size = std::max(graph->n_nodes, graph->n_leafs) + sched->n_splits*GGML_SCHED_MAX_SPLIT_INPUTS*2*sched->n_copies;
sched->graph.size = graph_size;
sched->graph.nodes = (ggml_tensor **) realloc(sched->graph.nodes, graph_size * sizeof(struct ggml_tensor *));
sched->graph.leafs = (ggml_tensor **) realloc(sched->graph.leafs, graph_size * sizeof(struct ggml_tensor *));
sched->graph.n_nodes = 0;
sched->graph.n_leafs = 0;
struct ggml_cgraph * graph_copy = &sched->graph;
//--> begin to construct the graph_copy, aka, sched->graph
// add leafs from the original graph
```

##### [`ggml_backend_sched_synchronize`](https://github.com/ggml-org/llama.cpp/blob/master/ggml/src/ggml-backend.cpp#L1599)

```c++
ggml_backend_sched_synchronize(sched);
----------
void ggml_backend_sched_synchronize(ggml_backend_sched_t sched) {}
```

```c++
for (int i = 0; i < sched->n_backends; i++) {
    --> ggml_backend_synchronize(sched->backends[i]);
}
if (!sched->is_alloc) {
    // if the graph is not already allocated, always use copy 0 after a synchronization
    // this ensures that during generation the same copy is used every time,
    // which avoids changes in the graph that could cause CUDA or other graphs to be disabled
    sched->cur_copy = 0;
}

void ggml_backend_synchronize(ggml_backend_t backend) {
    if (backend->iface.synchronize == NULL) {
        return;
    }

    backend->iface.synchronize(backend);
}
```

##### [`ggml_gallocr_reserve_n`](https://github.com/ggml-org/llama.cpp/blob/master/ggml/src/ggml-alloc.c#L673)


```c++
ggml_gallocr_reserve_n(sched->galloc, &sched->graph, sched->node_backend_ids, sched->leaf_backend_ids);
----------
bool ggml_gallocr_reserve_n(ggml_gallocr_t galloc, struct ggml_cgraph * graph, const int * node_buffer_ids, const int * leaf_buffer_ids) {}
```

<details>
<summary>struct hash_node</summary>

```c++
struct hash_node {
    int n_children;
    int n_views;
    int buffer_id;
    size_t offset; // offset within the buffer
    bool allocated;
};
```

</details>

```c++
size_t min_hash_size = graph->n_nodes + graph->n_leafs;
// add 25% margin to avoid hash collisions
min_hash_size += min_hash_size / 4;

// initialize hash table
if (galloc->hash_set.size < min_hash_size) {
    ggml_hash_set_free(&galloc->hash_set);
    galloc->hash_set = ggml_hash_set_new(min_hash_size);

    free(galloc->hash_values);
    galloc->hash_values = malloc(sizeof(struct hash_node) * galloc->hash_set.size);
}

// reset allocators
for (int i = 0; i < galloc->n_buffers; i++) {
    ggml_dyn_tallocr_reset(galloc->buf_tallocs[i]);
}

// allocate in hash table
--> ggml_gallocr_alloc_graph_impl;
```

<details>
<summary>struct tensor_alloc</summary>

```c++
struct tensor_alloc {
    int buffer_id;
    size_t offset;
    size_t size_max; // 0 = pre-allocated, unused, or view
};
```

</details>


<details>
<summary>struct node_alloc</summary>

```c++
struct node_alloc {
    struct tensor_alloc dst;
    struct tensor_alloc src[GGML_MAX_SRC];
};
```

</details>


```c++
// set the node_allocs from the hash table
if (galloc->n_nodes < graph->n_nodes) {
    free(galloc->node_allocs);
    galloc->node_allocs = calloc(graph->n_nodes, sizeof(struct node_alloc));
}
galloc->n_nodes = graph->n_nodes;
for (int i = 0; i < graph->n_nodes; i++) {
    struct ggml_tensor * node = graph->nodes[i];
    struct node_alloc * node_alloc = &galloc->node_allocs[i];
    if (node->view_src || node->data) {
        node_alloc->dst.buffer_id = -1;
        node_alloc->dst.offset = SIZE_MAX;
        node_alloc->dst.size_max = 0;
    } else {
        --> ggml_gallocr_hash_get --> struct hash_node * hn;
        node_alloc->dst.buffer_id = hn->buffer_id;
        node_alloc->dst.offset    = hn->offset;
        node_alloc->dst.size_max  = ggml_backend_buft_get_alloc_size(galloc->bufts[hn->buffer_id], node);
    }
    for (int j = 0; j < GGML_MAX_SRC; j++) {
        struct ggml_tensor * src = node->src[j];
        if (!src || src->view_src || src->data) {
            node_alloc->src[j].buffer_id = -1;
            node_alloc->src[j].offset = SIZE_MAX;
            node_alloc->src[j].size_max = 0;
        } else {
            --> ggml_gallocr_hash_get --> struct hash_node * hn;
            node_alloc->src[j].buffer_id = hn->buffer_id;
            node_alloc->src[j].offset   = hn->offset;
            node_alloc->src[j].size_max = ggml_backend_buft_get_alloc_size(galloc->bufts[hn->buffer_id], src);
        }
    }
}
```

<details>
<summary>struct leaf_alloc</summary>

```c++
struct leaf_alloc {
    struct tensor_alloc leaf;
};
```

</details>

```c++
if (galloc->n_leafs < graph->n_leafs) {
    free(galloc->leaf_allocs);
    galloc->leaf_allocs = calloc(graph->n_leafs, sizeof(galloc->leaf_allocs[0]));
}
galloc->n_leafs = graph->n_leafs;
for (int i = 0; i < graph->n_leafs; i++) {
    struct ggml_tensor * leaf = graph->leafs[i];
    --> ggml_gallocr_hash_get --> struct hash_node * hn;
    if (leaf->view_src || leaf->data) {
        galloc->leaf_allocs[i].leaf.buffer_id = -1;
        galloc->leaf_allocs[i].leaf.offset = SIZE_MAX;
        galloc->leaf_allocs[i].leaf.size_max = 0;
    } else {
        galloc->leaf_allocs[i].leaf.buffer_id = hn->buffer_id;
        galloc->leaf_allocs[i].leaf.offset = hn->offset;
        galloc->leaf_allocs[i].leaf.size_max = ggml_backend_buft_get_alloc_size(galloc->bufts[hn->buffer_id], leaf);
    }
}
```

```c++
// reallocate buffers if needed
for (int i = 0; i < galloc->n_buffers; i++) {
    // if the buffer type is used multiple times, we reuse the same buffer
    for (int j = 0; j < i; j++) {
        if (galloc->buf_tallocs[j] == galloc->buf_tallocs[i]) {
            galloc->buffers[i] = galloc->buffers[j];
            break;
        }
    }

    size_t cur_size = galloc->buffers[i] ? ggml_backend_buffer_get_size(galloc->buffers[i]) : 0;
    size_t new_size = ggml_dyn_tallocr_max_size(galloc->buf_tallocs[i]);

    // even if there are no tensors allocated in this buffer, we still need to allocate it to initialize views
    if (new_size > cur_size || galloc->buffers[i] == NULL) {
        ggml_backend_buffer_free(galloc->buffers[i]);
        // galloc->buffers[i] = ggml_backend_buft_alloc_buffer(galloc->bufts[i], new_size);
        --> ggml_backend_buft_alloc_buffer --> galloc->buffers[i];
        ggml_backend_buffer_set_usage(galloc->buffers[i], GGML_BACKEND_BUFFER_USAGE_COMPUTE);
    }
}

return true;
```

###### [`ggml_gallocr_alloc_graph_impl`](https://github.com/ggml-org/llama.cpp/blob/master/ggml/src/ggml-alloc.c#L566)


```c++
// allocate in hash table
ggml_gallocr_alloc_graph_impl(galloc, graph, node_buffer_ids, leaf_buffer_ids);
----------
static void ggml_gallocr_alloc_graph_impl(ggml_gallocr_t galloc, struct ggml_cgraph * graph, const int * node_buffer_ids, const int * leaf_buffer_ids) {}
```

```c++
// clear hash tables
ggml_hash_set_reset(&galloc->hash_set);
memset(galloc->hash_values, 0, sizeof(struct hash_node) * galloc->hash_set.size);

// allocate leafs
// these may be tensors that the application is not using in the graph, but may still want to allocate for other purposes
for (int i = 0; i < graph->n_leafs; i++) {
    struct ggml_tensor * leaf = graph->leafs[i];
    --> ggml_gallocr_allocate_node(galloc, leaf, get_node_buffer_id(leaf_buffer_ids, i));
}

// count number of children and views
// allocate other graph inputs and leafs first to avoid overwriting them
for (int i = 0; i < graph->n_nodes; i++) {
    struct ggml_tensor * node = graph->nodes[i];

    // TODO: better way to add external dependencies
    // GGML_OP_NONE does not appear normally in the graph nodes, but is used by ggml-backend to add dependencies to
    // control when some tensors are allocated and freed. in this case, the dependencies are in `src`, but the node
    // itself is never used and should not be considered a dependency
    if (ggml_is_view(node) && node->op != GGML_OP_NONE) {
        struct ggml_tensor * view_src = node->view_src;
        --> ggml_gallocr_hash_get(galloc, view_src)->n_views += 1;
    }

    if (node->flags & GGML_TENSOR_FLAG_INPUT) {
        --> ggml_gallocr_allocate_node(galloc, graph->nodes[i], get_node_buffer_id(node_buffer_ids, i));
    }

    for (int j = 0; j < GGML_MAX_SRC; j++) {
        struct ggml_tensor * src = node->src[j];
        --> ggml_gallocr_hash_get(galloc, src)->n_children += 1;

        // allocate explicit inputs
        if (src->flags & GGML_TENSOR_FLAG_INPUT) {
            --> ggml_gallocr_allocate_node(galloc, src, get_node_buffer_id(node_buffer_ids, i));
        }
    }
}

// allocate tensors
for (int i = 0; i < graph->n_nodes; i++) {
    struct ggml_tensor * node = graph->nodes[i];
    int buffer_id = get_node_buffer_id(node_buffer_ids, i);

    // allocate parents (only leafs need to be allocated at this point)
    for (int j = 0; j < GGML_MAX_SRC; j++) {
        struct ggml_tensor * parent = node->src[j];
        --> ggml_gallocr_allocate_node(galloc, parent, buffer_id);
    }

    // allocate node
    --> ggml_gallocr_allocate_node(galloc, node, buffer_id);

    // update parents
    for (int j = 0; j < GGML_MAX_SRC; j++) {
        struct ggml_tensor * parent = node->src[j];
        --> struct hash_node * p_hn = ggml_gallocr_hash_get(galloc, parent);
        p_hn->n_children -= 1;

        if (p_hn->n_children == 0 && p_hn->n_views == 0) {
            if (ggml_is_view(parent)) {
                struct ggml_tensor * view_src = parent->view_src;
                --> struct hash_node * view_src_hn = ggml_gallocr_hash_get(galloc, view_src);
                view_src_hn->n_views -= 1;
                if (view_src_hn->n_views == 0 && view_src_hn->n_children == 0 && view_src_hn->allocated) {
                    --> ggml_gallocr_free_node(galloc, view_src);
                }
            }
            else if (p_hn->allocated) {
                --> ggml_gallocr_free_node(galloc, parent);
            }
        }
    }
}
```

[`ggml_gallocr_allocate_node`](https://github.com/ggml-org/llama.cpp/blob/master/ggml/src/ggml-alloc.c#L478)

```c++
static void ggml_gallocr_allocate_node(ggml_gallocr_t galloc, struct ggml_tensor * node, int buffer_id) {
    GGML_ASSERT(buffer_id >= 0);
    // struct hash_node * hn = ggml_gallocr_hash_get(galloc, node);
    --> ggml_gallocr_hash_get --> struct hash_node * hn;

    if (!ggml_gallocr_is_allocated(galloc, node) && !ggml_is_view(node)) {
        hn->allocated = true;

        // try to reuse a parent's buffer (inplace)
        if (ggml_op_can_inplace(node->op)) {
            for (int i = 0; i < GGML_MAX_SRC; i++) {
                struct ggml_tensor * parent = node->src[i];

                // if the node's data is external, then we cannot re-use it
                if (!ggml_gallocr_is_own(galloc, parent)) continue;

                // outputs cannot be reused
                if (parent->flags & GGML_TENSOR_FLAG_OUTPUT || (parent->view_src != NULL && parent->view_src->flags & GGML_TENSOR_FLAG_OUTPUT)) continue;

                if (!ggml_are_same_layout(node, parent)) continue;

                // struct hash_node * p_hn = ggml_gallocr_hash_get(galloc, parent);
                --> ggml_gallocr_hash_get --> struct hash_node * p_hn;
                if (p_hn->n_children == 1 && p_hn->n_views == 0) {
                    if (ggml_is_view(parent)) {
                        struct ggml_tensor * view_src = parent->view_src;
                        // struct hash_node * view_src_hn = ggml_gallocr_hash_get(galloc, view_src);
                        --> ggml_gallocr_hash_get --> struct hash_node * view_src_hn;
                        if (view_src_hn->n_views == 1 && view_src_hn->n_children == 0 && view_src->data == parent->data) {
                            assert(view_src_hn->offset == p_hn->offset);
                            hn->buffer_id = p_hn->buffer_id;
                            hn->offset = p_hn->offset;
                            p_hn->allocated = false; // avoid freeing the parent
                            view_src_hn->allocated = false;
                            return;
                        }
                    } else {
                        hn->buffer_id = p_hn->buffer_id;
                        hn->offset = p_hn->offset;
                        p_hn->allocated = false; // avoid freeing the parent
                        return;
                    }
                }
            }
        }
        // allocate tensor from the buffer
        struct ggml_dyn_tallocr * alloc = galloc->buf_tallocs[buffer_id];
        ggml_backend_buffer_type_t buft = galloc->bufts[buffer_id];
        size_t size = ggml_backend_buft_get_alloc_size(buft, node);
        --> ggml_dyn_tallocr_alloc --> size_t offset;
        hn->buffer_id = buffer_id;
        hn->offset = offset;
    }
}
```

[`ggml_dyn_tallocr_alloc`](https://github.com/ggml-org/llama.cpp/blob/master/ggml/src/ggml-alloc.c#L153)

```c++
size_t offset = ggml_dyn_tallocr_alloc(alloc, size, node);
----------
static size_t ggml_dyn_tallocr_alloc(struct ggml_dyn_tallocr * alloc, size_t size, const struct ggml_tensor * tensor) {
    size = aligned_offset(NULL, size, alloc->alignment);
    // find the best fitting free block besides the last block
    int best_fit_block = -1;
    size_t best_fit_size = SIZE_MAX;
    for (int i = 0; i < alloc->n_free_blocks - 1; i++) {
        struct free_block * block = &alloc->free_blocks[i];
        if (block->size >= size && block->size <= best_fit_size) {
            best_fit_block = i;
            best_fit_size = block->size;
        }
    }

    if (best_fit_block == -1) {
        // the last block is our last resort
        struct free_block * block = &alloc->free_blocks[alloc->n_free_blocks - 1];
        // if (block->size >= size) {
        best_fit_block = alloc->n_free_blocks - 1;
    }

    struct free_block * block = &alloc->free_blocks[best_fit_block];
    size_t offset = block->offset;
    block->offset = offset + size;
    block->size -= size;
    if (block->size == 0) {
        // remove block if empty
        alloc->n_free_blocks--;
        for (int j = best_fit_block; j < alloc->n_free_blocks; j++) {
            alloc->free_blocks[j] = alloc->free_blocks[j+1];
        }
    }
    alloc->max_size = MAX(alloc->max_size, offset + size);
    return offset;
}
```

[`ggml_gallocr_free_node`](https://github.com/ggml-org/llama.cpp/blob/master/ggml/src/ggml-alloc.c#L545)

```c++
static void ggml_gallocr_free_node(ggml_gallocr_t galloc, struct ggml_tensor * node) {
    // graph outputs are never freed
    if (node->flags & GGML_TENSOR_FLAG_OUTPUT) {
        return;
    }

    // struct hash_node * hn = ggml_gallocr_hash_get(galloc, node);
    --> ggml_gallocr_hash_get --> struct hash_node * hn;
    size_t offset = hn->offset;
    int buffer_id = hn->buffer_id;
    struct ggml_dyn_tallocr * alloc = galloc->buf_tallocs[buffer_id];
    ggml_backend_buffer_type_t buft = galloc->bufts[buffer_id];
    size_t size = ggml_backend_buft_get_alloc_size(buft, node);
    --> ggml_dyn_tallocr_free_tensor;
    hn->allocated = false;
}
```


[`ggml_dyn_tallocr_free_tensor`](https://github.com/ggml-org/llama.cpp/blob/master/ggml/src/ggml-alloc.c#L238)

```c++
ggml_dyn_tallocr_free_tensor(alloc, offset, size, node);
----------
// this is a very naive implementation, but for our case the number of free blocks should be very small
static void ggml_dyn_tallocr_free_tensor(struct ggml_dyn_tallocr * alloc, size_t offset, size_t size, const struct ggml_tensor * tensor) {
    size = aligned_offset(NULL, size, alloc->alignment);

    // see if we can merge with an existing block
    for (int i = 0; i < alloc->n_free_blocks; i++) {
        struct free_block * block = &alloc->free_blocks[i];
        // check if ptr is at the end of the block
        if (block->offset + block->size == offset) {
            block->size += size;
            // check if we can merge with the next block
            if (i < alloc->n_free_blocks - 1 && block->offset + block->size == alloc->free_blocks[i+1].offset) {
                block->size += alloc->free_blocks[i+1].size;
                alloc->n_free_blocks--;
                for (int j = i+1; j < alloc->n_free_blocks; j++) {
                    alloc->free_blocks[j] = alloc->free_blocks[j+1];
                }
            }
            return;
        }
        // check if ptr is at the beginning of the block
        if (offset + size == block->offset) {
            block->offset = offset;
            block->size += size;
            // check if we can merge with the previous block
            if (i > 0 && alloc->free_blocks[i-1].offset + alloc->free_blocks[i-1].size == block->offset) {
                alloc->free_blocks[i-1].size += block->size;
                alloc->n_free_blocks--;
                for (int j = i; j < alloc->n_free_blocks; j++) {
                    alloc->free_blocks[j] = alloc->free_blocks[j+1];
                }
            }
            return;
        }
    }
    // otherwise, add a new block
    // insert the new block in the correct position to keep the array sorted by address (to make merging blocks faster)
    int insert_pos = 0;
    while (insert_pos < alloc->n_free_blocks && alloc->free_blocks[insert_pos].offset < offset) {
        insert_pos++;
    }
    // shift all blocks from insert_pos onward to make room for the new block
    for (int i = alloc->n_free_blocks; i > insert_pos; i--) {
        alloc->free_blocks[i] = alloc->free_blocks[i-1];
    }
    // insert the new block
    alloc->free_blocks[insert_pos].offset = offset;
    alloc->free_blocks[insert_pos].size = size;
    alloc->n_free_blocks++;
}
```

###### [`ggml_gallocr_hash_get`](https://github.com/ggml-org/llama.cpp/blob/master/ggml/src/ggml-alloc.c#L465)

```c++
struct hash_node * hn = ggml_gallocr_hash_get(galloc, node);
struct hash_node * hn = ggml_gallocr_hash_get(galloc, src);
struct hash_node * hn = ggml_gallocr_hash_get(galloc, leaf);
----------
static struct hash_node * ggml_gallocr_hash_get(ggml_gallocr_t galloc, struct ggml_tensor * t) {
    size_t i = ggml_hash_find_or_insert(&galloc->hash_set, t);
    return &galloc->hash_values[i];
}
```

## *[`llama_decode`](https://github.com/ggml-org/llama.cpp/blob/master/src/llama-context.cpp#886)

<details>
<summary>struct llama_batch</summary>

```c++
    // Input data for llama_decode
    // A llama_batch object can contain input about one or many sequences
    // The provided arrays (i.e. token, embd, pos, etc.) must have size of n_tokens
    //
    // - token  : the token ids of the input (used when embd is NULL)
    // - embd   : token embeddings (i.e. float vector of size n_embd) (used when token is NULL)
    // - pos    : the positions of the respective token in the sequence
    //            (if set to NULL, the token position will be tracked automatically by llama_decode)
    // - seq_id : the sequence to which the respective token belongs
    //            (if set to NULL, the sequence ID will be assumed to be 0)
    // - logits : if zero, the logits (and/or the embeddings) for the respective token will not be output
    //            (if set to NULL, only the logits for last token will be returned)
    //
    typedef struct llama_batch {
        int32_t n_tokens;

        llama_token  *  token;
        float        *  embd;
        llama_pos    *  pos;
        int32_t      *  n_seq_id; // TODO: remove, should belong to only 1 sequence
        llama_seq_id ** seq_id;   // TODO: become llama_seq_id * seq_id;
        int8_t       *  logits;   // TODO: rename this to "output"
    } llama_batch;


struct llama_batch llama_batch_get_one(
             llama_token * tokens,
                 int32_t   n_tokens) {
    return {
        /*n_tokens       =*/ n_tokens,
        /*tokens         =*/ tokens,
        /*embd           =*/ nullptr,
        /*pos            =*/ nullptr,
        /*n_seq_id       =*/ nullptr,
        /*seq_id         =*/ nullptr,
        /*logits         =*/ nullptr,
    };
}
```

</details>

```c++
llama_decode(lctx, llama_batch_get_one(tmp.data(), std::min(tmp.size(), (size_t) params.n_batch)));
----------
int32_t llama_decode(
        llama_context * ctx,
          llama_batch   batch) {
    --> const int ret = ctx->decode(batch);
    return ret;
}

int llama_context::decode(llama_batch & inp_batch) {}
```

```c++
--> llama_batch_allocr::init;
const llama_batch & batch = batch_allocr->get_batch();

const auto & vocab   = model.vocab;
const auto & hparams = model.hparams;
const int32_t n_vocab = vocab.n_tokens();
const int64_t n_tokens_all = batch.n_tokens;
const uint32_t n_outputs_all = batch_allocr->get_n_outputs();
n_queued_tokens += n_tokens_all;
bool did_optimize = false;
// handle any pending defrags/shifts
--> kv_self_update;
--> llama_kv_cache_unified::init_batch --> llama_memory_state_ptr mstate;
// reserve output buffer
--> output_reserve(n_outputs_all)
int64_t n_outputs_prev = 0;
const auto & ubatch = mstate->get_ubatch();
// count the outputs in this u_batch
int32_t n_outputs_new = 0;
for (uint32_t i = 0; i < ubatch.n_tokens; i++) n_outputs_new += (int32_t) (ubatch.output[i] != 0);
// needs to happen before the graph is built
// llama_context:: int32_t n_outputs; // number of actually-used outputs in the current ubatch or last logical batch
n_outputs = n_outputs_new;
ggml_backend_sched_reset(sched.get());

--> llama_context::process_ubatch --> const auto res;

    // plot the computation graph in dot format (for debugging purposes)
    //if (n_past%100 == 0) {
    //    ggml_graph_dump_dot(gf, NULL, "llama.dot");
    //}

auto * t_logits = res->get_logits();
// extract logits
ggml_backend_t backend_res = ggml_backend_sched_get_tensor_backend(sched.get(), t_logits);
float * logits_out = logits + n_outputs_prev*n_vocab;
--> ggml_backend_tensor_get_async --> logits_out
n_outputs_prev += n_outputs;

// set to total number of outputs in the batch, for use in llama_get_logits_ith
n_outputs = n_outputs_all;
// set output mappings
auto & out_ids = mstate->out_ids();

//llama_context:: std::vector<int32_t> output_ids; // map batch token positions to ids of the logits and embd buffers
for (int64_t i = 0; i < n_outputs_all; ++i)
    output_ids[out_ids[i]] = i;

// wait for the computation to finish (automatically done when obtaining the model output)
//synchronize();

// Reset state for the next token before backend sync, to allow the CPU activities in the reset to overlap with device computation.
ggml_backend_sched_reset(sched.get());
```

### [`llama_batch_allocr::llama_batch_allocr`](https://github.com/ggml-org/llama.cpp/blob/master/src/llama-batch.cpp#L299)

<details>
<summary>class llama_batch_allocr</summary>

```c++
typedef int32_t llama_pos;
typedef int32_t llama_seq_id;

class llama_batch_allocr {
    llama_batch batch;

    uint32_t n_outputs;

    std::array<llama_seq_id, 1> seq_id_0 = { 0 }; // default sequence id

    std::vector<llama_pos>      pos;
    std::vector<int32_t>        n_seq_id;
    std::vector<llama_seq_id *> seq_id;
    std::vector<int8_t>         output;

    std::vector<std::set<llama_pos>> seq_pos; // seq_pos[s]: the set of positions in sequence s
    std::vector<std::vector<bool>>   seq_cpl; // seq_cpl[s0][s1]: if sequence s0 is coupled to sequence s1

    int debug;
};
```

</details>

```c++
batch_allocr->init(batch_inp, model.vocab, memory.get());
----------
bool llama_batch_allocr::init(
        const llama_batch & batch_inp,
        const llama_vocab & vocab,
        const llama_memory_i * memory) {}
```

```c++
--> llama_batch_allocr::clear;
batch = batch_inp;
// validate input batch
if (batch.token) for (int32_t i = 0; i < batch.n_tokens; ++i) if (batch.token[i] < 0 || (uint32_t) batch.token[i] >= vocab.n_tokens()) LLAMA_LOG_ERROR("%s: invalid token[%d] = %d\n", __func__, i, batch.token[i]);
if (batch.seq_id) for (int32_t i = 0; i < batch.n_tokens; ++i) if (batch.seq_id && (batch.seq_id[i][s] < 0 || batch.seq_id[i][s] >= LLAMA_MAX_SEQ)) LLAMA_LOG_ERROR("%s: invalid seq_id[%d][%d] = %d > %d\n", __func__, i, s, batch.seq_id[i][s], LLAMA_MAX_SEQ);

// auto-generate missing fields
if (!batch.n_seq_id) {
    n_seq_id.resize(batch.n_tokens);
    for (int32_t i = 0; i < batch.n_tokens; i++) n_seq_id[i] = seq_id_0.size();
    batch.n_seq_id = n_seq_id.data();
}
if (!batch.seq_id) {
    seq_id.resize(batch.n_tokens + 1);
    seq_id[batch.n_tokens] = NULL;
    for (int32_t i = 0; i < batch.n_tokens; i++) seq_id[i] = seq_id_0.data();
    batch.seq_id = seq_id.data();
}
if (!batch.pos) {
    pos.resize(batch.n_tokens);
    // initialize the starting position for each sequence based on the positions in the memory
    llama_pos p0[LLAMA_MAX_SEQ];
    for (int32_t s = 0; s < LLAMA_MAX_SEQ; ++s) p0[s] = memory? memory->seq_pos_max(s) + 1 : 0;
    for (int32_t i = 0; i < batch.n_tokens; i++) {
        const llama_seq_id seq_id = batch.seq_id[i][0];
        pos[i] = p0[seq_id];
        for (int32_t s = 0; s < batch.n_seq_id[i]; ++s) p0[batch.seq_id[i][s]] = pos[i] + 1;
    }
    batch.pos = pos.data();
}
if (!batch.logits) {
    // by default return the output only for the last token
    logits.resize(batch.n_tokens);
    logits[logits.size() - 1] = true;
    batch.logits = logits.data();
}

// compute stats
for (int32_t i = 0; i < batch.n_tokens; ++i) n_outputs += batch.logits[i] != 0;

// determine coupled sequences these are pairs of sequences that have at least one token in the input batch that is assigned to both of them
for (int32_t i = 0; i < batch.n_tokens; ++i)
    for (int32_t s = 0; s < batch.n_seq_id[i]; ++s)
        seq_pos[batch.seq_id[i][s]].insert(batch.pos[i]);
        // mark that sequence s1 is coupled to s0
        if (s > 0) seq_cpl[batch.seq_id[i][s]][batch.seq_id[i][0]] = true;

// consistency checks
memory && seq_pos_min(s) == memory->seq_pos_max(s) + 1; //sequence %d does start from the last position stored in the memory
seq_pos_max(s) - seq_pos_min(s) < (int) seq_pos[s].size(); //sequence %d positions are continuous
if (memory) {
    for (int32_t s0 = 0; s0 < LLAMA_MAX_SEQ; ++s0)
    for (int32_t s1 = 0; s1 < LLAMA_MAX_SEQ; ++s1)
        if (seq_cpl[s0][s1])
            memory->seq_pos_min(s0) == memory->seq_pos_min(s1) ||
            memory->seq_pos_max(s0) == memory->seq_pos_max(s1);
            //sequence %d is coupled to %d in the input batch, but have divereged
}

return true;
```

[`llama_batch_allocr::clear`](https://github.com/ggml-org/llama.cpp/blob/master/src/llama-batch.cpp#L532)

```c++
clear();
----------
void llama_batch_allocr::clear() {
    n_outputs = 0;

    batch = {};
    pos.clear();
    n_seq_id.clear();
    seq_id.clear();
    output.clear();

    for (auto & cur : seq_pos) {
        cur.clear();
    }

    for (auto & cur : seq_cpl) {
        std::fill(cur.begin(), cur.end(), false);
    }
}
```

### [`llama_context::kv_self_update`](https://github.com/ggml-org/llama.cpp/blob/master/src/llama-context.cpp#L437)


```c++
kv_self_update(false);
----------
// deprecated
bool llama_context::kv_self_update(bool optimize) {}
```

```c++
// TODO: remove in the future
optimize |= memory_force_optimize;
memory_force_optimize = false;

--> memory --> llama_kv_cache_unified::init_update --> const auto mstate;
switch (mstate->get_status())
case LLAMA_MEMORY_STATUS_NO_UPDATE: return false; // no updates need to be performed
```

[`lama_kv_cache_unified::init_update`](https://github.com/ggml-org/llama.cpp/blob/master/src/llama-kv-cache-unified.cpp#L340)

```c++
const auto mstate = memory->init_update(this, optimize);
----------
llama_memory_state_ptr llama_kv_cache_unified::init_update(llama_context * lctx, bool optimize) {
    bool do_shift = get_has_shift();

    defrag_info dinfo;

    // see if we need to defrag
    {
        bool do_defrag = optimize;

        const auto thold = lctx->get_cparams().defrag_thold;

        if (!do_defrag && thold > 0.0f) {
            const auto n_kv = cells.used_max_p1();

            // - do not defrag small contexts (i.e. < 2048 tokens)
            // - count the padding towards the number of used tokens
            const float fragmentation = n_kv >= 2048 ? std::max(0.0f, 1.0f - (float(cells.get_used() + n_pad)/n_kv)) : 0.0f;

            if (fragmentation > thold) {
                LLAMA_LOG_DEBUG("%s: fragmentation: %.2f - requesting defrag\n", __func__, fragmentation);

                do_defrag = true;
            }
        }

        if (do_defrag) {
            dinfo = defrag_prepare(lctx->graph_max_nodes());
        }
    }

    --> return std::make_unique<llama_kv_cache_unified_state>(this, lctx, do_shift, std::move(dinfo));
}
```

[`llama_kv_cache_unified_state::llama_kv_cache_unified_state`](https://github.com/ggml-org/llama.cpp/blob/master/src/llama-kv-cache-unified.cpp#L1737)

```c++
llama_kv_cache_unified_state::llama_kv_cache_unified_state(
        llama_kv_cache_unified * kv,
        llama_context * lctx,
        bool do_shift,
        defrag_info dinfo) : status(LLAMA_MEMORY_STATUS_SUCCESS), kv(kv), lctx(lctx), do_shift(do_shift), dinfo(std::move(dinfo)) {
    if (!do_shift && this->dinfo.empty()) {
        status = LLAMA_MEMORY_STATUS_NO_UPDATE;
    }
}
```

### [`llama_kv_cache_unified::init_batch`](https://github.com/ggml-org/llama.cpp/blob/master/src/llama-kv-cache-unified.cpp#L310)

```c++
mstate = memory->init_batch(batch, cparams.n_ubatch, embd_pooled);
----------
llama_memory_state_ptr llama_kv_cache_unified::init_batch(
            const llama_batch & batch,
            uint32_t n_ubatch,
            bool embd_pooled) {}
```

```c++
--> llama_sbatch::llama_sbatch --> llama_sbatch sbatch;
--> llama_sbatch::split_simple --> std::vector<llama_ubatch> ubatches.push_back;
--> llama_kv_cache_unified::prepare --> llama_kv_cache_unified::ubatch_heads heads;
--> return llama_kv_cache_unified_state::llama_kv_cache_unified_state;
```

#### [`llama_sbatch::llama_sbatch`](https://github.com/ggml-org/llama.cpp/blob/master/src/llama-batch.cpp#L201)

```c++
auto sbatch = llama_sbatch(batch, hparams.n_embd, true);
----------
llama_sbatch::llama_sbatch(const llama_batch & batch, size_t n_embd, bool simple_split) {}
```

<details>
<summary>struct llama_sbatch</summary>

```c++
struct llama_sbatch {
    // tokens left in this batch
    size_t n_tokens;

    size_t n_embd;

    // sorted indices into the batch
    std::vector<int64_t> ids;
    // batch indices of the output
    std::vector<int64_t> out_ids;
    std::vector<llama_sbatch_seq> seq;

    const llama_batch * batch = nullptr;

    // buffers for the ubatches
    // TODO: very hacky, this needs a complete rework
    struct ubatch_data {
        std::vector<llama_token>    token;
        std::vector<float>          embd;
        std::vector<llama_pos>      pos;
        std::vector<int32_t>        n_seq_id;
        std::vector<llama_seq_id *> seq_id;
        std::vector<int8_t>         output;
    };

    std::vector<ubatch_data> udatas;

    llama_ubatch reserve_ubatch(size_t n_ubatch, bool has_embd = false);

    void add_seq_to_ubatch(llama_ubatch & ubatch, llama_sbatch_seq & seq, size_t length);

    // simple split, unknown number of sequences of unequal lengths
    llama_ubatch split_simple(size_t n_ubatch);

    // make batches of equal-length sequences
    llama_ubatch split_equal(size_t n_ubatch);

    // sequence-wise split
    llama_ubatch split_seq(size_t n_ubatch);

    llama_sbatch() = default;
    llama_sbatch(const llama_batch & batch, size_t n_embd, bool simple_split = false);
};
```

</details>

<details>
<summary>struct llama_sbatch_seq</summary>

```c++
struct llama_sbatch_seq {
    int32_t n_seq_id;

    llama_seq_id * seq_id;

    size_t offset;
    size_t length;
};
```

</details>

```c++
this->batch = &batch;
this->n_embd = n_embd;
n_tokens = batch.n_tokens;
ids.resize(n_tokens);
out_ids.clear();
// TODO: reserve out_ids and seq
for (size_t i = 0; i < n_tokens; ++i) ids[i] = i;
if (simple_split) {
    seq.resize(1);
    llama_sbatch_seq & s = seq[0];
    s.n_seq_id = 0;
    s.seq_id = nullptr;
    s.offset = 0;
    s.length = n_tokens;
    return;
}
```

#### [`llama_sbatch::split_simple`](https://github.com/ggml-org/llama.cpp/blob/master/src/llama-batch.cpp#L149)

```c++
ubatches.push_back(sbatch.split_simple(n_ubatch));
----------
llama_ubatch llama_sbatch::split_simple(size_t n_ubatch) {}
```

```c++
n_ubatch = n_tokens < n_ubatch ? n_tokens : n_ubatch;
--> llama_sbatch::reserve_ubatch --> llama_ubatch ubatch;
ubatch.equal_seqs = false;
llama_sbatch_seq & s = seq[0];
size_t length = s.length < n_ubatch ? s.length : n_ubatch;
--> llama_sbatch::add_seq_to_ubatch;
return ubatch;
```

##### [`llama_sbatch::reserve_ubatch`](https://github.com/ggml-org/llama.cpp/blob/master/src/llama-batch.cpp#L13)

```c++
llama_ubatch ubatch = reserve_ubatch(n_ubatch, /* has_embd */ batch->embd != nullptr);
----------
llama_ubatch llama_sbatch::reserve_ubatch(size_t n_ubatch, bool has_embd) {}
```

<details>
<summary>struct llama_ubatch</summary>

```c++
// very similar to llama_batch,
// but has more metadata about sequences
struct llama_ubatch {
    bool equal_seqs;
    // TODO: whole_seqs for embeddings?

    uint32_t n_tokens;     // total tokens (n_seq_tokens * n_seqs)
    uint32_t n_seq_tokens; // tokens per sequence
    uint32_t n_seqs;

    llama_token  *  token;    // [n_tokens]
    float        *  embd;     // [n_embd, n_tokens]
    llama_pos    *  pos;      // [n_tokens]
    int32_t      *  n_seq_id; // [n_seqs]
    llama_seq_id ** seq_id;   // [n_seqs]
    int8_t       *  output;   // [n_tokens]
};
```

</details>

```c++
// simple split 的话，下面这些都用不上

udatas.push_back({});

auto & udata = udatas.back();

udata.token.resize(n_ubatch);
udata.pos.resize(n_ubatch);
udata.n_seq_id.resize(n_ubatch);
udata.seq_id.resize(n_ubatch);
udata.output.resize(n_ubatch);

llama_ubatch ubatch = {
    /*equal_seqs   =*/ true,
    /*n_tokens     =*/ 0,
    /*n_seq_tokens =*/ 0,
    /*n_seqs       =*/ 0,
    /*token        =*/ udata.token.data(),
    /*embd         =*/ nullptr,
    /*pos          =*/ udata.pos.data(),
    /*n_seq_id     =*/ udata.n_seq_id.data(),
    /*seq_id       =*/ udata.seq_id.data(),
    /*output       =*/ udata.output.data(),
};

return ubatch;
```

##### [`llama_sbatch::add_seq_to_ubatch`](https://github.com/ggml-org/llama.cpp/blob/master/src/llama-batch.cpp#L52)

```c++
add_seq_to_ubatch(ubatch, s, length);
----------
void llama_sbatch::add_seq_to_ubatch(llama_ubatch & ubatch, llama_sbatch_seq & seq, size_t length) {}
```

```c++
// llama_sbatch:: const llama_batch * batch;

// simple split
ubatch.token = batch->token + seq.offset;
ubatch.embd = nullptr;
ubatch.pos = batch->pos + seq.offset;
ubatch.n_seq_id = batch->n_seq_id + seq.offset;
ubatch.seq_id = batch->seq_id + seq.offset;
ubatch.output = batch->logits + seq.offset;
for (size_t i = 0; i < length; ++i) if (ubatch.output[i] != 0) { out_ids.push_back(seq.offset + i); }
// if (ubatch.n_tokens == 0 && ubatch.n_seqs == 0)
ubatch.n_seq_tokens = 1;
ubatch.n_tokens += length;
ubatch.n_seqs += length; // virtual sequences for simple splits
seq.offset += length;
seq.length -= length;
n_tokens -= length;
```

#### [`llama_kv_cache_unified::prepare`](https://github.com/ggml-org/llama.cpp/blob/master/src/llama-kv-cache-unified.cpp#L373)

```c++
auto heads = prepare(ubatches);
----------
llama_kv_cache_unified::ubatch_heads llama_kv_cache_unified::prepare(const std::vector<llama_ubatch> & ubatches) {}
```

<details>
<summary>llama_kv_cache_unified::ubatch_heads</summary>

```c++
using llama_kv_cache_unified::ubatch_heads = std::vector<uint32_t>;
```

</details>

```c++
llama_kv_cache_unified::ubatch_heads res;

struct state {
    uint32_t head_old; // old position of the head, before placing the ubatch
    uint32_t head_new; // new position of the head, after placing the ubatch

    llama_kv_cells_unified cells; // copy of the old cells, before placing the ubatch
};

// remember the old state of the cells so we can restore it in the end
std::vector<state> states;

bool success = true;
//for (const auto & ubatch : ubatches)
// only find a suitable slot for the ubatch. don't modify the cells yet
--> llama_kv_cache_unified::find_slot ---> const int32_t head_new;
// remeber the position that we found
res.push_back(head_new);
// store the old state of the cells in the recovery stack
--> states.push_back({head, (uint32_t) head_new, cells.cp(head_new, ubatch.n_tokens)});
// 意思就是打上标记，此地被占用了，for循环的下一次find_slot不会找上这一块
// now emplace the ubatch
--> llama_kv_cache_unified::apply_ubatch;
// iterate backwards and restore the cells to their original state
for (auto it = states.rbegin(); it != states.rend(); ++it)
    --> cells.set(it->head_new, it->cells);
    head = it->head_old;
return res;
```

##### [`llama_kv_cache_unified::find_slot`](https://github.com/ggml-org/llama.cpp/blob/master/src/llama-kv-cache-unified.cpp#L510)

```c++
const int32_t head_new = find_slot(ubatch);
----------
int32_t llama_kv_cache_unified::find_slot(const llama_ubatch & ubatch) const {}
```

```c++
const uint32_t n_tokens = ubatch.n_tokens;
uint32_t head_cur = this->head;

// if we have enough unused cells before the current head ->
// better to start searching from the beginning of the cache, hoping to fill it
if (head_cur > cells.get_used() + 2*n_tokens) head_cur = 0;
while (true) {
    if (head_cur + n_tokens > cells.size()) {
        head_cur = 0;
        continue;
    }
    bool found = true;
    for (uint32_t i = 0; i < n_tokens; i++) {
        // can we use this cell? either:
        //  - the cell is empty
        //  - the cell is occupied only by one sequence: <-why?:/
        //    - (disabled) mask causally, if the sequence is the same as the one we are inserting
        //    - mask SWA, using current max pos for that sequence in the cache, always insert in the cell with minimum pos
        bool can_use = cells.is_empty(head_cur + i);
        if (!can_use && cells.seq_count(head_cur + i) == 1) {
            const llama_pos pos_cell = cells.pos_get(head_cur + i);
            const llama_seq_id seq_id_cell = cells.seq_get(head_cur + i);
            // (disabled) causal mask
            // note: it's better to purge any "future" tokens beforehand
            //if (cells.seq_has(head_cur + i, seq_id)) {
            //    can_use = pos_cell >= pos;
            //}
            // SWA mask
            if (!can_use && is_masked_swa(pos_cell, cells.seq_pos_max(seq_id_cell) + 1)) can_use = true;
        }
        if (!can_use) {
            found = false;
            head_cur += i + 1;
            break;
        }
    }
    if (found) break;
}
return head_cur;
```

##### [`llama_kv_cache_unified::apply_ubatch`](https://github.com/ggml-org/llama.cpp/blob/master/src/llama-kv-cache-unified.cpp#L646)

```c++
apply_ubatch(head_new, ubatch);
----------
void llama_kv_cache_unified::apply_ubatch(uint32_t head_cur, const llama_ubatch & ubatch) {}
```

```c++
// keep track of the max sequence position that we would overwrite with this ubatch for non-SWA cache, this would be always empty
llama_seq_id seq_pos_max_rm[LLAMA_MAX_SEQ];
for (int s = 0; s < LLAMA_MAX_SEQ; ++s) {
    seq_pos_max_rm[s] = -1;
}

for (uint32_t s = 0; s < ubatch.n_seqs; ++s) {
    // for (uint32_t j = 0; j < ubatch.n_seq_tokens; ++j) {
        // const uint32_t idx = s*ubatch.n_seq_tokens + j;
        // head_cur + idx
    if (!cells.is_empty(head_cur + s)) {
        assert(cells.seq_count(head_cur + s) == 1);
        const llama_seq_id seq_id = cells.seq_get(head_cur + s);
        const llama_pos    pos    = cells.pos_get(head_cur + s);
        seq_pos_max_rm[seq_id] = std::max(seq_pos_max_rm[seq_id], pos);
        cells.rm(head_cur + s);
    }
    --> cells.pos_set(head_cur + s, ubatch.pos[idx]);
    // TODO: fix indexing [UBATCH_IDX]
    for (int32_t i = 0; i < ubatch.n_seq_id[s]; i++) {
        --> cells.seq_add(head_cur + s, ubatch.seq_id[s][i]);
    }
}
// move the head at the end of the slot
head = head_cur + ubatch.n_tokens;
```

#### [`llama_kv_cache_unified_state::llama_kv_cache_unified_state`](https://github.com/ggml-org/llama.cpp/blob/master/src/llama-kv-cache-unified.cpp#L1747)

```c++
return std::make_unique<llama_kv_cache_unified_state>(
        this, std::move(sbatch), std::move(heads), std::move(ubatches));
----------
llama_kv_cache_unified_state::llama_kv_cache_unified_state(
        llama_kv_cache_unified * kv,
        llama_sbatch sbatch,
        llama_kv_cache_unified::ubatch_heads heads,
        std::vector<llama_ubatch> ubatches) :
        status(LLAMA_MEMORY_STATUS_SUCCESS), kv(kv), sbatch(std::move(sbatch)),
        heads(std::move(heads)), ubatches(std::move(ubatches)) {
}
```

### **[`llama_context::process_ubatch`](https://github.com/ggml-org/llama.cpp/blob/master/src/llama-context.cpp#L681)

```c++
const auto res = process_ubatch(ubatch, LLM_GRAPH_TYPE_DECODER, mstate.get(), status);
----------
llm_graph_result_ptr llama_context::process_ubatch(const llama_ubatch & ubatch, llm_graph_type gtype, llama_memory_state_i * mstate, ggml_status & ret) {}
```

```c++
--> graph_init --> ggml_cgraph * gf;
// auto res = graph_build(ctx_compute.get(), gf, ubatch, LLM_GRAPH_TYPE_DECODER, mstate);
--> llama_context::graph_build --> llama_model::build_graph --> llm_build_llama::llm_build_llama --> std::unique_ptr<llm_graph_result> res;
--> ggml_backend_sched_alloc_graph;
--> ggml_backend_cpu_buffer_set_tensor;/llama_kv_cache_unified::set_input_kq_mask;
--> llama_context::graph_compute --> ggml_backend_sched_graph_compute_async --> ggml_status ggml_backend_sched_compute_splits --> ggml_backend_graph_compute_async --> ggml_backend_cpu_graph_compute;
ret = GGML_STATUS_SUCCESS;
return res;
```

#### [`ggml_backend_sched_alloc_graph`](https://github.com/ggml-org/llama.cpp/blob/master/ggml/src/ggml-backend.cpp#L1565)

```c++
ggml_backend_sched_alloc_graph(sched.get(), gf);
----------
bool ggml_backend_sched_alloc_graph(ggml_backend_sched_t sched, struct ggml_cgraph * graph) {}
```

```c++
--> ggml_backend_sched_split_graph;
--> ggml_backend_sched_alloc_splits;
sched->is_alloc = true;
return true;
```

##### [`ggml_backend_sched_alloc_splits`](https://github.com/ggml-org/llama.cpp/blob/master/ggml/src/ggml-backend.cpp#L1321)

```c++
ggml_backend_sched_alloc_splits(sched);
----------
static bool ggml_backend_sched_alloc_splits(ggml_backend_sched_t sched) {}
```

```c++
bool backend_ids_changed = false;
for (int i = 0; i < sched->graph.n_nodes; i++) {
    if (sched->node_backend_ids[i] != sched->prev_node_backend_ids[i] &&
        sched->bufts[sched->node_backend_ids[i]] != sched->bufts[sched->prev_node_backend_ids[i]]) {
        backend_ids_changed = true;
        break;
    }
}
if (!backend_ids_changed) {
    for (int i = 0; i < sched->graph.n_leafs; i++) {
        if (sched->leaf_backend_ids[i] != sched->prev_leaf_backend_ids[i] &&
            sched->bufts[sched->leaf_backend_ids[i]] != sched->bufts[sched->prev_leaf_backend_ids[i]]) {
            backend_ids_changed = true;
            break;
        }
    }
}
// allocate graph
--> if (backend_ids_changed || !ggml_gallocr_alloc_graph(sched->galloc, &sched->graph)) {
    // the re-allocation may cause the split inputs to be moved to a different address
    // synchronize without ggml_backend_sched_synchronize to avoid changing cur_copy
    --> for (int i = 0; i < sched->n_backends; i++) ggml_backend_synchronize(sched->backends[i]);
    --> ggml_gallocr_reserve_n(sched->galloc, &sched->graph, sched->node_backend_ids, sched->leaf_backend_ids);
    --> if (!ggml_gallocr_alloc_graph(sched->galloc, &sched->graph)) {
        GGML_LOG_ERROR("%s: failed to allocate graph\n", __func__);
        return false;
    }
}
return true;
```

[`ggml_gallocr_alloc_graph`](https://github.com/ggml-org/llama.cpp/blob/master/ggml/src/ggml-alloc.c#L871)

```c++
ggml_gallocr_alloc_graph(sched->galloc, &sched->graph);
----------
bool ggml_gallocr_alloc_graph(ggml_gallocr_t galloc, struct ggml_cgraph * graph) {}
```

```c++
--> if (ggml_gallocr_needs_realloc(galloc, graph)) {
    if (galloc->n_buffers == 1) {
        --> if (!ggml_gallocr_reserve(galloc, graph)) return false;
            return ggml_gallocr_reserve_n(galloc, graph, NULL, NULL);
    } else return false;
}
// reset buffers
for (int i = 0; i < galloc->n_buffers; i++) {
    if (galloc->buffers[i] != NULL) {
        --> ggml_backend_buffer_reset(galloc->buffers[i]);
    }
}
// allocate the graph tensors from the previous assignments leafs
for (int i = 0; i < graph->n_leafs; i++) {
    struct ggml_tensor * leaf = graph->leafs[i];
    struct leaf_alloc * leaf_alloc = &galloc->leaf_allocs[i];
    --> ggml_gallocr_init_tensor(galloc, leaf, &leaf_alloc->leaf);
}
// nodes
for (int i = 0; i < graph->n_nodes; i++) {
    struct ggml_tensor * node = graph->nodes[i];
    struct node_alloc * node_alloc = &galloc->node_allocs[i];
    for (int j = 0; j < GGML_MAX_SRC; j++) {
        struct ggml_tensor * src = node->src[j];
        if (src == NULL) continue;
        --> ggml_gallocr_init_tensor(galloc, src, &node_alloc->src[j]);
    }
    --> ggml_gallocr_init_tensor(galloc, node, &node_alloc->dst);
}

return true;
```

[`ggml_gallocr_needs_realloc`](https://github.com/ggml-org/llama.cpp/blob/master/ggml/src/ggml-alloc.c#L828)

```c++
ggml_gallocr_needs_realloc(galloc, graph);
----------
static bool ggml_gallocr_needs_realloc(ggml_gallocr_t galloc, struct ggml_cgraph * graph) {
    if (galloc->n_nodes != graph->n_nodes) return true;
    if (galloc->n_leafs != graph->n_leafs) return true;
    for (int i = 0; i < graph->n_nodes; i++) {
        struct ggml_tensor * node = graph->nodes[i];
        struct node_alloc * node_alloc = &galloc->node_allocs[i];
        --> if (!ggml_gallocr_node_needs_realloc(galloc, node, &node_alloc->dst)) return true;
        for (int j = 0; j < GGML_MAX_SRC; j++) {
            struct ggml_tensor * src = node->src[j];
            if (src == NULL) continue;
            --> if (!ggml_gallocr_node_needs_realloc(galloc, src, &node_alloc->src[j])) return true;
        }
    }
    return false;
}

static bool ggml_gallocr_node_needs_realloc(ggml_gallocr_t galloc, struct ggml_tensor * node, struct tensor_alloc * talloc) {
    size_t node_size = 0;
    if (!node->data && !node->view_src) {
        // If we previously had data but don't now then reallocate
        if (talloc->buffer_id < 0) return false;
        node_size = ggml_backend_buft_get_alloc_size(galloc->bufts[talloc->buffer_id], node);
    }
    return talloc->size_max >= node_size;
}
```

[`ggml_gallocr_init_tensor`](https://github.com/ggml-org/llama.cpp/blob/master/ggml/src/ggml-alloc.c#L787)

```c++
ggml_gallocr_init_tensor(galloc, leaf, &leaf_alloc->leaf);
ggml_gallocr_init_tensor(galloc, src, &node_alloc->src[j]);
ggml_gallocr_init_tensor(galloc, node, &node_alloc->dst);
----------
static void ggml_gallocr_init_tensor(ggml_gallocr_t galloc, struct ggml_tensor * tensor, struct tensor_alloc * tensor_alloc) {
    int buffer_id = tensor_alloc->buffer_id;
    if (tensor->view_src != NULL && tensor->buffer == NULL && tensor->view_src->buffer != NULL)
        --> ggml_backend_view_init(tensor);
    if (tensor->view_src == NULL && tensor->data == NULL)
        void * base = ggml_backend_buffer_get_base(galloc->buffers[buffer_id]);
        void * addr = (char *)base + tensor_alloc->offset;
        --> ggml_backend_tensor_alloc(galloc->buffers[buffer_id], tensor, addr);
}
```

#### [`ggml_backend_cpu_buffer_set_tensor`](https://github.com/ggml-org/llama.cpp/blob/master/ggml/src/ggml-backend.cpp#L1890)

```c++
res->set_inputs(&ubatch);
llm_graph_result::set_inputs -->
    for (auto & input : inputs) input->set_input(ubatch);
    llm_graph_input_embd::set_input;
    llm_graph_input_pos::set_input;

ggml_backend_tensor_set(tokens, ubatch->token, 0, n_tokens*ggml_element_size(tokens));
ggml_backend_tensor_set(pos, ubatch->pos, 0, n_tokens*n_pos_per_embd*ggml_element_size(pos));
----------
void ggml_backend_tensor_set(struct ggml_tensor * tensor, const void * data, size_t offset, size_t size) {
    ggml_backend_buffer_t buf = tensor->view_src ? tensor->view_src->buffer : tensor->buffer;
    if (size == 0) {
        return;
    }
    --> buf->iface.set_tensor(buf, tensor, data, offset, size);
}

static void ggml_backend_cpu_buffer_set_tensor(ggml_backend_buffer_t buffer, struct ggml_tensor * tensor, const void * data, size_t offset, size_t size)
    memcpy((char *)tensor->data + offset, data, size);
```

#### [`llama_kv_cache_unified::set_input_kq_mask`](https://github.com/ggml-org/llama.cpp/blob/master/src/llama-kv-cache-unified.cpp#L794)

```c++
res->set_inputs(&ubatch);
llm_graph_result::set_inputs -->
    for (auto & input : inputs) input->set_input(ubatch);
----------
void llm_graph_input_attn_kv_unified::set_input(const llama_ubatch * ubatch) {
    --> kv_state->set_input_kq_mask(self_kq_mask, ubatch, cparams.causal_attn);
}

void llama_kv_cache_unified_state::set_input_kq_mask(ggml_tensor * dst, const llama_ubatch * ubatch, bool causal_attn) const {
    --> kv->set_input_kq_mask(dst, ubatch, causal_attn);
}

void llama_kv_cache_unified::set_input_kq_mask(ggml_tensor * dst, const llama_ubatch * ubatch, bool causal_attn) const {}
```

```c++
const uint32_t n_tokens     = ubatch->n_tokens;
const uint32_t n_seq_tokens = ubatch->n_seq_tokens;
const uint32_t n_seqs       = ubatch->n_seqs;

float * data = (float *) dst->data;
const int64_t n_kv = dst->ne[0];
// Use only the previous KV cells of the correct sequence for each token of the ubatch.
// It's assumed that if a token in the batch has multiple sequences, they are equivalent.
// Example with a cache of 10 tokens, 2 tokens populated in cache and 3 tokens in batch:
//   Causal mask:
//      xxx-------
//      xxxx------
//      xxxxx-----
//   Non-causal mask:
//      xxxxx-----
//      xxxxx-----
//      xxxxx-----
// To visualize the mask, see https://github.com/ggml-org/llama.cpp/pull/12615
for (uint32_t s = 0; s < n_seqs; ++s) {
    const llama_seq_id seq_id = ubatch->seq_id[s][0];
    // for (uint32_t j = 0; j < n_seq_tokens; ++j)
        // const uint32_t idx = s*n_seq_tokens + j;
    const llama_pos p1 = ubatch->pos[s];
    for (uint32_t i = 0; i < n_kv; ++i) {
        float f = 0.0f;
        bool masked = false;
        if (cells.is_empty(i)) {
            masked = true;
        } else {
            const llama_pos p0 = cells.pos_get(i);
            // mask the token if not the same sequence
            masked = masked || (!cells.seq_has(i, seq_id));
            // mask future tokens
            masked = masked || (causal_attn && p0 > p1);
            // apply SWA if any
            masked = masked || (is_masked_swa(p0, p1));
            if (!masked && hparams.use_alibi) f = -std::abs(p0 - p1);
        }
        if (masked) f = -INFINITY;
        data[n_kv*n_tokens + s*n_kv + i] = f;
    }
}

// mask padded tokens
if (data) {
    for (uint32_t j = n_tokens; j < GGML_PAD(n_tokens, GGML_KQ_MASK_PAD); ++j) {
        for (uint32_t i = 0; i < n_kv; ++i) {
            data[n_kv*n_tokens + j*n_kv + i] = -INFINITY;
        }
    }
}
```

#### ***[`ggml_backend_cpu_graph_compute`](https://github.com/ggml-org/llama.cpp/blob/master/ggml/src/ggml-cpu/ggml-cpu.cpp#L153)

- [`llama_context::graph_compute`](https://github.com/ggml-org/llama.cpp/blob/master/src/llama-context.cpp#L1370)
- [`ggml_backend_sched_compute_splits`](https://github.com/ggml-org/llama.cpp/blob/master/ggml/src/ggml-backend.cpp#L1360)

```c++
const auto status = graph_compute(gf, ubatch.n_tokens > 1);
----------
ggml_status llama_context::graph_compute(
            ggml_cgraph * gf,
                   bool   batched) {
    int n_threads        = batched ? cparams.n_threads_batch : cparams.n_threads;
    
    ggml_threadpool_t tp = batched ? threadpool_batch        : threadpool;
    auto * reg = ggml_backend_dev_backend_reg(ggml_backend_get_device(backend_cpu));
    auto * set_threadpool_fn = (decltype(ggml_backend_cpu_set_threadpool) *) ggml_backend_reg_get_proc_address(reg, "ggml_backend_cpu_set_threadpool");
    set_threadpool_fn(backend_cpu, tp);
        void ggml_backend_cpu_set_threadpool(ggml_backend_t backend_cpu, ggml_threadpool_t threadpool) {}
    
    // set the number of threads for all the backends
    for (const auto & set_n_threads_fn : set_n_threads_fns)
        set_n_threads_fn.second(set_n_threads_fn.first, n_threads);
            void ggml_backend_cpu_set_n_threads(ggml_backend_t backend_cpu, int n_threads) {}
    
    --> auto status = ggml_backend_sched_graph_compute_async(sched.get(), gf);
    return status;
}

enum ggml_status ggml_backend_sched_graph_compute_async(ggml_backend_sched_t sched, struct ggml_cgraph * graph) {
    if (!sched->is_reset && !sched->is_alloc) ggml_backend_sched_reset(sched);
    if (!sched->is_alloc)
        if (!ggml_backend_sched_alloc_graph(sched, graph)) return GGML_STATUS_ALLOC_FAILED;
    --> return ggml_backend_sched_compute_splits(sched);
}

static enum ggml_status ggml_backend_sched_compute_splits(ggml_backend_sched_t sched) {
    struct ggml_backend_sched_split * splits = sched->splits;
    // for (int i = 0; i < sched->n_splits; i++)
    struct ggml_backend_sched_split * split = &splits[0];
    int split_backend_id = split->backend_id;
    ggml_backend_t split_backend = sched->backends[split_backend_id];
    // copy the input tensors to the split backend
    for (int j = 0; j < split->n_inputs; j++) {}

    --> ggml_backend_graph_compute_async(split_backend, &split->graph);

    // record the event of this copy
    if (split->n_inputs > 0 && sched->events[split_backend_id][sched->cur_copy] != NULL) {}

    sched->cur_copy = (sched->cur_copy + 1) % sched->n_copies;
    return GGML_STATUS_SUCCESS;
}

enum ggml_status ggml_backend_graph_compute_async(ggml_backend_t backend, struct ggml_cgraph * cgraph) {
    --> return backend->iface.graph_compute(backend, cgraph);
}

static enum ggml_status ggml_backend_cpu_graph_compute(ggml_backend_t backend, struct ggml_cgraph * cgraph) {}
```

```c++
struct ggml_backend_cpu_context * cpu_ctx = (struct ggml_backend_cpu_context *)backend->context;

--> ggml_graph_plan --> struct ggml_cplan cplan;

if (cpu_ctx->work_size < cplan.work_size) {
    delete[] cpu_ctx->work_data;
    cpu_ctx->work_data = new uint8_t[cplan.work_size];
    cpu_ctx->work_size = cplan.work_size;
}
cplan.work_data = (uint8_t *)cpu_ctx->work_data;
--> return ggml_graph_compute;
```

##### [`ggml_graph_plan`](https://github.com/ggml-org/llama.cpp/blob/master/ggml/src/ggml-cpu/ggml-cpu.c#L2674)

<details>
<summary>struct ggml_cplan</summary>

```c++
// the compute plan that needs to be prepared for ggml_graph_compute()
// since https://github.com/ggml-org/ggml/issues/287
struct ggml_cplan {
    size_t    work_size; // size of work buffer, calculated by `ggml_graph_plan()`
    uint8_t * work_data; // work buffer, to be allocated by caller before calling to `ggml_graph_compute()`

    int n_threads;
    struct ggml_threadpool * threadpool;

    // abort ggml_graph_compute when true
    ggml_abort_callback abort_callback;
    void *              abort_callback_data;
};
```

</details>

```c++
struct ggml_cplan cplan = ggml_graph_plan(cgraph, cpu_ctx->n_threads, cpu_ctx->threadpool);
----------
struct ggml_cplan ggml_graph_plan(
          const struct ggml_cgraph * cgraph,
                               int   n_threads,
            struct ggml_threadpool * threadpool) {}
```

<details>
<summary>struct ggml_type_traits_cpu</summary>

```c++
struct ggml_type_traits_cpu {
    ggml_from_float_t        from_float;
    ggml_vec_dot_t           vec_dot;
    enum ggml_type           vec_dot_type;
    int64_t                  nrows; // number of rows to process simultaneously
};

[GGML_TYPE_Q4_0] = {
    .from_float               = quantize_row_q4_0,
    .vec_dot                  = ggml_vec_dot_q4_0_q8_0,
    .vec_dot_type             = GGML_TYPE_Q8_0,
    .nrows                    = 1,
},
```

</details>

```c++
size_t work_size = 0;
struct ggml_cplan cplan;
memset(&cplan, 0, sizeof(struct ggml_cplan));
int max_tasks = 1;
// thread scheduling for the different operations + work buffer size estimation
for (int i = 0; i < cgraph->n_nodes; i++)
    struct ggml_tensor * node = cgraph->nodes[i];
    const int n_tasks = ggml_get_n_tasks(node, n_threads);
    max_tasks = MAX(max_tasks, n_tasks);
    size_t cur = 0;
    // 暂时将extra功能给禁了
    // if (!ggml_cpu_extra_work_size(n_threads, node, &cur));
    switch (node->op)
        case GGML_OP_MUL_MAT:
            {
                const enum ggml_type vec_dot_type = type_traits_cpu[node->src[0]->type].vec_dot_type;

                if (node->src[1]->type != vec_dot_type) {
                    cur = ggml_row_size(vec_dot_type, ggml_nelements(node->src[1]));
                }
            } break;
        case GGML_OP_SOFT_MAX:
        case GGML_OP_ROPE:
        case GGML_OP_ROPE_BACK:
            {
                cur = ggml_type_size(GGML_TYPE_F32) * node->ne[0] * n_tasks;
            } break;
        case GGML_OP_CPY:
        case GGML_OP_DUP:
            {
                if (ggml_is_quantized(node->type) ||
                    // F16 -> BF16 and BF16 -> F16 copies go through intermediate F32
                    (node->src[0]->type == GGML_TYPE_F16  && node->src[1] && node->src[1]->type == GGML_TYPE_BF16) ||
                    (node->src[0]->type == GGML_TYPE_BF16 && node->src[1] && node->src[1]->type == GGML_TYPE_F16)) {
                    cur = ggml_type_size(GGML_TYPE_F32) * node->ne[0] * n_tasks;
                }
            } break;
        case GGML_OP_ADD:
        case GGML_OP_ADD1:
            {
                if (ggml_is_quantized(node->src[0]->type)) {
                    cur = ggml_type_size(GGML_TYPE_F32) * node->src[0]->ne[0] * n_tasks;
                }
            } break;
    work_size = MAX(work_size, cur);

work_size += CACHE_LINE_SIZE*(n_threads);
cplan.threadpool = threadpool;
cplan.n_threads  = MIN(max_tasks, n_threads);
cplan.work_size  = work_size;
cplan.work_data  = NULL;

return cplan;
```

##### ****[`ggml_graph_compute`](https://github.com/ggml-org/llama.cpp/blob/master/ggml/src/ggml-cpu/ggml-cpu.c#L3118)

<details>
<summary>struct ggml_threadpool</summary>

```c++
// Threadpool def
struct ggml_threadpool {
    ggml_mutex_t mutex;       // mutex for cond.var
    ggml_cond_t  cond;        // cond.var for waiting for new work

    struct ggml_cgraph * cgraph;
    struct ggml_cplan  * cplan;

    // synchronization primitives
    atomic_int n_graph;       // incremented when there is work to be done (i.e each graph)
    atomic_int GGML_CACHE_ALIGN n_barrier;
    atomic_int GGML_CACHE_ALIGN n_barrier_passed;
    atomic_int GGML_CACHE_ALIGN current_chunk; // currently processing chunk during Mat_Mul, shared between all the threads.

    // these are atomic as an annotation for thread-sanitizer
    atomic_bool stop;         // Used for stopping the threadpool altogether
    atomic_bool pause;        // Used for pausing the threadpool or individual threads
    atomic_int abort;         // Used for aborting processing of a graph

    struct ggml_compute_state * workers;   // per thread state
    int          n_threads_max; // number of threads in the pool
    atomic_int   n_threads_cur; // number of threads used in the current graph

    int32_t      prio;        // Scheduling priority
    uint32_t     poll;        // Polling level (0 - no polling)

    enum ggml_status ec;
};

void ggml_threadpool_params_init(struct ggml_threadpool_params * p, int n_threads) {
    p->n_threads  = n_threads;
    p->prio       = 0;     // default priority (usually means normal or inherited)
    p->poll       = 50;    // hybrid-polling enabled
    p->strict_cpu = false; // no strict placement (all threads share same cpumask)
    p->paused     = false; // threads are ready to go
    memset(p->cpumask, 0, GGML_MAX_N_THREADS); // all-zero means use the default affinity (usually inherited)
}

struct ggml_threadpool_params ggml_threadpool_params_default(int n_threads) {
    struct ggml_threadpool_params p;
    ggml_threadpool_params_init(&p, n_threads);
    return p;
}

static struct ggml_threadpool * ggml_threadpool_new_impl(
    struct ggml_threadpool_params * tpp,
               struct ggml_cgraph * cgraph,
                struct ggml_cplan * cplan) {

    struct ggml_threadpool * threadpool =
        ggml_aligned_malloc(sizeof(struct ggml_threadpool));
    {
        threadpool->cgraph           = cgraph;
        threadpool->cplan            = cplan;
        threadpool->n_graph          = 0;
        threadpool->n_barrier        = 0;
        threadpool->n_barrier_passed = 0;
        threadpool->current_chunk    = 0;
        threadpool->stop             = false;
        threadpool->pause            = tpp->paused;
        threadpool->abort            = -1;
        threadpool->workers          = NULL;
        threadpool->n_threads_max    = tpp->n_threads;
        threadpool->n_threads_cur    = tpp->n_threads;
        threadpool->poll             = tpp->poll;
        threadpool->prio             = tpp->prio;
        threadpool->ec               = GGML_STATUS_SUCCESS;
    }

    // Allocate and init workers state
    const size_t workers_size = sizeof(struct ggml_compute_state) * tpp->n_threads;
    struct ggml_compute_state * workers = ggml_aligned_malloc(workers_size);

    memset(workers, 0, workers_size);
    for (int j = 0; j < tpp->n_threads; j++) {
        workers[j].threadpool = threadpool;
        workers[j].ith        = j;
    }

    threadpool->workers = workers;

    return threadpool;
}

void ggml_threadpool_free(struct ggml_threadpool* threadpool) {
    if (!threadpool) return;
    const int n_threads = threadpool->n_threads_max;
    const size_t workers_size = sizeof(struct ggml_compute_state) * n_threads;
    ggml_aligned_free(threadpool->workers, workers_size);
    ggml_aligned_free(threadpool, sizeof(struct ggml_threadpool));
}

```

</details>

<details>
<summary>struct ggml_threadpool_params</summary>

```c++
// threadpool params
// Use ggml_threadpool_params_default() or ggml_threadpool_params_init() to populate the defaults
struct ggml_threadpool_params {
    bool                cpumask[GGML_MAX_N_THREADS]; // mask of cpu cores (all-zeros means use default affinity settings)
    int                 n_threads;                   // number of threads
    enum ggml_sched_priority prio;                   // thread priority
    uint32_t            poll;                        // polling level (0 - no polling, 100 - aggressive polling)
    bool                strict_cpu;                  // strict cpu placement
    bool                paused;                      // start in paused state
};
```

</details>

```c++
ggml_graph_compute(cgraph, &cplan);
----------
enum ggml_status ggml_graph_compute(struct ggml_cgraph * cgraph, struct ggml_cplan * cplan) {}
```

```c++
ggml_cpu_init();
int n_threads = cplan->n_threads;
struct ggml_threadpool * threadpool = cplan->threadpool;
bool disposable_threadpool = false;
if (threadpool == NULL) {
    //GGML_PRINT_DEBUG("Threadpool is not specified. Will create a disposable threadpool : n_threads %d\n", n_threads);
    disposable_threadpool = true;

    struct ggml_threadpool_params ttp = ggml_threadpool_params_default(n_threads);
    threadpool = ggml_threadpool_new_impl(&ttp, cgraph, cplan);
}

if (n_threads > 1) {
    #pragma omp parallel num_threads(n_threads)
    {
        #pragma omp single
        {
            // update the number of threads from the actual number of threads that we got from OpenMP
            n_threads = omp_get_num_threads();
            atomic_store_explicit(&threadpool->n_threads_cur, n_threads, memory_order_relaxed);
        }
        --> ggml_graph_compute_thread(&threadpool->workers[omp_get_thread_num()]);
    }
} else {
    atomic_store_explicit(&threadpool->n_threads_cur, 1, memory_order_relaxed);
    --> ggml_graph_compute_thread(&threadpool->workers[0]);
}
enum ggml_status ret = threadpool->ec;

if (disposable_threadpool) {
    ggml_threadpool_free(threadpool);
}

return ret;
```

###### [`ggml_graph_compute_thread`](https://github.com/ggml-org/llama.cpp/blob/master/ggml/src/ggml-cpu/ggml-cpu.c#L2862)

<details>
<summary>struct ggml_compute_state</summary>

```c++
// Per-thread state
struct ggml_compute_state {
    struct ggml_threadpool * threadpool;
    int ith;
};
```

</details>

<details>
<summary>struct ggml_compute_params</summary>

```c++
struct ggml_compute_params {
    // ith = thread index, nth = number of threads
    int ith, nth;

    // work buffer for all threads
    size_t wsize;
    void * wdata;

    struct ggml_threadpool * threadpool;
};
```

</details>

```c++
static thread_ret_t ggml_graph_compute_thread(void * data) {
    struct ggml_compute_state * state = (struct ggml_compute_state *) data;
    struct ggml_threadpool    * tp    = state->threadpool;

    const struct ggml_cgraph * cgraph = tp->cgraph;
    const struct ggml_cplan  * cplan  = tp->cplan;

    struct ggml_compute_params params = {
        /*.ith       =*/ state->ith,
        /*.nth       =*/ atomic_load_explicit(&tp->n_threads_cur, memory_order_relaxed),
        /*.wsize     =*/ cplan->work_size,
        /*.wdata     =*/ cplan->work_data,
        /*.threadpool=*/ tp,
    };

    for (int node_n = 0; node_n < cgraph->n_nodes && atomic_load_explicit(&tp->abort, memory_order_relaxed) != node_n; node_n++) {
        struct ggml_tensor * node = cgraph->nodes[node_n];
        --> ggml_compute_forward(&params, node);
        if (node_n + 1 < cgraph->n_nodes) {
            --> ggml_barrier(state->threadpool);
        }
    }

    --> ggml_barrier(state->threadpool);

    return 0;
}

static void ggml_compute_forward(struct ggml_compute_params * params, struct ggml_tensor * tensor) {
    if (tensor->op == GGML_OP_NONE || ggml_is_empty(tensor)) return;

    // extra_buffer op? 暂时将extra功能给禁了
    if (ggml_cpu_extra_compute_forward(params, tensor)) return;
    switch (tensor->op) {}
}

void ggml_barrier(struct ggml_threadpool * tp) {
    int n_threads = atomic_load_explicit(&tp->n_threads_cur, memory_order_relaxed);
    if (n_threads == 1) return;

    #pragma omp barrier
}
```

[`ggml_compute_forward_get_rows_q`](ggml/src/ggml-cpu/ops.cpp#L4236)

```c++
ggml_compute_forward_get_rows_q(params, dst);
----------
static void ggml_compute_forward_get_rows_q(
        const ggml_compute_params * params,
              ggml_tensor * dst) {
    const ggml_tensor * src0 = dst->src[0];
    const ggml_tensor * src1 = dst->src[1];

    GGML_TENSOR_BINARY_OP_LOCALS

    const int64_t nc = ne00;
    const int64_t nr = ggml_nelements(src1);

    const ggml_type type = src0->type;
    ggml_to_float_t const dequantize_row_q = ggml_get_type_traits(type)->to_float;

    const int ith = params->ith;
    const int nth = params->nth;

    // rows per thread
    const int dr = (nr + nth - 1)/nth;

    // row range for this thread
    const int ir0 = dr*ith;
    const int ir1 = MIN(ir0 + dr, nr);

    for (int64_t i = ir0; i < ir1; ++i) {
        const int64_t i12 = i/(ne11*ne10);
        const int64_t i11 = (i - i12*ne11*ne10)/ne10;
        const int64_t i10 = (i - i12*ne11*ne10 - i11*ne10);
        const int64_t i01 = *(int32_t *) ((char *) src1->data + i10*nb10 + i11*nb11 + i12*nb12);
        dequantize_row_q(
                (const void *) ((char *) src0->data + i01*nb01 + i11*nb02 + i12*nb03),
                     (float *) ((char *)  dst->data + i10*nb1  + i11*nb2  + i12*nb3), nc);
    }
}

void dequantize_row_q8_0(const block_q8_0 * GGML_RESTRICT x, float * GGML_RESTRICT y, int64_t k) {
    static const int qk = QK8_0;
    const int nb = k / qk;
    for (int i = 0; i < nb; i++) {
        const float d = GGML_FP16_TO_FP32(x[i].d);
        for (int j = 0; j < qk; ++j) {
            y[i*qk + j] = x[i].qs[j]*d;
        }
    }
}
```

[`ggml_compute_forward_rms_norm_f32`](ggml/src/ggml-cpu/ops.cpp#L3270)

```c++
ggml_compute_forward_rms_norm(params, tensor);
----------
static void ggml_compute_forward_rms_norm_f32(
        const ggml_compute_params * params,
        ggml_tensor * dst) {
    const ggml_tensor * src0 = dst->src[0];
    const int ith = params->ith;
    const int nth = params->nth;
    GGML_TENSOR_UNARY_OP_LOCALS
    float eps;
    memcpy(&eps, dst->op_params, sizeof(float));
    // TODO: optimize
    for (int64_t i03 = 0; i03 < ne03; i03++) {
        for (int64_t i02 = 0; i02 < ne02; i02++) {
            for (int64_t i01 = ith; i01 < ne01; i01 += nth) {
                const float * x = (float *) ((char *) src0->data + i01*nb01 + i02*nb02 + i03*nb03);
                ggml_float sum = 0.0;
                for (int64_t i00 = 0; i00 < ne00; i00++) sum += (ggml_float)(x[i00] * x[i00]);
                const float mean = sum/ne00;
                float * y = (float *) ((char *) dst->data + i01*nb1 + i02*nb2 + i03*nb3);
                memcpy(y, x, ne00 * sizeof(float));
                // for (int i00 = 0; i00 < ne00; i00++) {
                //     y[i00] = x[i00];
                // }
                const float scale = 1.0f/sqrtf(mean + eps);
                ggml_vec_scale_f32(ne00, y, scale);
            }
        }
    }
}
```

[`apply_binary_op`](ggml/src/ggml-cpu/binary-ops.cpp#L50)

```c++
ggml_compute_forward_mul(params, tensor);
----------
void ggml_compute_forward_mul(const ggml_compute_params * params, ggml_tensor * dst) {
    binary_op<op_mul>(params, dst);
}
static void binary_op(const ggml_compute_params * params, ggml_tensor * dst)
    const ggml_tensor * src0 = dst->src[0];
    const ggml_tensor * src1 = dst->src[1];
    /*  */ if (src0->type == GGML_TYPE_F32  && src1->type == GGML_TYPE_F32  && dst->type == GGML_TYPE_F32) // all f32
        apply_binary_op<op, float, float, float>(params, dst);

template <float (*op)(float, float), typename src0_t, typename src1_t, typename dst_t>
static void apply_binary_op(const ggml_compute_params * params, ggml_tensor * dst) {
    const ggml_tensor * src0 = dst->src[0];
    const ggml_tensor * src1 = dst->src[1];

    GGML_ASSERT(ggml_can_repeat(src1, src0) && ggml_are_same_shape(src0, dst));

    GGML_TENSOR_BINARY_OP_LOCALS

    GGML_ASSERT( nb0 == sizeof(dst_t));
    GGML_ASSERT(nb00 == sizeof(src0_t));

    const auto [ir0, ir1] = get_thread_range(params, src0);
    const bool is_src1_contiguous = (nb10 == sizeof(src1_t)); // why:-?

    if (!is_src1_contiguous) { // broadcast not implemented yet for non-contiguous
        GGML_ASSERT(ggml_are_same_shape(src0, src1));
    }

    for (int64_t ir = ir0; ir < ir1; ++ir) {
        const int64_t i03 = ir/(ne02*ne01);
        const int64_t i02 = (ir - i03*ne02*ne01)/ne01;
        const int64_t i01 = (ir - i03*ne02*ne01 - i02*ne01);

        const int64_t i13 = i03 % ne13;
        const int64_t i12 = i02 % ne12;
        const int64_t i11 = i01 % ne11;

        dst_t        * dst_ptr  = (dst_t  *)       ((char *)       dst->data  + i03*nb3  + i02*nb2  + i01*nb1 );
        const src0_t * src0_ptr = (const src0_t *) ((const char *) src0->data + i03*nb03 + i02*nb02 + i01*nb01);
        const src1_t * src1_ptr = (const src1_t *) ((const char *) src1->data + i13*nb13 + i12*nb12 + i11*nb11);

        if (is_src1_contiguous) {
            // src1 is broadcastable across src0 and dst in i1, i2, i3
            const int64_t nr0 = ne00 / ne10;

            for (int64_t r = 0; r < nr0; ++r) {
                --> vec_binary_op_contiguous<op>(ne10, dst_ptr + r*ne10, src0_ptr + r*ne10, src1_ptr);
            }
        } else {
            vec_binary_op_non_contiguous<op>(ne0, ne10, nb10, dst_ptr, src0_ptr, src1_ptr);
        }
    }
}

template <float (*op)(float, float), typename src0_t, typename src1_t, typename dst_t>
static inline void vec_binary_op_contiguous(const int64_t n, dst_t * z, const src0_t * x, const src1_t * y) {
    constexpr auto src0_to_f32 = type_conversion_table<src0_t>::to_f32;
    constexpr auto src1_to_f32 = type_conversion_table<src1_t>::to_f32;
    constexpr auto f32_to_dst  = type_conversion_table<dst_t >::from_f32;

    for (int i = 0; i < n; i++) {
        z[i] = f32_to_dst(op(src0_to_f32(x[i]), src1_to_f32(y[i])));
    }
}

static inline float f32_to_f32(float x) {
    return x;
}

template <float (*op)(float, float), typename src0_t, typename src1_t, typename dst_t>
static inline void vec_binary_op_non_contiguous(const int64_t n, const int64_t ne10, const int64_t nb10, dst_t * z, const src0_t * x, const src1_t * y) {
    constexpr auto src0_to_f32 = type_conversion_table<src0_t>::to_f32;
    constexpr auto src1_to_f32 = type_conversion_table<src1_t>::to_f32;
    constexpr auto f32_to_dst  = type_conversion_table<dst_t >::from_f32;

    for (int i = 0; i < n; i++) {
        int i10 = i % ne10;
        const src1_t * y_ptr = (const src1_t *)((const char *)y + i10*nb10);
        z[i] = f32_to_dst(op(src0_to_f32(x[i]), src1_to_f32(*y_ptr)));
    }
}
```

[`ggml_compute_forward_mul_mat`](ggml/src/ggml-cpu/ggml-cpu.c#L1269)


```c++
static void ggml_compute_forward_mul_mat(
        const struct ggml_compute_params * params,
              struct ggml_tensor * dst) {

    const struct ggml_tensor * src0 = dst->src[0];
    const struct ggml_tensor * src1 = dst->src[1];

    GGML_TENSOR_BINARY_OP_LOCALS

    const int ith = params->ith;
    const int nth = params->nth;

    enum ggml_type           const vec_dot_type         = type_traits_cpu[src0->type].vec_dot_type;
    ggml_from_float_t        const from_float           = type_traits_cpu[vec_dot_type].from_float;
    int64_t                  const vec_dot_num_rows     = type_traits_cpu[src0->type].nrows;

    // nb01 >= nb00 - src0 is not transposed
    //   compute by src0 rows

    // TODO: extract to "extra_op"

    // if (src1->type != vec_dot_type) {
    char * wdata = params->wdata;
    const size_t nbw0 = ggml_type_size(vec_dot_type);
    const size_t nbw1 = ggml_row_size(vec_dot_type, ne10);
    const size_t nbw2 = nbw1*ne11;
    const size_t nbw3 = nbw2*ne12;

    for (int64_t i11 = 0; i11 < ne11; ++i11) {
        size_t bs = ggml_blck_size(vec_dot_type);
        int64_t ne10_block_start = (ith * ne10/bs) / nth;
        int64_t ne10_block_end   = ((ith + 1) * ne10/bs) / nth;
        --> from_float(
          (float *)((char *) src1->data + i11*nb11 + ne10_block_start*bs*nb10),
          (void *) (wdata + i11*nbw1 + ne10_block_start*nbw0),
          (ne10_block_end - ne10_block_start) * bs);
    }

    if (ith == 0) {
        // Every thread starts at ith, so the first unprocessed chunk is nth.  This save a bit of coordination right at the start.
        // 没看懂这个是什么意思啊
        // 结合着后面的atomic_fetch_add_explicit原子操作，这下看懂了
        atomic_store_explicit(&params->threadpool->current_chunk, nth, memory_order_relaxed);
    }

    ggml_barrier(params->threadpool);

    // This is the size of the first dimension of the result, so we can iterate that way. (see the ASSERT above, these are the same numbers)
    const int64_t nr0 = ne0;

    // This is the size of the rest of the dimensions of the result
    const int64_t nr1 = ne1 * ne2 * ne3;

    // Now select a reasonable chunk size.
    int chunk_size = 16;

    // We need to step up the size if it's small
    if (nr0 == 1 || nr1 == 1) {
        chunk_size = 64;
    }

    // distribute the work across the inner or outer loop based on which one is larger
    // The number of chunks in the 0/1 dim.
    // CEIL(nr0/chunk_size)
    int64_t nchunk0 = (nr0 + chunk_size - 1) / chunk_size;
    int64_t nchunk1 = (nr1 + chunk_size - 1) / chunk_size;

    // If the chunking is poor for the number of threads on this setup, scrap the whole plan.  Re-chunk it by thread.
    //   Also, chunking by thread was measured to have perform better on NUMA systems.  See https://github.com/ggml-org/llama.cpp/pull/6915
    //   In theory, chunking should be just as useful on NUMA and non NUMA systems, but testing disagreed with that.
    if (nchunk0 * nchunk1 < nth * 4 || ggml_is_numa()) {
        // distribute the thread work across the inner or outer loop based on which one is larger
        nchunk0 = nr0 > nr1 ? nth : 1; // parallelize by src0 rows
        nchunk1 = nr0 > nr1 ? 1 : nth; // parallelize by src1 rows
    }

    // The number of elements in each chunk
    const int64_t dr0 = (nr0 + nchunk0 - 1) / nchunk0;
    const int64_t dr1 = (nr1 + nchunk1 - 1) / nchunk1;

    // The first chunk comes from our thread_id, the rest will get auto-assigned.
    int current_chunk = ith;

    while (current_chunk < nchunk0 * nchunk1) {
        const int64_t ith0 = current_chunk % nchunk0;
        const int64_t ith1 = current_chunk / nchunk0;

        const int64_t ir0_start = dr0 * ith0;
        const int64_t ir0_end = MIN(ir0_start + dr0, nr0);

        const int64_t ir1_start = dr1 * ith1;
        const int64_t ir1_end = MIN(ir1_start + dr1, nr1);

        // dot kernels can handle 1 row and col at a time, but mmla kernels can process 2 rows and cols
        int64_t num_rows_per_vec_dot = vec_dot_num_rows;

        // these checks are needed to avoid crossing dim1 boundaries
        // can be optimized, but the logic would become more complicated, so keeping it like this for simplicity
        if ((nr0 % 2 != 0) || (ne11 % 2 != 0) || ((ir0_end - ir0_start) % 2 != 0) || ((ir1_end - ir1_start) % 2 != 0)) {
            num_rows_per_vec_dot = 1;
        }
        --> ggml_compute_forward_mul_mat_one_chunk(params, dst, src0->type, num_rows_per_vec_dot, ir0_start, ir0_end, ir1_start, ir1_end);

        if (nth >= nchunk0 * nchunk1) {
            break;
        }

        current_chunk = atomic_fetch_add_explicit(&params->threadpool->current_chunk, 1, memory_order_relaxed);
    }
}
```


```c++
static void ggml_compute_forward_mul_mat_one_chunk(
    const struct ggml_compute_params * params,
    struct ggml_tensor * dst,
    const enum ggml_type type,
    const int64_t num_rows_per_vec_dot,
    const int64_t ir0_start,
    const int64_t ir0_end,
    const int64_t ir1_start,
    const int64_t ir1_end) {

    const struct ggml_tensor * src0 = dst->src[0];
    const struct ggml_tensor * src1 = dst->src[1];

    GGML_TENSOR_BINARY_OP_LOCALS

    const bool src1_cont = ggml_is_contiguous(src1);

    ggml_vec_dot_t const vec_dot      = type_traits_cpu[type].vec_dot;
    enum ggml_type const vec_dot_type = type_traits_cpu[type].vec_dot_type;

    // broadcast factors 我们的case貌似不需要考虑？
    // threads with no work simply yield (not sure if it helps)
    if (ir0_start >= ir0_end || ir1_start >= ir1_end) {
        return;
    }

    const void * wdata = params->wdata;
    const size_t row_size = ggml_row_size(vec_dot_type, ne10);

    assert(ne12 % ne02 == 0);
    assert(ne13 % ne03 == 0);

    // block-tiling attempt
    const int64_t blck_0 = 16;
    const int64_t blck_1 = 16;

    // const size_t src1_col_stride = src1_cont || src1->type != vec_dot_type ? row_size : nb11; 貌似我们不用考虑src1不cont的情况
    const size_t src1_col_stride = row_size;

    // attempt to reduce false-sharing (does not seem to make a difference)
    // 16 * 2, accounting for mmla kernels
    // float tmp[32];
    for (int64_t iir1 = ir1_start; iir1 < ir1_end; iir1 += blck_1)
    for (int64_t iir0 = ir0_start; iir0 < ir0_end; iir0 += blck_0)
    // 理想情况下，上面两个循环是没有用的
    for (int64_t ir1 = iir1; ir1 < iir1 + blck_1 && ir1 < ir1_end; ++ir1)
    for (int64_t ir0 = iir0; ir0 < iir0 + blck_0 && ir0 < ir0_end; ++ir0)
        --> vec_dot(
              n=ne00,
              s=(float*)((char*)dst->data + ir0 + ir1 * nb1), //dst_col
              bs=0, // UNUSED
              vx=(const char*)src0->data + ir0 * nb01, //src0_row
              bx=0, // UNUSED
              vy=(const char*)wdata + ir1 * row_size, // src1_col
              by=0, // UNUSED
              nrc=1 // UNUSED
          );
    }
```

[`ggml_vec_dot_q4_0_q8_0`](ggml/src/ggml-cpu/arch/x86/quants.c#L531)

| 特性                | `_mm256_maddubs_epi16`               | `_mm256_madd_epi16`               |
|---------------------|--------------------------------------|-----------------------------------|
| **输入数据类型**    | 无符号 8 位 × 有符号 8 位            | 有符号 16 位 × 有符号 16 位       |
| **输出数据类型**    | 16 位有符号整数（16 个结果）         | 32 位有符号整数（8 个结果）       |
| **运算粒度**        | 8 位 → 16 位                         | 16 位 → 32 位                     |
| **相邻加法**        | 每对 8 位乘积相加                    | 每对 16 位乘积相加                |
| **适用场景**        | 低精度计算（如图像处理）             | 高精度计算（如数值运算）          |

<details>
<summary>original implementation</summary>

```c++
float sumf = 0;
for (ib = 0; ib < nb; ++ib) {
    int sumi0 = 0;
    int sumi1 = 0;

    for (int j = 0; j < qk/2; ++j) {
        const int v0 = (x[ib].qs[j] & 0x0F) - 8;
        const int v1 = (x[ib].qs[j] >>   4) - 8;

        sumi0 += (v0 * y[ib].qs[j]);
        sumi1 += (v1 * y[ib].qs[j + qk/2]);
    }

    int sumi = sumi0 + sumi1;
    sumf += sumi*GGML_FP16_TO_FP32(x[ib].d)*GGML_FP16_TO_FP32(y[ib].d);
}
```

</details>

```c++
void ggml_vec_dot_q4_0_q8_0(int n, float * GGML_RESTRICT s, size_t, const void * GGML_RESTRICT vx, size_t, const void * GGML_RESTRICT vy, size_t, int) {
    const int qk = QK8_0;
    const int nb = n / qk;
    const block_q4_0 * GGML_RESTRICT x = vx;
    const block_q8_0 * GGML_RESTRICT y = vy;
    int ib = 0;
    float sumf = 0;
    // Initialize accumulator with zeros
    __m256 acc = _mm256_setzero_ps();

    // Main loop
    for (; ib < nb; ++ib) {
        /* Compute combined scale for the block */
        const __m256 d = _mm256_set1_ps( GGML_FP16_TO_FP32(x[ib].d) * GGML_FP16_TO_FP32(y[ib].d) );

        --> __m256i qx = bytes_from_nibbles_32(x[ib].qs);

        // Now we have a vector with bytes in [ 0 .. 15 ] interval. Offset them into [ -8 .. +7 ] interval.
        const __m256i off = _mm256_set1_epi8( 8 );
        qx = _mm256_sub_epi8( qx, off );

        __m256i qy = _mm256_loadu_si256((const __m256i *)y[ib].qs);

        --> const __m256 q = mul_sum_i8_pairs_float(qx, qy);

        /* Multiply q with scale and accumulate */
        acc = _mm256_fmadd_ps( d, q, acc );
    }

    --> sumf = hsum_float_8(acc);
    *s = sumf;
}

// Unpack 32 4-bit fields into 32 bytes
// The output vector contains 32 bytes, each one in [ 0 .. 15 ] interval
static inline __m256i bytes_from_nibbles_32(const uint8_t * rsi)
{
    const __m128i tmp = _mm_loadu_si128((const __m128i *)rsi);
    const __m256i bytes = MM256_SET_M128I(_mm_srli_epi16(tmp, 4), tmp);
    const __m256i lowMask = _mm256_set1_epi8( 0xF );
    return _mm256_and_si256(lowMask, bytes);
}

// 4->8->16->32

// multiply int8_t, add results pairwise twice and return as float vector
static inline __m256 mul_sum_i8_pairs_float(const __m256i x, const __m256i y) {
#if __AVXVNNIINT8__
    const __m256i zero = _mm256_setzero_si256();
    const __m256i summed_pairs = _mm256_dpbssd_epi32(zero, x, y);
    return _mm256_cvtepi32_ps(summed_pairs);
#else
    // Get absolute values of x vectors
    const __m256i ax = _mm256_sign_epi8(x, x);
    // Sign the values of the y vectors
    const __m256i sy = _mm256_sign_epi8(y, x);
    return mul_sum_us8_pairs_float(ax, sy);
#endif
}

static inline __m256 mul_sum_us8_pairs_float(const __m256i ax, const __m256i sy) {
#if defined(__AVXVNNI__)
    const __m256i zero = _mm256_setzero_si256();
    // const __m256i summed_pairs = _mm256_dpbusd_epi32(zero, ax, sy);
    const __m256i summed_pairs = _mm256_dpbusd_avx_epi32(zero, ax, sy);
    return _mm256_cvtepi32_ps(summed_pairs);
#else
    // Perform multiplication and create 16-bit values
    const __m256i dot = _mm256_maddubs_epi16(ax, sy);
    const __m256i ones = _mm256_set1_epi16(1);
    const __m256i summed_pairs = _mm256_madd_epi16(ones, dot);
    return _mm256_cvtepi32_ps(summed_pairs);
#endif
}

// horizontally add 8 floats
static inline float hsum_float_8(const __m256 x) {
    __m128 res = _mm256_extractf128_ps(x, 1);
    res = _mm_add_ps(res, _mm256_castps256_ps128(x));
    res = _mm_add_ps(res, _mm_movehl_ps(res, res));
    res = _mm_add_ss(res, _mm_movehdup_ps(res));
    return _mm_cvtss_f32(res);
}
```

[`llamafile_sgemm`](ggml/src/ggml-cpu/llamafile/sgemm.cpp#L3275)

加速比大概是2倍，但是为了简单起见，我们先禁用了。

```c++
llamafile_sgemm(params,
                m=ne01, n=ne11, k=ne00/ggml_blck_size(src0->type),
                A=(const char *)src0->data,
                lda=nb01/ggml_type_size(src0->type),
                B=(const char *)src1->data,
                ldb=nb11/ggml_type_size(src1->type),
                C=(char *)dst->data,
                ldc=nb1/ggml_type_size(dst->type),
                Atype=src0->type,
                Btype=src1->type,
                Ctype=dst->type);

const void* wdata = params->wdata;
const size_t row_size = ggml_row_size(vec_dot_type, ne10);
llamafile_sgemm(params,
                ne01, ne11, ne00/ggml_blck_size(src0->type),
                (const char *)src0->data,
                nb01/ggml_type_size(src0->type),
                (const char *)wdata,
                row_size/ggml_type_size(vec_dot_type),
                (char *)dst->data,
                nb1/ggml_type_size(dst->type),
                src0->type,
                vec_dot_type,
                dst->type);
----------
/**
 * Performs optimized matrix multiplication on CPU.
 *
 * This subroutine may compute C = Aᵀ * B with column major ordering.
 * Despite its name, this isn't a generalized implementation. Work is
 * only performed when a handwritten kernel is written and available.
 * Otherwise the caller should fall back to a general matmul routine.
 *
 * For example, for single-threaded single-precision GEMM you can say
 *
 *     llamafile_sgemm(m, n, k, A, lda, B, ldb, C, ldc,
 *                     0, 1,
 *                     GGML_TYPE_F32, GGML_TYPE_F32, GGML_TYPE_F32);
 *
 * @param m is rows in `A` and `C`
 * @param n is cols in `B` and `C`
 * @param k is cols in `A` and rows in `B`
 * @param A is first input matrix (always transposed)
 * @param lda is row stride of `A`
 * @param B is second input matrix (never transposed)
 * @param ldb is row stride of `B`
 * @param C is input/output array of output matrices
 * @param ldc is row stride of `C`
 * @param ith is thread id (must be less than `nth`)
 * @param nth is number of threads (must be greater than zero)
 * @param Atype is GGML data type of `A`
 * @param Btype is GGML data type of `B`
 * @param Ctype is GGML data type of `C`
 * @return true if this function was able to service the matmul request
 */
bool llamafile_sgemm(const struct ggml_compute_params * params, int64_t m, int64_t n, int64_t k,
                     const void *A, int64_t lda, const void *B, int64_t ldb, void *C,
                     int64_t ldc, int Atype, int Btype, int Ctype) {}
```


```c++
void quantize_row_q8_0(const float * GGML_RESTRICT x, void * GGML_RESTRICT vy, int64_t k) {
    assert(QK8_0 == 32);
    assert(k % QK8_0 == 0);
    const int nb = k / QK8_0;
    block_q8_0 * GGML_RESTRICT y = vy;

    for (int i = 0; i < nb; i++) {
        // Load elements into 4 AVX vectors
        __m256 v0 = _mm256_loadu_ps( x );
        __m256 v1 = _mm256_loadu_ps( x + 8 );
        __m256 v2 = _mm256_loadu_ps( x + 16 );
        __m256 v3 = _mm256_loadu_ps( x + 24 );
        x += 32;
        // Compute max(abs(e)) for the block
        const __m256 signBit = _mm256_set1_ps( -0.0f );
        __m256 maxAbs = _mm256_andnot_ps( signBit, v0 );
        maxAbs = _mm256_max_ps( maxAbs, _mm256_andnot_ps( signBit, v1 ) );
        maxAbs = _mm256_max_ps( maxAbs, _mm256_andnot_ps( signBit, v2 ) );
        maxAbs = _mm256_max_ps( maxAbs, _mm256_andnot_ps( signBit, v3 ) );
        __m128 max4 = _mm_max_ps( _mm256_extractf128_ps( maxAbs, 1 ), _mm256_castps256_ps128( maxAbs ) );
        max4 = _mm_max_ps( max4, _mm_movehl_ps( max4, max4 ) );
        max4 = _mm_max_ss( max4, _mm_movehdup_ps( max4 ) );
        const float maxScalar = _mm_cvtss_f32( max4 );
        // Quantize these floats
        const float d = maxScalar / 127.f;
        y[i].d = GGML_FP32_TO_FP16(d);
        const float id = ( maxScalar != 0.0f ) ? 127.f / maxScalar : 0.0f;
        const __m256 mul = _mm256_set1_ps( id );
        // Apply the multiplier
        v0 = _mm256_mul_ps( v0, mul );
        v1 = _mm256_mul_ps( v1, mul );
        v2 = _mm256_mul_ps( v2, mul );
        v3 = _mm256_mul_ps( v3, mul );
        // Round to nearest integer
        v0 = _mm256_round_ps( v0, _MM_ROUND_NEAREST );
        v1 = _mm256_round_ps( v1, _MM_ROUND_NEAREST );
        v2 = _mm256_round_ps( v2, _MM_ROUND_NEAREST );
        v3 = _mm256_round_ps( v3, _MM_ROUND_NEAREST );
        // Convert floats to integers
        __m256i i0 = _mm256_cvtps_epi32( v0 );
        __m256i i1 = _mm256_cvtps_epi32( v1 );
        __m256i i2 = _mm256_cvtps_epi32( v2 );
        __m256i i3 = _mm256_cvtps_epi32( v3 );
        // Convert int32 to int16
        i0 = _mm256_packs_epi32( i0, i1 );	// 0, 1, 2, 3,  8, 9, 10, 11,  4, 5, 6, 7, 12, 13, 14, 15
        i2 = _mm256_packs_epi32( i2, i3 );	// 16, 17, 18, 19,  24, 25, 26, 27,  20, 21, 22, 23, 28, 29, 30, 31
                                            // Convert int16 to int8
        i0 = _mm256_packs_epi16( i0, i2 );	// 0, 1, 2, 3,  8, 9, 10, 11,  16, 17, 18, 19,  24, 25, 26, 27,  4, 5, 6, 7, 12, 13, 14, 15, 20, 21, 22, 23, 28, 29, 30, 31
        // We got our precious signed bytes, but the order is now wrong
        // These AVX2 pack instructions process 16-byte pieces independently
        // The following instruction is fixing the order
        const __m256i perm = _mm256_setr_epi32( 0, 4, 1, 5, 2, 6, 3, 7 );
        i0 = _mm256_permutevar8x32_epi32( i0, perm );
        _mm256_storeu_si256((__m256i *)y[i].qs, i0);
    }
}

```

### [`ggml_backend_tensor_get_async`](https://github.com/ggml-org/llama.cpp/blob/master/ggml/src/ggml-backend.cpp#L245)

```c++
ggml_backend_tensor_get_async(backend_res, t_logits, logits_out, 0, n_outputs*n_vocab*sizeof(float));
----------
void ggml_backend_tensor_get_async(ggml_backend_t backend, const struct ggml_tensor * tensor, void * data, size_t offset, size_t size) {}
```

```c++
if (backend->iface.get_tensor_async == NULL) {
    --> ggml_backend_tensor_get(tensor, data, offset, size);
} else {
    backend->iface.get_tensor_async(backend, tensor, data, offset, size);
}
```

[`ggml_backend_cpu_buffer_get_tensor`](https://github.com/ggml-org/llama.cpp/blob/master/ggml/src/ggml-backend.cpp#L1896)

```c++
void ggml_backend_tensor_get(const struct ggml_tensor * tensor, void * data, size_t offset, size_t size) {
    ggml_backend_buffer_t buf = tensor->view_src ? tensor->view_src->buffer : tensor->buffer;
    --> buf->iface.get_tensor(buf, tensor, data, offset, size);
}
----------
static void ggml_backend_cpu_buffer_get_tensor(ggml_backend_buffer_t buffer, const struct ggml_tensor * tensor, void * data, size_t offset, size_t size)
    memcpy(data, (const char *)tensor->data + offset, size);
```

# [`perplexity`](https://github.com/ggml-org/llama.cpp/blob/master/tools/perplexity/perplexity.cpp#L441)

```c++
results = perplexity(ctx, params, n_ctx);
----------
static results_perplexity perplexity(llama_context * ctx, const common_params & params, const int32_t n_ctx) {}
```

```c++
// Download: https://huggingface.co/datasets/ggml-org/ci/resolve/main/wikitext-2-raw-v1.zip
// Run `./llama-perplexity -m models/7B/ggml-model-q4_0.bin -f wiki.test.raw`
// Output: `perplexity: 13.5106 [114/114]`
// BOS tokens will be added for each chunk before eval

const llama_model * model = llama_get_model(ctx);
const llama_vocab * vocab = llama_model_get_vocab(model);
const bool add_bos = llama_vocab_get_add_bos(vocab);
--> common_tokenize --> llama_tokenize --> vocab->tokenize --> llama_vocab::tokenize --> llama_vocab::impl::tokenize --> std::vector<llama_token> tokens;

std::vector<float> logit_history;
logit_history.resize(tokens.size());

std::vector<float> prob_history;
prob_history.resize(tokens.size());

const int n_chunk_max = tokens.size() / n_ctx;
const int n_chunk = params.n_chunks < 0 ? n_chunk_max : std::min(params.n_chunks, n_chunk_max);
const int n_batch = params.n_batch;
const int n_vocab = llama_vocab_n_tokens(vocab);
int count = 0;
double nll = 0.0;
double nll2 = 0.0;
const int num_batches = (n_ctx + n_batch - 1) / n_batch;
const int n_seq = std::max(1, n_batch / n_ctx);
--> llama_batch_init --> llama_batch batch;
std::vector<std::thread> workers(std::thread::hardware_concurrency() - 1);

// We get the logits for all the tokens in the context window (params.n_ctx)
// from llama_eval above.  Now, based on https://huggingface.co/docs/transformers/perplexity,
// calculate the perplexity over the last half of the window (so the model always has
// some context to predict the token).
//
// We rely on the fact that attention in the forward pass only looks at previous
// tokens here, so the logits returned for each token are an accurate representation
// of what the model would have predicted at that point.
//
// Example, we have a context window of 512, we will compute perplexity for each of the
// last 256 tokens.  Then, we split the input up into context window size chunks to
// process the entire prompt.
const int first = n_ctx/2;
for (int i = 0; i < n_chunk; i += n_seq)
    const int start =     i * n_ctx;
    const int end   = start + n_ctx;
    const int n_seq_batch = std::min(n_seq, n_chunk - i);
    // clear the KV cache
    llama_memory_clear(llama_get_memory(ctx), true);
    // for (int j = 0; j < num_batches; ++j)
    const int batch_start = start + j * n_batch;
    const int batch_size  = std::min(end - batch_start, n_batch);
    int n_outputs = 0;
    batch.n_tokens = 0;
    for (int seq = 0; seq < n_seq_batch; seq++)
        int seq_start = batch_start + seq*n_ctx;
        // save original token and restore it after eval
        const auto token_org = tokens[seq_start];
        if (add_bos && j == 0) tokens[seq_start] = llama_vocab_bos(vocab);
        for (int k = 0; k < batch_size; ++k) {
            const int idx = seq*n_ctx + k;
            batch.token   [idx]    = tokens[seq_start + k];
            batch.pos     [idx]    = j*n_batch + k;
            batch.n_seq_id[idx]    = 1;
            batch.seq_id  [idx][0] = seq;
            batch.logits  [idx]    = batch.pos[idx] >= first ? 1 : 0;

            n_outputs += batch.logits[idx] != 0;
        }
        batch.n_tokens += batch_size;
        // restore the original token in case it was set to BOS
        tokens[seq_start] = token_org;

    --> llama_decode(ctx, batch);
    if (i == 0) llama_synchronize(ctx);

    for (int seq = 0; seq < n_seq_batch; seq++)
        const float * all_logits = llama_get_logits_ith(ctx, seq*n_ctx + first);
            ctx->synchronize();
            return ctx:: logits + output_ids[seq*n_ctx + first]*model.vocab.n_tokens();
        llama_token * tokens_data = tokens.data() + start + seq*n_ctx + first;
        --> process_logits(n_vocab, all_logits,
                tokens_data, n_ctx - 1 - first,
                workers, nll, nll2,
                logit_history.data() + start + seq*n_ctx + first,
                prob_history.data()  + start + seq*n_ctx + first);
        count += n_ctx - first - 1;

nll2 /= count;
nll /= count;
const double ppl = exp(nll);
nll2 -= nll * nll;
nll2 = sqrt(nll2/(count-1));

--> llama_batch_free;
return {tokens, ppl, logit_history, prob_history};
```

## [`llama_vocab::impl::tokenize`](https://github.com/ggml-org/llama.cpp/blob/master/src/llama-vocab.cpp#L2389)

```c++
std::vector<llama_token> tokens = common_tokenize(ctx, params.prompt, true);
----------
```

```c++
// tokenizes a string into a vector of tokens
// should work similar to Python's `tokenizer.encode`
std::vector<llama_token> common_tokenize(
  const struct llama_context * ctx,
           const std::string & text,
                        bool   add_special,
                        bool   parse_special = false) {
    const llama_model * model = llama_get_model(ctx);
    const llama_vocab * vocab = llama_model_get_vocab(model);
    --> return common_tokenize(vocab, text, add_special, parse_special);
}

std::vector<llama_token> common_tokenize(
    const struct llama_vocab * vocab,
           const std::string & text,
                        bool   add_special,
                        bool   parse_special) {
    // upper limit for the number of tokens
    int n_tokens = text.length() + 2 * add_special;
    std::vector<llama_token> result(n_tokens);
    --> n_tokens = llama_tokenize(vocab, text.data(), text.length(), result.data(), result.size(), add_special, parse_special);
    result.resize(n_tokens);
    return result;
}

int32_t llama_tokenize(
    const struct llama_vocab * vocab,
                  const char * text,
                     int32_t   text_len,
                 llama_token * tokens,
                     int32_t   n_tokens_max,
                        bool   add_special,
                        bool   parse_special) {
    --> return vocab->tokenize(text, text_len, tokens, n_tokens_max, add_special, parse_special);
}

int32_t llama_vocab::tokenize(
                  const char * text,
                     int32_t   text_len,
                 llama_token * tokens,
                     int32_t   n_tokens_max,
                        bool   add_special,
                        bool   parse_special) const {
    --> auto res = tokenize(std::string(text, text_len), add_special, parse_special);
    for (size_t i = 0; i < res.size(); i++) {
        tokens[i] = res[i];
    }
    return res.size();
}

std::vector<llama_token> llama_vocab::tokenize(
        const std::string & raw_text,
        bool add_special,
        bool parse_special) const {
    --> return pimpl->tokenize(raw_text, add_special, parse_special);
}

std::vector<llama_token> llama_vocab::impl::tokenize(
        const std::string & raw_text,
        bool add_special,
        bool parse_special) const {}
```

```c++
std::vector<llama_token> output;
std::forward_list<fragment_buffer_variant> fragment_buffer;

fragment_buffer.emplace_front(raw_text, 0, raw_text.length());
tokenizer_st_partition(fragment_buffer, parse_special);

llm_tokenizer_bpe_session session(vocab, *static_cast<const llm_tokenizer_bpe *>(tokenizer.get()));
// it calls some other methods that are not exist in llm_tokenizer,
// here just cast it to bpe tokenizer object
if (add_special) {
    session.append_bos(output);
}
for (const auto & fragment : fragment_buffer) {
    std::string text = fragment.raw_text.substr(fragment.offset, fragment.length);
    --> llm_tokenizer_bpe_session::tokenize;
}

if (add_special) {
    session.append_eos(output);
    session.check_double_bos_eos(output);
}

return output;
```

[`llm_tokenizer_bpe_session::tokenize`](https://github.com/ggml-org/llama.cpp/blob/master/src/llama-vocab.cpp#L482)

<details>
<summary>struct llm_tokenizer_bpe_session</summary>

```c++
struct llm_tokenizer_bpe_session {
    llm_tokenizer_bpe_session(const llama_vocab & vocab, const llm_tokenizer_bpe & tokenizer) : vocab(vocab), tokenizer(tokenizer) {}

    static void append(const llama_token token_id, std::vector<llama_token> & output)  {
    bool append_bos(std::vector<llama_token> & output) const
    bool append_eos(std::vector<llama_token> & output) const
    void check_double_bos_eos(const std::vector<llama_token> & output) const
    void tokenize(const std::string & text, std::vector<llama_token> & output)

private:
    void add_new_bigram(int left, int right)
    const llama_vocab & vocab;
    const llm_tokenizer_bpe & tokenizer;

    std::vector<llm_symbol> symbols;
    std::vector<llm_symbol> symbols_final;
    llm_bigram_bpe::queue work_queue;
}
```

</details>


```c++
session.tokenize(text, output);
----------
void llm_tokenizer_bpe_session::tokenize(const std::string & text, std::vector<llama_token> & output) {}
```

## [`llama_batch_init`](https://github.com/ggml-org/llama.cpp/blob/master/src/llama-batch.cpp#L339)

```c++
llama_batch batch = llama_batch_init(std::min(n_batch, n_ctx*n_seq), 0, 1);
----------
struct llama_batch llama_batch_init(int32_t n_tokens_alloc, int32_t embd, int32_t n_seq_max) {}
```

```c++
llama_batch batch = {
    /*n_tokens       =*/ 0,
    /*tokens         =*/ nullptr,
    /*embd           =*/ nullptr,
    /*pos            =*/ nullptr,
    /*n_seq_id       =*/ nullptr,
    /*seq_id         =*/ nullptr,
    /*logits         =*/ nullptr,
};

if (embd) {
    batch.embd = (float *) malloc(sizeof(float) * n_tokens_alloc * embd);
} else {
    batch.token = (llama_token *) malloc(sizeof(llama_token) * n_tokens_alloc);
}

batch.pos      = (llama_pos *)     malloc(sizeof(llama_pos)      * n_tokens_alloc);
batch.n_seq_id = (int32_t *)       malloc(sizeof(int32_t)        * n_tokens_alloc);
batch.seq_id   = (llama_seq_id **) malloc(sizeof(llama_seq_id *) * (n_tokens_alloc + 1));
for (int i = 0; i < n_tokens_alloc; ++i) {
    batch.seq_id[i] = (llama_seq_id *) malloc(sizeof(llama_seq_id) * n_seq_max);
}
batch.seq_id[n_tokens_alloc] = nullptr;

batch.logits   = (int8_t *)        malloc(sizeof(int8_t)         * n_tokens_alloc);

return batch;
```

## [`process_logits`](https://github.com/ggml-org/llama.cpp/blob/master/tools/perplexity/perplexity.cpp#L107)

```c++
process_logits(n_vocab, all_logits,
        tokens_data, n_ctx - 1 - first,
        workers, nll, nll2,
        logit_history.data() + start + seq*n_ctx + first,
        prob_history.data()  + start + seq*n_ctx + first);
----------
static void process_logits(
    int n_vocab, const float * logits, const int * tokens, int n_token, std::vector<std::thread> & workers,
    double & nll, double & nll2, float * logit_history, float * prob_history) {}
```

```c++
std::mutex mutex;
int counter = 0;
auto compute = [&mutex, &counter, &nll, &nll2, logit_history, prob_history, n_vocab, logits, tokens, n_token] () {
    double local_nll  = 0;
    double local_nll2 = 0;
    while (true) {
        std::unique_lock<std::mutex> lock(mutex);
        int i = counter++;
        if (i >= n_token) {
            nll += local_nll; nll2 += local_nll2;
            break;
        }
        lock.unlock();
        --> log_softmax --> const results_log_softmax results;
        const double v = -results.log_softmax;
        local_nll += v;
        local_nll2 += v*v;

        logit_history[i] = results.logit;
        prob_history[i]  = results.prob;
    }
};
for (auto & w : workers) {
    w = std::thread(compute);
}
compute();
for (auto & w : workers) {
    w.join();
}
```

[`log_softmax`](https://github.com/ggml-org/llama.cpp/blob/master/tools/perplexity/perplexity.cpp#L58)

<details>
<summary>struct results_log_softmax</summary>

```c++
struct results_log_softmax {
    double log_softmax;
    float  logit;
    float  prob;
};
```

</details>


```c++
const results_log_softmax results = log_softmax(n_vocab, logits + size_t(i)*n_vocab, tokens[i+1]);
----------
static results_log_softmax log_softmax(int n_vocab, const float * logits, int tok) {
    float max_logit = logits[0];
    for (int i = 1; i < n_vocab; ++i) {
        max_logit = std::max(max_logit, logits[i]);
    }
    double sum_exp = 0.0;
    for (int i = 0; i < n_vocab; ++i) {
        sum_exp += expf(logits[i] - max_logit);
    }
    return {logits[tok] - max_logit - log(sum_exp), logits[tok], expf(logits[tok] - max_logit) / (float) sum_exp};
}

```

## [`llama_batch_free`](https://github.com/ggml-org/llama.cpp/blob/master/src/llama-batch.cpp#L369)

```c++
llama_batch_free(batch);
----------
void llama_batch_free(struct llama_batch batch) {}
```

```c++
if (batch.token)    free(batch.token);
if (batch.embd)     free(batch.embd);
if (batch.pos)      free(batch.pos);
if (batch.n_seq_id) free(batch.n_seq_id);
if (batch.seq_id) {
    for (int i = 0; batch.seq_id[i] != nullptr; ++i) {
        free(batch.seq_id[i]);
    }
    free(batch.seq_id);
}
if (batch.logits)   free(batch.logits);
```

# [`ggml_quantize_free`](https://github.com/ggml-org/llama.cpp/blob/master/ggml/src/ggml.c#L6457)

```c++
llama_backend_free();
----------
void llama_backend_free(void) {
    --> ggml_quantize_free();
}

void ggml_quantize_free(void) {
    ggml_critical_section_start();

    iq2xs_free_impl(GGML_TYPE_IQ2_XXS);
    iq2xs_free_impl(GGML_TYPE_IQ2_XS);
    iq2xs_free_impl(GGML_TYPE_IQ1_S);
    iq3xs_free_impl(256);

    ggml_critical_section_end();
}

void iq2xs_free_impl(enum ggml_type type)
    const int gindex = iq2_data_index(type);
    if (iq2_data[gindex].grid) {
        free(iq2_data[gindex].grid);       iq2_data[gindex].grid = NULL;
        free(iq2_data[gindex].map);        iq2_data[gindex].map  = NULL;
        free(iq2_data[gindex].neighbours); iq2_data[gindex].neighbours = NULL;
    }

void iq3xs_free_impl(int grid_size)
    const int gindex = iq3_data_index(grid_size);
    if (iq3_data[gindex].grid) {
        free(iq3_data[gindex].grid);       iq3_data[gindex].grid = NULL;
        free(iq3_data[gindex].map);        iq3_data[gindex].map  = NULL;
        free(iq3_data[gindex].neighbours); iq3_data[gindex].neighbours = NULL;
    }
```